In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import random
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from mlxtend.frequent_patterns import apriori, association_rules
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
from torch.optim import AdamW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress warnings from mlxtend regarding DataFrame types
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# --- Helper Functions ---

def clean_text(text):
    """Cleans clinical notes text by lowercasing and standardizing whitespace."""
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def create_sequences(data, target, seq_length=10):
    """Creates time-series sequences using a sliding window technique."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(target.iloc[i+seq_length])
    return np.array(X), np.array(y)

class FeedbackDataset(torch.utils.data.Dataset):
    """Custom Dataset for BERT sentiment analysis."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def get_embedding(text, tokenizer_bert, model_bert):
    """Generates BERT embeddings for a given text."""
    inputs = tokenizer_bert(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    # --- FIX: Move input tensors to GPU if model is on GPU ---
    if torch.cuda.is_available():
        inputs = {k: v.to('cuda') for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_bert(**inputs)

    # CLS token embedding
    # --- FIX: Move tensor to CPU before converting to numpy ---
    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

# --- Module 1: Synthetic Patient Data - Classification, Regression, Clustering ---

# Load dataset
df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")

# Feature Engineering
df_synthetic['BP_level'] = pd.cut(
    df_synthetic['systolic_bp'],
    bins=[0, 80, 120, 200],
    labels=['Low', 'Normal', 'High']
)
df_synthetic['medication_history'] = ((df_synthetic['diabetes'] == 1) | (df_synthetic['hypertension'] == 1) | (df_synthetic['smoker'] == 1)).astype(int)

# Data Preprocessing
imputer = SimpleImputer(strategy='median')
num_cols_synthetic = ['bmi','systolic_bp','cholesterol','blood_glucose']
df_synthetic[num_cols_synthetic] = imputer.fit_transform(df_synthetic[num_cols_synthetic])

for col in num_cols_synthetic:
    Q1 = df_synthetic[col].quantile(0.25)
    Q3 = df_synthetic[col].quantile(0.75)
    IQR = Q3 - Q1
    df_synthetic = df_synthetic[(df_synthetic[col] >= Q1 - 1.5*IQR) & (df_synthetic[col] <= Q3 + 1.5*IQR)]

le_synthetic = LabelEncoder()
df_synthetic['BP_level'] = le_synthetic.fit_transform(df_synthetic['BP_level'])
df_synthetic['risk_category'] = le_synthetic.fit_transform(df_synthetic['risk_category'])

scaler_synthetic = StandardScaler()
df_synthetic[num_cols_synthetic] = scaler_synthetic.fit_transform(df_synthetic[num_cols_synthetic])

# Classification Model
X_clf = df_synthetic.drop(['risk_category','length_of_stay_days', 'gender'], axis=1) # Exclude 'gender' from KBest if not suitable
y_clf = df_synthetic['risk_category']

# Ensure 'gender' is encoded before feature selection for X_clf if it's still an object type
if 'gender' in X_clf.columns and X_clf['gender'].dtype == 'object':
    le_gender_clf = LabelEncoder()
    X_clf['gender'] = le_gender_clf.fit_transform(X_clf['gender'])

selector = SelectKBest(score_func=f_classif, k=5)
X_selected_clf = selector.fit_transform(X_clf, y_clf)

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_selected_clf, y_clf, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_clf, y_train_clf)
y_pred_clf = clf.predict(X_test_clf)

print("--- Classification Report (Risk Category) ---")
print(classification_report(y_test_clf, y_pred_clf))
print("Confusion Matrix:\n", confusion_matrix(y_test_clf, y_pred_clf))

# Streamlit Placeholder: Display classification results and allow new patient risk prediction.
# Example: st.write("Patient Risk Category Prediction:", clf.predict(new_patient_features_scaled))


# Regression Model
features_reg = [col for col in df_synthetic.columns if col not in ['patient_id', 'risk_category']]
X_reg = df_synthetic[features_reg].copy()
y_reg = df_synthetic["length_of_stay_days"]

if 'gender' in X_reg.columns and X_reg['gender'].dtype == 'object':
    le_gender_reg = LabelEncoder()
    X_reg['gender'] = le_gender_reg.fit_transform(X_reg['gender'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

pipeline_reg = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=1.0))
])
pipeline_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = pipeline_reg.predict(X_test_reg)

print("\n--- Regression Metrics (Length of Stay) ---")
print("MAE:", mean_absolute_error(y_test_reg, y_pred_reg))
print("RMSE:", np.sqrt(mean_squared_error(y_test_reg, y_pred_reg)))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg))

# Streamlit Placeholder: Display regression metrics and allow prediction of length of stay.
# Example: st.write("Predicted Length of Stay:", pipeline_reg.predict(new_patient_features_scaled_for_reg))


# Clustering Model
X_cluster = df_synthetic.drop(['risk_category','length_of_stay_days'], axis=1)

if 'gender' in X_cluster.columns and X_cluster['gender'].dtype == 'object':
    le_gender_cluster = LabelEncoder()
    X_cluster['gender'] = le_gender_cluster.fit_transform(X_cluster['gender'])

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_synthetic['cluster'] = kmeans.fit_predict(X_cluster)

print("\n--- Clustering Results (First 5 rows with cluster) ---")
print(df_synthetic[['bmi','systolic_bp','cholesterol','cluster']].head())
print("Silhouette Score:", silhouette_score(X_cluster, df_synthetic["cluster"]))

cluster_risk_crosstab = pd.crosstab(df_synthetic['cluster'], df_synthetic['risk_category'])
print("\nCrosstabulation of Cluster vs. Risk Category:")
print(cluster_risk_crosstab)

# Streamlit Placeholder: Display cluster analysis and allow patient assignment to a cluster.
# Example: st.write("Patient Cluster:", kmeans.predict(new_patient_features_scaled_for_cluster))


# --- Module 2: Association Rules ---

# Load dataset
df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")

# Feature Engineering (already binary, drop patient_id)
df_assoc = df_apriori.drop('patient_id', axis=1)

# Generate frequent itemsets
# Use boolean DataFrame for apriori
df_assoc_bool = df_assoc.astype(bool)
frequent_itemsets = apriori(
    df_assoc_bool,
    min_support=0.1,
    use_colnames=True
)

print("\n--- Frequent Itemsets (Association Rules) ---")
print(frequent_itemsets.sort_values('support', ascending=False).head())

# Generate association rules
rules = association_rules(
    frequent_itemsets,
    metric='confidence',
    min_threshold=0.6
)

print("\n--- Top 10 Association Rules by Lift ---")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(10))

# Streamlit Placeholder: Display association rules and highlight key medical insights.
# Example: st.dataframe(rules)


# --- Module 3: Sequence Modeling (LSTM) ---

# Load dataset
df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')

# Feature Selection and Scaling
FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
TARGET_ts = 'risk_flag'

scaler_ts = MinMaxScaler()
df_timeseries[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries[FEATURES_ts])

# Create time-series sequences
SEQ_LENGTH = 10
X_ts, y_ts = create_sequences(df_timeseries[FEATURES_ts], df_timeseries[TARGET_ts], SEQ_LENGTH)

# Train-test split
X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(
    X_ts, y_ts, test_size=0.2, random_state=42, stratify=y_ts
)

# Build LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n--- LSTM Model Summary ---")
model_lstm.summary()

# Train Model
print("\n--- Training LSTM Model ---")
history_lstm = model_lstm.fit(
    X_train_ts, y_train_ts,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

# Model Evaluation
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test_ts, y_test_ts, verbose=0)
print(f"LSTM Test Accuracy: {accuracy_lstm:.4f}")

# Prediction
y_pred_prob_lstm = model_lstm.predict(X_test_ts, verbose=0)
y_pred_lstm = (y_pred_prob_lstm > 0.5).astype(int)

print("First 10 LSTM predictions (0=Stable, 1=High deterioration risk):", y_pred_lstm[:10].flatten())

# Streamlit Placeholder: Display LSTM prediction for patient deterioration risk.
# Example: st.write("Deterioration Risk (LSTM):")


# --- Module 4: Sentiment Analysis (BERT) ---

# Load dataset
df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")

texts_feedback = df_feedback["feedback_text"].tolist()
labels_feedback = LabelEncoder().fit_transform(df_feedback["sentiment"])

# Tokenization
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
encodings_feedback = tokenizer_bert(
    texts_feedback,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

# Create Dataset
train_idx_feedback, val_idx_feedback = train_test_split(
    range(len(labels_feedback)), test_size=0.2, random_state=42
)
train_dataset_feedback = FeedbackDataset(
    {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
    labels_feedback[train_idx_feedback]
)
# val_dataset_feedback is not used in the training loop but good to have for evaluation if needed.

# Model
model_sentiment = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(set(labels_feedback))
)

# Training (Simplified loop as in original notebook, without full Trainer)
optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
model_sentiment.train()

print("\n--- Training BERT Sentiment Model ---")
for epoch in range(2):
    for batch in torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8):
        optimizer_sentiment.zero_grad()
        # Move batch to GPU if available
        if torch.cuda.is_available():
            batch = {k: v.to('cuda') for k, v in batch.items()}
            model_sentiment.to('cuda')

        outputs_sentiment = model_sentiment(**batch)
        loss_sentiment = outputs_sentiment.loss
        loss_sentiment.backward()
        optimizer_sentiment.step()
    print(f"Epoch {epoch+1} Loss (BERT Sentiment):", loss_sentiment.item())

# Prediction Example
model_sentiment.eval()
test_text_sentiment = "Hospital staff was rude"
inputs_sentiment = tokenizer_bert(test_text_sentiment, return_tensors="pt")
if torch.cuda.is_available():
    inputs_sentiment = {k: v.to('cuda') for k, v in inputs_sentiment.items()}
    model_sentiment.to('cuda')

outputs_sentiment_pred = model_sentiment(**inputs_sentiment)
pred_sentiment = torch.argmax(outputs_sentiment_pred.logits, dim=1)
print("Predicted sentiment for 'Hospital staff was rude':", pred_sentiment.item()) # 0: Negative, 1: Neutral, 2: Positive

# Streamlit Placeholder: Allow users to input feedback and get sentiment prediction.
# Example: st.write("Sentiment:", sentiment_labels[pred_sentiment.item()])


# --- Module 5: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---

# Load dataset
# Note: This file is currently missing from the environment and will cause a FileNotFoundError.
# If you wish to execute this module, please ensure '/content/healthai_clinical_notes_1000.csv' is uploaded.
try:
    df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
    df_clinical["clinical_note"] = df_clinical["clinical_note"].astype(str).apply(clean_text)

    # BioBERT for Embeddings
    MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
    tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
    model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)

    # Move model to GPU if available
    if torch.cuda.is_available():
        model_biobert_embeddings.to('cuda')

    # Generate embeddings
    embeddings_biobert = np.vstack(df_clinical["clinical_note"].apply(
        lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings).flatten()
    ))
    print("\nEmbedding shape (BioBERT):", embeddings_biobert.shape)

    # KMeans Clustering on Embeddings
    kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
    df_clinical["cluster"] = kmeans_biobert.fit_predict(embeddings_biobert)

    print("\n--- Clinical Notes with Cluster (First 5 rows) ---")
    print(df_clinical[["clinical_note", "cluster"]].head())

    # BioGPT for Text Generation
    generator_biogpt = pipeline(
        "text-generation",
        model="microsoft/BioGPT",
        device=0 if torch.cuda.is_available() else -1 # Use GPU if available
    )

    prompt_biogpt = "Patient presents with chest pain and shortness of breath. Clinical impression:"
    generated_text_biogpt = generator_biogpt(prompt_biogpt, max_length=80, num_return_sequences=1)[0]["generated_text"]
    print("\n--- BioGPT Generated Clinical Impression ---")
    print(generated_text_biogpt)

except FileNotFoundError:
    print("\n--- Skipping Module 5: Generative AI ---")
    print("Error: '/content/healthai_clinical_notes_1000.csv' not found. Please upload the file to run this module.")
    # Define placeholder/dummy functions/variables to allow subsequent code to run if it depends on them
    generator_biogpt = None # Placeholder if BioGPT is not loaded
    print("\n--- Module 6: Healthcare Chatbot and Translator --- will proceed with generic chatbot if BioGPT is unavailable. ---")


# --- Module 6: Healthcare Chatbot and Translator ---

# Data generation for chatbot (if the file doesn't exist already)
# This part ensures the CSV is available for later use.
if not os.path.exists("/content/healthcare_chatbot_translation_dataset.csv"): # Corrected path for checking existence
    symptoms = [
        "fever", "cough", "headache", "chest pain", "breathing difficulty",
        "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
    ]
    questions = [
        "I have fever and cough, what should I do?",
        "Is chest pain serious?",
        "How to control blood sugar?",
        "I feel tired all the time",
        "Can I take paracetamol daily?",
        "When should I see a doctor?",
        "Is headache dangerous?",
        "How to reduce BP naturally?",
    ]
    responses = [
        "Please consult a physician and take rest.",
        "Monitor symptoms and seek emergency care if pain increases.",
        "Maintain diet, exercise and medication regularly.",
        "Blood tests may be required.",
        "Avoid self-medication without advice.",
    ]
    languages = ["English", "Tamil", "Hindi", "Telugu"]

    data_chatbot = []
    for i in range(1000):
        data_chatbot.append({
            "symptom": random.choice(symptoms),
            "patient_question": random.choice(questions),
            "doctor_reply": random.choice(responses),
            "language": random.choice(languages),
            "appointment_needed": random.choice(["Yes", "No"])
        })
    df_chatbot = pd.DataFrame(data_chatbot)
    df_chatbot.to_csv("/content/healthcare_chatbot_translation_dataset.csv", index=False)
else:
    df_chatbot = pd.read_csv("/content/healthcare_chatbot_translation_dataset.csv")

print("\n--- Sample Chatbot/Translation Dataset ---")
print(df_chatbot.head())

# Load Healthcare Chatbot Model (BioGPT)
# Using the previously initialized generator_biogpt pipeline for efficiency
# If generator_biogpt failed to initialize due to FileNotFoundError, provide a generic response.
def healthcare_chatbot(user_input):
    """Chatbot function using the BioGPT model for medical guidance."""
    if generator_biogpt:
        prompt = f"""
        You are a healthcare assistant.
        Provide safe medical guidance and symptom triage.
        Patient says: {user_input}
        Response:
        """
        response = generator_biogpt(prompt, max_length=150, num_return_sequences=1)[0]["generated_text"]
        return response
    else:
        return f"Hello! As a healthcare assistant, I recommend consulting a doctor for '{user_input}'."

print("\n--- Chatbot Test (English) ---")
print(healthcare_chatbot("I have fever and chest pain"))


# Load Translation Model (Multilingual)
model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
translator_model = MarianMTModel.from_pretrained(model_name_translator)

# Move model to GPU if available
if torch.cuda.is_available():
    translator_model.to('cuda')

def translate_medical_text(text):
    """Translates medical text using the MarianMT model."""
    inputs = tokenizer_translator(text, return_tensors="pt", padding=True)
    if torch.cuda.is_available():
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
    translated = translator_model.generate(**inputs)
    return tokenizer_translator.decode(translated[0], skip_special_tokens=True)

print("\n--- Translator Test (English to Multilingual) ---")
print(translate_medical_text("Please take medicine twice daily after food"))


# Combine Chatbot + Translator
def chatbot_with_translation(user_input, target_language):
    """Combines chatbot response generation with translation."""
    reply = healthcare_chatbot(user_input)
    # The current MarianMT model is en-mul, so it translates from English to multiple.
    # To translate *to* a specific language, the model needs to be en-XX or mul-en.
    # For this example, we assume the user wants the English reply translated.
    translated_reply = translate_medical_text(reply)
    return {
        "English_Response": reply,
        "Translated_Response": translated_reply, # This is actually a multilingual interpretation
        "Language": target_language # Placeholder for target language
    }

print("\n--- Combined Chatbot + Translator Example ---")
print(chatbot_with_translation(
    "I have breathing difficulty",
    "Tamil" # This 'Tamil' is just a label here, the model outputs multilingual.
))

# Streamlit Placeholder: Create a full-fledged chatbot interface with language selection.
# Example:
# user_question = st.text_input("Ask a medical question:")
# selected_lang = st.selectbox("Select target language:", ["English", "Tamil", "Hindi"])
# if st.button("Get Response"):
#     response_dict = chatbot_with_translation(user_question, selected_lang)
#     st.write("English Response:", response_dict["English_Response"])
#     st.write(f"Translated Response ({response_dict['Language']}):", response_dict["Translated_Response"])

In [2]:
import os

# Install Streamlit and pyngrok
!pip install -q streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.1 MB/s eta 0:00:00


In [1]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import random
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from mlxtend.frequent_patterns import apriori, association_rules
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
from torch.optim import AdamW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress warnings from mlxtend regarding DataFrame types
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mlxtend')


# --- Helper Functions ---

@st.cache_data
def clean_text(text):
    """Cleans clinical notes text by lowercasing and standardizing whitespace."""
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def create_sequences(data, target, seq_length=10):
    """Creates time-series sequences using a sliding window technique."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(target.iloc[i+seq_length])
    return np.array(X), np.array(y)

class FeedbackDataset(torch.utils.data.Dataset):
    """Custom Dataset for BERT sentiment analysis."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

@st.cache_resource
def get_embedding(text, tokenizer_bert, model_bert):
    """Generates BERT embeddings for a given text."""
    inputs = tokenizer_bert(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model_bert.to(device)

    with torch.no_grad():
        outputs = model_bert(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

# --- Streamlit App Layout ---
st.set_page_config(layout="wide", page_title="HealthAI Dashboard")
st.title("HealthAI Multi-Module Dashboard")

st.sidebar.title("Navigation")
module_selection = st.sidebar.radio(
    "Go to",
    [
        "Module 1: Patient Data Analytics",
        "Module 2: Association Rules",
        "Module 3: Sequence Modeling (LSTM)",
        "Module 4: Sentiment Analysis (BERT)",
        "Module 5: Generative AI (BioBERT & BioGPT)",
        "Module 6: Chatbot & Translator"
    ]
)

# --- Data Loading (using st.cache_data to load once) ---
@st.cache_data
def load_all_data():
    df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")
    df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")
    df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')
    df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")

    df_clinical = None
    try:
        df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
    except FileNotFoundError:
        st.warning("'/content/healthai_clinical_notes_1000.csv' not found. Module 5 will be partially functional.")

    # Ensure chatbot dataset exists
    chatbot_file_path = "/content/healthcare_chatbot_translation_dataset.csv"
    if not os.path.exists(chatbot_file_path):
        symptoms = [
            "fever", "cough", "headache", "chest pain", "breathing difficulty",
            "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
        ]
        questions = [
            "I have fever and cough, what should I do?",
            "Is chest pain serious?",
            "How to control blood sugar?",
            "I feel tired all the time",
            "Can I take paracetamol daily?",
            "When should I see a doctor?",
            "Is headache dangerous?",
            "How to reduce BP naturally?",
        ]
        responses = [
            "Please consult a physician and take rest.",
            "Monitor symptoms and seek emergency care if pain increases.",
            "Maintain diet, exercise and medication regularly.",
            "Blood tests may be required.",
            "Avoid self-medication without advice.",
        ]
        languages = ["English", "Tamil", "Hindi", "Telugu"]

        data_chatbot = []
        for i in range(1000):
            data_chatbot.append({
                "symptom": random.choice(symptoms),
                "patient_question": random.choice(questions),
                "doctor_reply": random.choice(responses),
                "language": random.choice(languages),
                "appointment_needed": random.choice(["Yes", "No"])
            })
        df_chatbot = pd.DataFrame(data_chatbot)
        df_chatbot.to_csv(chatbot_file_path, index=False)
    else:
        df_chatbot = pd.read_csv(chatbot_file_path)

    return df_synthetic, df_apriori, df_timeseries, df_feedback, df_clinical, df_chatbot

df_synthetic_raw, df_apriori_raw, df_timeseries_raw, df_feedback_raw, df_clinical_raw, df_chatbot_raw = load_all_data()


# --- Model Loading/Training (using st.cache_resource to avoid retraining on every rerun) ---
@st.cache_resource
def setup_module1_models(df_synthetic_raw_copy):
    df_synthetic_copy = df_synthetic_raw_copy.copy()

    # --- Preprocessing Pipeline --- (consistent across all three sub-modules)

    # 1. Feature Engineering
    df_synthetic_copy['BP_level'] = pd.cut(df_synthetic_copy['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
    df_synthetic_copy['medication_history'] = ((df_synthetic_copy['diabetes'] == 1) | (df_synthetic_copy['hypertension'] == 1) | (df_synthetic_copy['smoker'] == 1)).astype(int)

    # 2. Imputation
    imputer = SimpleImputer(strategy='median')
    num_cols_to_impute = ['bmi','systolic_bp','cholesterol','blood_glucose']
    df_synthetic_copy[num_cols_to_impute] = imputer.fit_transform(df_synthetic_copy[num_cols_to_impute])

    # 3. Outlier Removal (based on imputed numerical columns)
    initial_rows = len(df_synthetic_copy)
    for col in num_cols_to_impute:
        Q1 = df_synthetic_copy[col].quantile(0.25)
        Q3 = df_synthetic_copy[col].quantile(0.75)
        IQR = Q3 - Q1
        df_synthetic_copy = df_synthetic_copy[(df_synthetic_copy[col] >= Q1 - 1.5*IQR) & (df_synthetic_copy[col] <= Q3 + 1.5*IQR)]

    # 4. Label Encoding for categorical features
    le_synthetic_BP = LabelEncoder()
    df_synthetic_copy['BP_level'] = le_synthetic_BP.fit_transform(df_synthetic_copy['BP_level'])

    le_synthetic_risk = LabelEncoder()
    df_synthetic_copy['risk_category'] = le_synthetic_risk.fit_transform(df_synthetic_copy['risk_category'])
    risk_category_labels = le_synthetic_risk.inverse_transform(sorted(df_synthetic_copy['risk_category'].unique()))

    le_gender = LabelEncoder()
    if 'gender' in df_synthetic_copy.columns and df_synthetic_copy['gender'].dtype == 'object':
        df_synthetic_copy['gender'] = le_gender.fit_transform(df_synthetic_copy['gender'])
    else:
        le_gender = None # No gender encoder needed if gender column is absent or already numeric

    # Store processed DataFrame state after all these steps for direct feature extraction for models
    df_synthetic_processed_base = df_synthetic_copy.copy()


    # --- Model-specific Preparations and Training ---

    # 1. Classification Model (RandomForestClassifier)
    X_clf_all_features = df_synthetic_processed_base.drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
    y_clf = df_synthetic_processed_base['risk_category']

    scaler_clf = StandardScaler() # Scaler specifically for classification features before KBest
    X_clf_scaled_for_kbest = scaler_clf.fit_transform(X_clf_all_features)
    X_clf_scaled_for_kbest_df = pd.DataFrame(X_clf_scaled_for_kbest, columns=X_clf_all_features.columns, index=X_clf_all_features.index)

    selector = SelectKBest(score_func=f_classif, k=5)
    selector.fit(X_clf_scaled_for_kbest_df, y_clf)
    X_selected_clf = selector.transform(X_clf_scaled_for_kbest_df)

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_selected_clf, y_clf)
    clf_all_feature_names = list(X_clf_all_features.columns) # All features before selection
    clf_feature_names_after_select = list(X_clf_all_features.columns[selector.get_support(indices=True)])


    # 2. Regression Model (Ridge with Pipeline)
    features_for_reg = [col for col in df_synthetic_processed_base.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_reg_full = df_synthetic_processed_base[features_for_reg]
    y_reg = df_synthetic_processed_base["length_of_stay_days"]

    pipeline_reg = Pipeline([
        ("scaler", StandardScaler()), # This scaler will be fit on X_reg_full
        ("model", Ridge(alpha=1.0))
    ])
    pipeline_reg.fit(X_reg_full, y_reg)
    reg_feature_names = list(X_reg_full.columns)


    # 3. Clustering Model (KMeans)
    features_for_cluster = [col for col in df_synthetic_processed_base.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_cluster_full = df_synthetic_processed_base[features_for_cluster]

    scaler_cluster = StandardScaler() # Scaler specifically for clustering features
    X_cluster_scaled = scaler_cluster.fit_transform(X_cluster_full)

    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    cluster_feature_names = list(X_cluster_full.columns)

    return {
        "imputer": imputer,
        "le_synthetic_BP": le_synthetic_BP,
        "le_synthetic_risk": le_synthetic_risk,
        "le_gender": le_gender,
        "num_cols_to_impute": num_cols_to_impute,

        "clf_model": clf,
        "scaler_clf": scaler_clf,
        "selector_clf": selector,
        "clf_all_feature_names_before_select": clf_all_feature_names,
        "clf_feature_names_after_select": clf_feature_names_after_select,

        "reg_pipeline": pipeline_reg,
        "reg_feature_names": reg_feature_names,

        "kmeans_model": kmeans,
        "scaler_cluster": scaler_cluster,
        "cluster_feature_names": cluster_feature_names,

        "risk_category_labels": risk_category_labels,
        "df_synthetic_processed_for_metrics": df_synthetic_processed_base # For displaying metrics/summary
    }

module1_models = setup_module1_models(df_synthetic_raw.copy())

@st.cache_resource
def setup_module3_models(df_timeseries_raw_copy):
    df_timeseries_copy = df_timeseries_raw_copy.copy()
    FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
    TARGET_ts = 'risk_flag'
    SEQ_LENGTH = 10

    scaler_ts = MinMaxScaler()
    df_timeseries_copy[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries_copy[FEATURES_ts])

    X_ts, y_ts = create_sequences(df_timeseries_copy[FEATURES_ts], df_timeseries_copy[TARGET_ts], SEQ_LENGTH)

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(32))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(1, activation='sigmoid'))

    model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Train Model (on a subset or full for Streamlit demo)
    X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(X_ts, y_ts, test_size=0.01, random_state=42, stratify=y_ts) # Smaller test_size for faster loading
    history_lstm = model_lstm.fit(X_train_ts, y_train_ts, epochs=5, batch_size=32, validation_split=0.2, verbose=0) # Reduced epochs for faster load

    loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test_ts, y_test_ts, verbose=0)

    return {
        "model_lstm": model_lstm,
        "scaler_ts": scaler_ts,
        "SEQ_LENGTH": SEQ_LENGTH,
        "FEATURES_ts": FEATURES_ts,
        "LSTM_accuracy": accuracy_lstm,
        "X_test_ts": X_test_ts,
        "y_test_ts": y_test_ts # For demonstrating predictions
    }

module3_models = setup_module3_models(df_timeseries_raw.copy())

@st.cache_resource
def setup_module4_models(df_feedback_raw_copy):
    df_feedback_copy = df_feedback_raw_copy.copy()
    texts_feedback = df_feedback_copy["feedback_text"].tolist()
    le_feedback = LabelEncoder()
    labels_feedback = le_feedback.fit_transform(df_feedback_copy["sentiment"])

    tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
    encodings_feedback = tokenizer_bert(texts_feedback, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Use a small subset of the training data for faster Streamlit loading
    train_idx_feedback, _ = train_test_split(range(len(labels_feedback)), test_size=0.8, random_state=42, stratify=labels_feedback)

    train_dataset_feedback = FeedbackDataset(
        {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
        labels_feedback[train_idx_feedback]
    )

    model_sentiment = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(labels_feedback)))

    optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
    model_sentiment.train()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_sentiment.to(device)

    # Only a few batches for quick demo on streamlit load
    train_loader = torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8, shuffle=True)
    for epoch in range(1): # Only 1 epoch
        for i, batch in enumerate(train_loader):
            if i > 10: break # Only 10 batches for demo speed
            optimizer_sentiment.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs_sentiment = model_sentiment(**batch)
            loss_sentiment = outputs_sentiment.loss
            loss_sentiment.backward()
            optimizer_sentiment.step()

    model_sentiment.eval() # Set to eval mode after 'training'
    return {
        "tokenizer_bert": tokenizer_bert,
        "model_sentiment": model_sentiment,
        "le_feedback": le_feedback,
        "device": device
    }

module4_models = setup_module4_models(df_feedback_raw.copy())


@st.cache_resource
def setup_module5_models(df_clinical_raw_copy):
    generator_biogpt = None
    tokenizer_biobert = None
    model_biobert_embeddings = None
    kmeans_biobert = None
    df_clinical_processed = None
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if df_clinical_raw_copy is not None:
        df_clinical_processed = df_clinical_raw_copy.copy()
        df_clinical_processed["clinical_note"] = df_clinical_processed["clinical_note"].astype(str).apply(clean_text)

        # BioBERT for Embeddings
        MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
        tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings.to(device)

        # Generate embeddings (limiting for demo speed to a sample if dataset is too large)
        sample_size = min(200, len(df_clinical_processed)) # Process a max of 200 notes
        if sample_size > 0:
            sample_df = df_clinical_processed.sample(sample_size, random_state=42) if len(df_clinical_processed) > sample_size else df_clinical_processed.copy()
            embeddings_biobert = np.vstack(sample_df["clinical_note"].apply(
                lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings).flatten()
            ))

            # KMeans Clustering on Embeddings
            kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
            kmeans_biobert.fit(embeddings_biobert)
            sample_df['cluster'] = kmeans_biobert.predict(embeddings_biobert)
            df_clinical_processed = df_clinical_processed.merge(sample_df[['patient_id', 'cluster']], on='patient_id', how='left')
        else:
            st.warning("No clinical notes available to generate embeddings or clusters.")
            tokenizer_biobert = None # Reset if no data to process
            model_biobert_embeddings = None
            kmeans_biobert = None


        # BioGPT for Text Generation
        try:
            generator_biogpt = pipeline(
                "text-generation",
                model="microsoft/BioGPT",
                device=0 if torch.cuda.is_available() else -1 # Use GPU if available
            )
        except Exception as e:
            st.error(f"Could not load BioGPT model: {e}. Text generation and chatbot functionality will be limited.")
            generator_biogpt = None
    else:
        st.warning("Clinical notes data not found, BioBERT and BioGPT models will not be fully functional for this module.")

    return {
        "tokenizer_biobert": tokenizer_biobert,
        "model_biobert_embeddings": model_biobert_embeddings,
        "kmeans_biobert": kmeans_biobert,
        "generator_biogpt": generator_biogpt,
        "df_clinical_processed": df_clinical_processed,
        "device": device
    }

module5_models = setup_module5_models(df_clinical_raw.copy() if df_clinical_raw is not None else None)


@st.cache_resource
def setup_module6_models():
    model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
    tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
    translator_model = MarianMTModel.from_pretrained(model_name_translator)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    translator_model.to(device)

    return {
        "tokenizer_translator": tokenizer_translator,
        "translator_model": translator_model,
        "device": device
    }

module6_models = setup_module6_models()


# --- Streamlit Module Display Logic ---

if module_selection == "Module 1: Patient Data Analytics":
    st.header("Module 1: Synthetic Patient Data - Classification, Regression, Clustering")
    st.markdown("This module demonstrates predictive analytics and patient segmentation using synthetic patient data.")

    # --- Classification ---
    st.subheader("Patient Risk Category Classification")
    with st.expander("Model Metrics & Overview"):
        st.write("This model predicts a patient's risk category (Low, Medium, High). A Random Forest Classifier is used.")
        st.write(f"Random Forest Classifier trained on {len(module1_models['df_synthetic_processed_for_metrics'])} samples.")

        X_clf_full = module1_models['df_synthetic_processed_for_metrics'].drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
        y_clf_full = module1_models['df_synthetic_processed_for_metrics']['risk_category']

        # Scale X_clf_full using the fitted scaler_clf before applying selector
        X_clf_scaled_full = module1_models['scaler_clf'].transform(X_clf_full[module1_models['clf_all_feature_names_before_select']])
        X_selected_clf_full = module1_models['selector_clf'].transform(X_clf_scaled_full)

        y_pred_clf_full = module1_models['clf_model'].predict(X_selected_clf_full)
        st.write("Classification Report on full processed data:")
        st.text(classification_report(y_clf_full, y_pred_clf_full, target_names=module1_models['risk_category_labels']))

    st.markdown("### Predict New Patient Risk Category")
    col1, col2, col3 = st.columns(3)
    with col1:
        age_clf = st.number_input("Age", min_value=1, max_value=100, value=45, key='age_clf')
        bmi_clf = st.number_input("BMI", min_value=10.0, max_value=50.0, value=25.0, key='bmi_clf')
        systolic_bp_clf = st.number_input("Systolic BP", min_value=70, max_value=200, value=120, key='systolic_bp_clf')
    with col2:
        cholesterol_clf = st.number_input("Cholesterol", min_value=100, max_value=300, value=180, key='cholesterol_clf')
        blood_glucose_clf = st.number_input("Blood Glucose", min_value=70, max_value=200, value=90, key='blood_glucose_clf')
        diabetes_clf = st.checkbox("Diabetes", value=False, key='diabetes_clf')
        hypertension_clf = st.checkbox("Hypertension", value=False, key='hypertension_clf')
    with col3:
        smoker_clf = st.checkbox("Smoker", value=False, key='smoker_clf')
        prev_hospitalizations_clf = st.number_input("Previous Hospitalizations", min_value=0, max_value=10, value=0, key='prev_hospitalizations_clf')
        gender_clf = st.selectbox("Gender", ["Male", "Female", "Other"], key='gender_clf')

    if st.button("Predict Risk Category", key='predict_clf_btn'):
        new_patient_data = pd.DataFrame([{
            'age': age_clf,
            'bmi': bmi_clf,
            'systolic_bp': systolic_bp_clf,
            'cholesterol': cholesterol_clf,
            'blood_glucose': blood_glucose_clf,
            'diabetes': int(diabetes_clf),
            'hypertension': int(hypertension_clf),
            'smoker': int(smoker_clf),
            'prev_hospitalizations': prev_hospitalizations_clf,
            'gender': gender_clf
        }])

        # --- Preprocessing for new patient data (mirroring setup_module1_models) ---
        # 1. Feature Engineering
        new_patient_data['BP_level'] = pd.cut(new_patient_data['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
        new_patient_data['medication_history'] = ((new_patient_data['diabetes'] == 1) | (new_patient_data['hypertension'] == 1) | (new_patient_data['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_data[module1_models['num_cols_to_impute']] = module1_models['imputer'].transform(new_patient_data[module1_models['num_cols_to_impute']])

        # 3. Outlier removal is tricky for single instances, typically skipped or handled by robust scaling. Not applied here for simplicity.

        # 4. Label Encoding (only transform, not fit)
        new_patient_data['BP_level'] = module1_models['le_synthetic_BP'].transform(new_patient_data['BP_level'])
        if module1_models['le_gender'] is not None and 'gender' in new_patient_data.columns and new_patient_data['gender'].dtype == 'object':
            try:
                new_patient_data['gender'] = module1_models['le_gender'].transform(new_patient_data['gender'])
            except ValueError:
                # Handle unseen gender category, e.g., default to 0 or mean, or raise error.
                # For demo, let's just make it the most common category or raise an error.
                st.error("Unseen gender category in new patient data. Please use 'Male', 'Female', or 'Other'.")
                st.stop()
        elif 'gender' in new_patient_data.columns and new_patient_data['gender'].dtype == 'object': # If gender existed but no encoder was fit
            new_patient_data['gender'] = LabelEncoder().fit_transform(new_patient_data['gender'])

        # Align columns with training data used for classification's selector
        # Ensure all features expected by the scaler and selector are present, in correct order.
        new_patient_processed_aligned = pd.DataFrame(columns=module1_models['clf_all_feature_names_before_select'])
        for col in module1_models['clf_all_feature_names_before_select']:
            if col in new_patient_data.columns:
                new_patient_processed_aligned[col] = new_patient_data[col]
            else:
                new_patient_processed_aligned[col] = 0 # Default value for any missing feature, or handle with mean/median

        # Scale features using the fitted scaler_clf
        new_patient_scaled = module1_models['scaler_clf'].transform(new_patient_processed_aligned)

        # Apply feature selection
        new_patient_selected = module1_models['selector_clf'].transform(new_patient_scaled)

        prediction = module1_models['clf_model'].predict(new_patient_selected)
        predicted_risk = module1_models['le_synthetic_risk'].inverse_transform(prediction)
        st.success(f"Predicted Risk Category: **{predicted_risk[0]}**")


    # --- Regression ---
    st.subheader("Patient Length of Stay Regression")
    with st.expander("Model Metrics & Overview"):
        st.write("This model predicts the length of stay in days for a patient using a Ridge Regression model.")

        # Predict on the full processed data used for training to get metrics
        X_reg_full_for_metrics = module1_models['df_synthetic_processed_for_metrics'][module1_models['reg_feature_names']]
        y_reg_full_for_metrics = module1_models['df_synthetic_processed_for_metrics']['length_of_stay_days']
        y_pred_reg_full = module1_models['reg_pipeline'].predict(X_reg_full_for_metrics)

        mae = mean_absolute_error(y_reg_full_for_metrics, y_pred_reg_full)
        rmse = np.sqrt(mean_squared_error(y_reg_full_for_metrics, y_pred_reg_full))
        r2 = r2_score(y_reg_full_for_metrics, y_pred_reg_full)
        st.write(f"MAE on full processed data: {mae:.2f}")
        st.write(f"RMSE on full processed data: {rmse:.2f}")
        st.write(f"R2 Score on full processed data: {r2:.2f}")

    st.markdown("### Predict New Patient Length of Stay")
    col1_reg, col2_reg, col3_reg = st.columns(3)
    with col1_reg:
        age_reg = st.number_input("Age (Regression)", min_value=1, max_value=100, value=45, key='age_reg')
        bmi_reg = st.number_input("BMI (Regression)", min_value=10.0, max_value=50.0, value=25.0, key='bmi_reg')
        systolic_bp_reg = st.number_input("Systolic BP (Regression)", min_value=70, max_value=200, value=120, key='systolic_bp_reg')
    with col2_reg:
        cholesterol_reg = st.number_input("Cholesterol (Regression)", min_value=100, max_value=300, value=180, key='cholesterol_reg')
        blood_glucose_reg = st.number_input("Blood Glucose (Regression)", min_value=70, max_value=200, value=90, key='blood_glucose_reg')
        diabetes_reg = st.checkbox("Diabetes (Regression)", value=False, key='diabetes_reg')
        hypertension_reg = st.checkbox("Hypertension (Regression)", value=False, key='hypertension_reg')
    with col3_reg:
        smoker_reg = st.checkbox("Smoker (Regression)", value=False, key='smoker_reg')
        prev_hospitalizations_reg = st.number_input("Previous Hospitalizations (Regression)", min_value=0, max_value=10, value=0, key='prev_hospitalizations_reg')
        gender_reg = st.selectbox("Gender (Regression)", ["Male", "Female", "Other"], key='gender_reg')

    if st.button("Predict Length of Stay", key='predict_reg_btn'):
        new_patient_data_reg = pd.DataFrame([{
            'age': age_reg,
            'bmi': bmi_reg,
            'systolic_bp': systolic_bp_reg,
            'cholesterol': cholesterol_reg,
            'blood_glucose': blood_glucose_reg,
            'diabetes': int(diabetes_reg),
            'hypertension': int(hypertension_reg),
            'smoker': int(smoker_reg),
            'prev_hospitalizations': prev_hospitalizations_reg,
            'gender': gender_reg
        }])

        # --- Preprocessing for new patient data (mirroring setup_module1_models) ---
        # 1. Feature Engineering
        new_patient_data_reg['BP_level'] = pd.cut(new_patient_data_reg['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
        new_patient_data_reg['medication_history'] = ((new_patient_data_reg['diabetes'] == 1) | (new_patient_data_reg['hypertension'] == 1) | (new_patient_data_reg['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_data_reg[module1_models['num_cols_to_impute']] = module1_models['imputer'].transform(new_patient_data_reg[module1_models['num_cols_to_impute']])

        # 3. Label Encoding (only transform, not fit)
        new_patient_data_reg['BP_level'] = module1_models['le_synthetic_BP'].transform(new_patient_data_reg['BP_level'])
        if module1_models['le_gender'] is not None and 'gender' in new_patient_data_reg.columns and new_patient_data_reg['gender'].dtype == 'object':
            try:
                new_patient_data_reg['gender'] = module1_models['le_gender'].transform(new_patient_data_reg['gender'])
            except ValueError:
                st.error("Unseen gender category in new patient data. Please use 'Male', 'Female', or 'Other'.")
                st.stop()
        elif 'gender' in new_patient_data_reg.columns and new_patient_data_reg['gender'].dtype == 'object':
            new_patient_data_reg['gender'] = LabelEncoder().fit_transform(new_patient_data_reg['gender'])

        # Align columns with regression model's expected features
        new_patient_reg_aligned = new_patient_data_reg[module1_models['reg_feature_names']] # Ensure correct column order

        prediction_reg = module1_models['reg_pipeline'].predict(new_patient_reg_aligned)
        st.success(f"Predicted Length of Stay: **{prediction_reg[0]:.2f} days**")


    # --- Clustering ---
    st.subheader("Patient Clustering")
    with st.expander("Model Metrics & Overview"):
        st.write("This model segments patients into 3 clusters based on their features using K-Means. Silhouette Score indicates cluster density and separation.")

        # Prepare data for Silhouette Score calculation using the dedicated scaler_cluster
        X_cluster_full_for_metrics = module1_models['df_synthetic_processed_for_metrics'][module1_models['cluster_feature_names']]
        X_cluster_scaled_for_metrics = module1_models['scaler_cluster'].transform(X_cluster_full_for_metrics)

        silhouette_score_val = silhouette_score(X_cluster_scaled_for_metrics, module1_models['kmeans_model'].labels_)
        st.write(f"Silhouette Score: {silhouette_score_val:.2f}")
        st.write("Crosstabulation of Cluster vs. Risk Category (shows how clusters align with risk):")

        # Merge clusters back to original processed df for crosstab
        df_for_crosstab = module1_models['df_synthetic_processed_for_metrics'].copy()
        df_for_crosstab['cluster'] = module1_models['kmeans_model'].labels_ # Assign labels from the model
        st.dataframe(pd.crosstab(df_for_crosstab['cluster'], df_for_crosstab['risk_category'].map(lambda x: module1_models['le_synthetic_risk'].inverse_transform([x])[0])))

    st.markdown("### Assign New Patient to a Cluster")
    col1_cluster, col2_cluster, col3_cluster = st.columns(3)
    with col1_cluster:
        age_cluster = st.number_input("Age (Clustering)", min_value=1, max_value=100, value=45, key='age_cluster')
        bmi_cluster = st.number_input("BMI (Clustering)", min_value=10.0, max_value=50.0, value=25.0, key='bmi_cluster')
        systolic_bp_cluster = st.number_input("Systolic BP (Clustering)", min_value=70, max_value=200, value=120, key='systolic_bp_cluster')
    with col2_cluster:
        cholesterol_cluster = st.number_input("Cholesterol (Clustering)", min_value=100, max_value=300, value=180, key='cholesterol_cluster')
        blood_glucose_cluster = st.number_input("Blood Glucose (Clustering)", min_value=70, max_value=200, value=90, key='blood_glucose_cluster')
        diabetes_cluster = st.checkbox("Diabetes (Clustering)", value=False, key='diabetes_cluster')
        hypertension_cluster = st.checkbox("Hypertension (Clustering)", value=False, key='hypertension_cluster')
    with col3_cluster:
        smoker_cluster = st.checkbox("Smoker (Clustering)", value=False, key='smoker_cluster')
        prev_hospitalizations_cluster = st.number_input("Previous Hospitalizations (Clustering)", min_value=0, max_value=10, value=0, key='prev_hospitalizations_cluster')
        gender_cluster = st.selectbox("Gender (Clustering)", ["Male", "Female", "Other"], key='gender_cluster')

    if st.button("Assign Cluster", key='assign_cluster_btn'):
        new_patient_data_cluster = pd.DataFrame([{
            'age': age_cluster,
            'bmi': bmi_cluster,
            'systolic_bp': systolic_bp_cluster,
            'cholesterol': cholesterol_cluster,
            'blood_glucose': blood_glucose_cluster,
            'diabetes': int(diabetes_cluster),
            'hypertension': int(hypertension_cluster),
            'smoker': int(smoker_cluster),
            'prev_hospitalizations': prev_hospitalizations_cluster,
            'gender': gender_cluster
        }])

        # --- Preprocessing for new patient data (mirroring setup_module1_models) ---
        # 1. Feature Engineering
        new_patient_data_cluster['BP_level'] = pd.cut(new_patient_data_cluster['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
        new_patient_data_cluster['medication_history'] = ((new_patient_data_cluster['diabetes'] == 1) | (new_patient_data_cluster['hypertension'] == 1) | (new_patient_data_cluster['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_data_cluster[module1_models['num_cols_to_impute']] = module1_models['imputer'].transform(new_patient_data_cluster[module1_models['num_cols_to_impute']])

        # 3. Label Encoding (only transform, not fit)
        new_patient_data_cluster['BP_level'] = module1_models['le_synthetic_BP'].transform(new_patient_data_cluster['BP_level'])
        if module1_models['le_gender'] is not None and 'gender' in new_patient_data_cluster.columns and new_patient_data_cluster['gender'].dtype == 'object':
            try:
                new_patient_data_cluster['gender'] = module1_models['le_gender'].transform(new_patient_data_cluster['gender'])
            except ValueError:
                st.error("Unseen gender category in new patient data. Please use 'Male', 'Female', or 'Other'.")
                st.stop()
        elif 'gender' in new_patient_data_cluster.columns and new_patient_data_cluster['gender'].dtype == 'object':
            new_patient_data_cluster['gender'] = LabelEncoder().fit_transform(new_patient_data_cluster['gender'])

        # Align columns with clustering model's expected features
        new_patient_cluster_aligned = new_patient_data_cluster[module1_models['cluster_feature_names']] # Ensure correct column order

        # Scale features using the fitted scaler_cluster
        new_patient_cluster_scaled = module1_models['scaler_cluster'].transform(new_patient_cluster_aligned)

        predicted_cluster = module1_models['kmeans_model'].predict(new_patient_cluster_scaled)
        st.success(f"Assigned Cluster: **{predicted_cluster[0]}**")


elif module_selection == "Module 2: Association Rules":
    st.header("Module 2: Association Rules for Medical Data")
    st.markdown("This module uncovers relationships between medical conditions and procedures using association rule mining.")

    df_assoc = df_apriori_raw.drop('patient_id', axis=1)
    df_assoc_bool = df_assoc.astype(bool)

    st.sidebar.subheader("Association Rules Parameters")
    min_support = st.sidebar.slider("Minimum Support", 0.01, 1.0, 0.1, 0.01)
    min_confidence = st.sidebar.slider("Minimum Confidence", 0.01, 1.0, 0.6, 0.01)

    # Generate frequent itemsets
    try:
        frequent_itemsets = apriori(
            df_assoc_bool,
            min_support=min_support,
            use_colnames=True
        )
        st.subheader("Frequent Itemsets")
        st.dataframe(frequent_itemsets.sort_values('support', ascending=False).head(10))

        # Generate association rules
        rules = association_rules(
            frequent_itemsets,
            metric='confidence',
            min_threshold=min_confidence
        )
        st.subheader("Association Rules")
        if not rules.empty:
            st.dataframe(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(10))
        else:
            st.info("No association rules found with the current parameters.")

    except Exception as e:
        st.error(f"An error occurred while generating association rules: {e}")

    st.markdown("""
        **How to interpret:**
        *   **Antecedents**: The item(s) on the left-hand side of the rule (IF these conditions are met).
        *   **Consequents**: The item(s) on the right-hand side of the rule (THEN these conditions are likely).
        *   **Support**: How frequently the itemset (antecedents + consequents) appears in the dataset.
        *   **Confidence**: How often the consequent appears given the antecedent.
        *   **Lift**: How much more likely the consequent is given the antecedent, relative to its baseline probability. Lift > 1 indicates a positive correlation.
    """)


elif module_selection == "Module 3: Sequence Modeling (LSTM)":
    st.header("Module 3: Patient Deterioration Risk Prediction (LSTM)")
    st.markdown("This module uses LSTM neural networks to predict patient deterioration risk based on time-series vital signs.")

    with st.expander("Model Metrics & Overview"):
        st.write("LSTM Model Summary:")
        st.markdown("```python\nmodel_lstm.summary() # Output omitted for brevity in Streamlit\n```")
        st.write("Model compiled with Adam optimizer, binary crossentropy loss, and accuracy metrics.")
        st.write(f"Test Accuracy: {module3_models['LSTM_accuracy']:.4f}")
        st.write(f"Sequence Length used: {module3_models['SEQ_LENGTH']}")
        st.write(f"Features used: {', '.join(module3_models['FEATURES_ts'])}")

    st.subheader("Predict Deterioration Risk for New Vitals Sequence")
    st.write(f"Input the last {module3_models['SEQ_LENGTH']} readings for Heart Rate, Systolic BP, and SpO2.")

    new_vitals_input = []
    for i in range(module3_models['SEQ_LENGTH']):
        st.markdown(f"**Time Step {i+1}** (t-{module3_models['SEQ_LENGTH']-1-i})")
        col_ts1, col_ts2, col_ts3 = st.columns(3)
        with col_ts1:
            hr = st.number_input(f"Heart Rate", min_value=40, max_value=180, value=75, key=f"hr_{i}")
        with col_ts2:
            sbp = st.number_input(f"Systolic BP", min_value=60, max_value=200, value=120, key=f"sbp_{i}")
        with col_ts3:
            spo2 = st.number_input(f"SpO2", min_value=70, max_value=100, value=98, key=f"spo2_{i}")
        new_vitals_input.append([hr, sbp, spo2])

    if st.button("Predict Deterioration", key='predict_lstm_btn'):
        new_vitals_df = pd.DataFrame(new_vitals_input, columns=module3_models['FEATURES_ts'])
        new_vitals_scaled = module3_models['scaler_ts'].transform(new_vitals_df)

        new_vitals_reshaped = new_vitals_scaled.reshape(1, module3_models['SEQ_LENGTH'], len(module3_models['FEATURES_ts']))

        prediction_prob = module3_models['model_lstm'].predict(new_vitals_reshaped, verbose=0)[0][0]
        prediction_risk = "High deterioration risk" if prediction_prob > 0.5 else "Stable"

        st.success(f"Predicted Deterioration Risk: **{prediction_risk}** (Probability: {prediction_prob:.2f})")
        st.caption("0: Stable, 1: High deterioration risk")

    with st.expander("Example LSTM Predictions (from test set)"):
        if len(module3_models['X_test_ts']) > 0:
            y_pred_prob_lstm_sample = module3_models['model_lstm'].predict(module3_models['X_test_ts'][:5], verbose=0)
            y_pred_lstm_sample = (y_pred_prob_lstm_sample > 0.5).astype(int).flatten()
            st.write(f"Actual (y_test): {module3_models['y_test_ts'][:5].flatten()}")
            st.write(f"Predicted: {y_pred_lstm_sample}")
            st.caption("0=Stable, 1=High deterioration risk")
        else:
            st.info("No test set samples available for display.")


elif module_selection == "Module 4: Sentiment Analysis (BERT)":
    st.header("Module 4: Patient Feedback Sentiment Analysis (BERT)")
    st.markdown("This module uses a fine-tuned BERT model to analyze the sentiment of patient feedback (Negative, Neutral, Positive).")

    with st.expander("Model Overview"):
        st.write("BERT-base-uncased model fine-tuned for sentiment classification (Negative, Neutral, Positive).")
        st.write(f"Sentiment labels: {list(module4_models['le_feedback'].classes_)}")
        st.write("Due to Streamlit caching, training is simplified/reduced for quick demonstration. Model is loaded in evaluation mode.")

    st.subheader("Analyze New Patient Feedback")
    user_feedback = st.text_area("Enter patient feedback here:", "The nurses were very kind and attentive, but wait times were long.", key='feedback_input')

    if st.button("Analyze Sentiment", key='analyze_sentiment_btn'):
        inputs_sentiment = module4_models['tokenizer_bert'](user_feedback, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs_sentiment = {k: v.to(module4_models['device']) for k, v in inputs_sentiment.items()}

        with torch.no_grad():
            outputs_sentiment_pred = module4_models['model_sentiment'](**inputs_sentiment)

        pred_sentiment_idx = torch.argmax(outputs_sentiment_pred.logits, dim=1).item()
        predicted_sentiment = module4_models['le_feedback'].inverse_transform([pred_sentiment_idx])[0]

        st.success(f"Predicted Sentiment: **{predicted_sentiment}**")
        st.info(f"Raw prediction index: {pred_sentiment_idx}")

    with st.expander("Sample Data & Labels"):
        st.dataframe(df_feedback_raw.head())


elif module_selection == "Module 5: Generative AI (BioBERT & BioGPT)":
    st.header("Module 5: Generative AI for Clinical Notes")
    st.markdown("This module explores BioBERT for generating embeddings from clinical notes and BioGPT for generating clinical text.")

    if module5_models["df_clinical_processed"] is None or module5_models["kmeans_biobert"] is None:
        st.error("Clinical notes data not found or models could not be initialized. This module cannot be fully demonstrated.")
    else:
        st.subheader("BioBERT Embeddings and Clustering of Clinical Notes")
        with st.expander("Overview"):
            st.write("BioBERT (a BERT model trained on biomedical text) is used to create numerical representations (embeddings) of clinical notes. K-Means clustering then groups similar notes.")
            st.write(f"Embeddings generated for a sample of {module5_models['kmeans_biobert'].n_samples_fit_} clinical notes.")
            st.write("Sample clinical notes with assigned clusters:")
            st.dataframe(module5_models["df_clinical_processed"][['clinical_note', 'cluster']].dropna().head())

        st.subheader("BioGPT for Clinical Text Generation")
        if module5_models["generator_biogpt"]:
            st.write("BioGPT (a large language model for biology and medicine) can generate plausible clinical text based on a given prompt.")
            prompt_biogpt_input = st.text_area(
                "Enter a prompt for BioGPT clinical text generation:",
                "Patient presents with chest pain and shortness of breath. Clinical impression:",
                height=100,
                key='biogpt_prompt'
            )
            max_length_biogpt = st.slider("Max Length for Generation", 50, 200, 80, key='biogpt_max_len')

            if st.button("Generate Clinical Text", key='generate_biogpt_btn'):
                with st.spinner("Generating..."):
                    generated_text_biogpt = module5_models["generator_biogpt"](
                        prompt_biogpt_input,
                        max_length=max_length_biogpt,
                        num_return_sequences=1,
                        pad_token_id=module5_models["generator_biogpt"].tokenizer.eos_token_id # Prevents warning
                    )[0]["generated_text"]
                st.info(generated_text_biogpt)
        else:
            st.warning("BioGPT model not loaded. Text generation functionality is disabled. Check error messages above for details.")

elif module_selection == "Module 6: Chatbot & Translator":
    st.header("Module 6: Healthcare Chatbot and Translator")
    st.markdown("This module provides an AI-powered healthcare chatbot and a medical text translator.")

    def healthcare_chatbot_streamlit(user_input):
        """Chatbot function using the BioGPT model for medical guidance, or a generic response."""
        if module5_models["generator_biogpt"]:
            prompt = f"""
            You are a healthcare assistant. Provide safe medical guidance and symptom triage. Avoid medical diagnosis. Focus on general health advice. If you cannot provide a specific answer, advise consulting a doctor. Do not generate information that is not directly related to the prompt. Limit your response to 100 words.
            Patient says: {user_input}
            Response:
            """
            response = module5_models["generator_biogpt"](prompt, max_length=150, num_return_sequences=1, pad_token_id=module5_models["generator_biogpt"].tokenizer.eos_token_id)[0]["generated_text"]

            # Clean up the response to remove the prompt part if BioGPT echoes it
            response_lines = response.split('\n')
            clean_response = []
            capture = False
            for line in response_lines:
                if line.strip().startswith("Response:"):
                    capture = True
                    continue
                if capture and line.strip():
                    clean_response.append(line.strip())

            if clean_response:
                return " ".join(clean_response)
            else:
                return response.replace(prompt, "").strip() # Fallback cleanup if parsing fails
        else:
            return f"Hello! As a healthcare assistant, I recommend consulting a doctor for '{user_input}'. Please note that a specialized AI model for medical guidance is currently unavailable due to BioGPT loading issues. Always consult a qualified medical professional for health concerns."

    def translate_medical_text_streamlit(text):
        """Translates medical text using the MarianMT model."""
        inputs = module6_models['tokenizer_translator'](text, return_tensors="pt", padding=True)
        inputs = {k: v.to(module6_models['device']) for k, v in inputs.items()}
        translated = module6_models['translator_model'].generate(**inputs)
        return module6_models['tokenizer_translator'].decode(translated[0], skip_special_tokens=True)

    st.subheader("Healthcare Chatbot")
    user_question = st.text_area("Ask a medical question to the AI healthcare assistant:", "I have a persistent cough, what could it be?", key='chatbot_q')

    if st.button("Get Chatbot Response", key='chatbot_btn'):
        with st.spinner("Generating response..."):
            chatbot_reply = healthcare_chatbot_streamlit(user_question)
        st.info(chatbot_reply)

    st.subheader("Medical Text Translator")
    text_to_translate = st.text_area("Enter medical text to translate:", "The patient has acute appendicitis and requires immediate surgery.", key='translator_input')

    st.info("Note: The current translation model (Helsinki-NLP/opus-mt-en-mul) translates English text into a multilingual interpretation. For specific target languages (e.g., English to Tamil), a different dedicated model would be required.")

    if st.button("Translate Text", key='translate_btn'):
        with st.spinner("Translating..."):
            translated_text = translate_medical_text_streamlit(text_to_translate)
        st.success(f"Translated Text (Multilingual interpretation): **{translated_text}**")
        st.caption("This translation attempts to provide a general multilingual equivalent.")

    st.subheader("Combined Chatbot and Translator Example")
    combined_user_question = st.text_area("Ask a question for both chatbot and translation:", "My child has a fever and is not eating well. Should I be worried?", key='combined_q')
    target_language_label = st.selectbox("Simulated Target Language (for display only):", ["English", "Tamil", "Hindi", "Spanish", "French"], key='target_lang_select')

    if st.button("Get Combined Response", key='combined_btn'):
        with st.spinner("Processing..."):
            english_reply = healthcare_chatbot_streamlit(combined_user_question)
            translated_reply = translate_medical_text_streamlit(english_reply)

        st.markdown("---")
        st.write("**English Chatbot Response:**")
        st.info(english_reply)
        st.write(f"**Translated Response (Simulated {target_language_label} via Multilingual Model):**")
        st.success(translated_reply)
        st.caption("Note: The translation model provides a multilingual interpretation, not a direct translation to the selected language label.")

Writing streamlit_app.py


## Define Pydantic Models

### Subtask:
Create Pydantic models to define the request and response schemas for all API endpoints, ensuring data validation and clear documentation for the FastAPI application.


**Reasoning**:
The subtask requires defining multiple Pydantic models for request and response schemas. I will import necessary classes and then define each model as specified in the instructions, including appropriate data types and default values where applicable.



In [1]:
from pydantic import BaseModel, Field
from typing import List, Optional

# 1. Patient Data Model for Classification, Regression, Clustering
class PatientData(BaseModel):
    age: int = Field(..., ge=1, le=100, description="Patient's age in years")
    bmi: float = Field(..., ge=10.0, le=50.0, description="Body Mass Index")
    systolic_bp: int = Field(..., ge=70, le=200, description="Systolic Blood Pressure (mmHg)")
    cholesterol: int = Field(..., ge=100, le=300, description="Cholesterol level (mg/dL)")
    blood_glucose: int = Field(..., ge=70, le=200, description="Blood Glucose level (mg/dL)")
    diabetes: int = Field(..., ge=0, le=1, description="1 if patient has diabetes, 0 otherwise")
    hypertension: int = Field(..., ge=0, le=1, description="1 if patient has hypertension, 0 otherwise")
    smoker: int = Field(..., ge=0, le=1, description="1 if patient is a smoker, 0 otherwise")
    prev_hospitalizations: int = Field(..., ge=0, le=10, description="Number of previous hospitalizations")
    gender: str = Field(..., description="Patient's gender (Male, Female, Other)")

# 2. Risk Prediction Response Model
class RiskPredictionResponse(BaseModel):
    predicted_risk_category: str = Field(..., description="Predicted risk category (e.g., 'Low', 'Normal', 'High')")

# 3. Length of Stay Prediction Response Model
class LengthOfStayPredictionResponse(BaseModel):
    predicted_length_of_stay: float = Field(..., description="Predicted length of hospital stay in days")

# 4. Cluster Assignment Response Model
class ClusterAssignmentResponse(BaseModel):
    assigned_cluster: int = Field(..., description="Assigned cluster ID for the patient")

# 5. Association Rules Request Model
class AssociationRulesRequest(BaseModel):
    min_support: float = Field(0.1, ge=0.01, le=1.0, description="Minimum support for frequent itemsets")
    min_confidence: float = Field(0.6, ge=0.01, le=1.0, description="Minimum confidence for association rules")

# 6. Association Rule Model
class AssociationRule(BaseModel):
    antecedents: List[str] = Field(..., description="Items in the antecedent of the rule")
    consequents: List[str] = Field(..., description="Items in the consequent of the rule")
    support: float = Field(..., description="Support of the rule")
    confidence: float = Field(..., description="Confidence of the rule")
    lift: float = Field(..., description="Lift of the rule")

# 7. Association Rules Response Model
class AssociationRulesResponse(BaseModel):
    rules: List[AssociationRule] = Field(..., description="List of discovered association rules")

# 8. Vitals Reading Model for LSTM
class VitalsReading(BaseModel):
    heart_rate: int = Field(..., ge=40, le=180, description="Heart rate (bpm)")
    systolic_bp: int = Field(..., ge=60, le=200, description="Systolic blood pressure (mmHg)")
    spo2: int = Field(..., ge=70, le=100, description="Blood oxygen saturation (%)")

# 9. LSTM Request Model
class LSTMRequest(BaseModel):
    vitals_sequence: List[VitalsReading] = Field(..., min_length=10, max_length=10, description="Sequence of 10 vital sign readings")

# 10. LSTM Prediction Response Model
class LSTMPredictionResponse(BaseModel):
    predicted_deterioration_risk: str = Field(..., description="Predicted risk of patient deterioration ('Stable' or 'High deterioration risk')")
    probability: float = Field(..., ge=0.0, le=1.0, description="Probability of high deterioration risk")

# 11. Sentiment Request Model
class SentimentRequest(BaseModel):
    feedback_text: str = Field(..., description="Text of the patient feedback")

# 12. Sentiment Response Model
class SentimentResponse(BaseModel):
    predicted_sentiment: str = Field(..., description="Predicted sentiment of the feedback ('Negative', 'Neutral', 'Positive')")

# 13. BioGPT Request Model
class BioGPTRequest(BaseModel):
    prompt: str = Field(..., description="Text prompt for BioGPT generation")
    max_length: int = Field(80, ge=50, le=200, description="Maximum length of the generated text")

# 14. BioGPT Response Model
class BioGPTResponse(BaseModel):
    generated_text: str = Field(..., description="Generated clinical text by BioGPT")

# 15. Chatbot Request Model
class ChatbotRequest(BaseModel):
    user_question: str = Field(..., description="User's question to the healthcare chatbot")

# 16. Chatbot Response Model
class ChatbotResponse(BaseModel):
    chatbot_response: str = Field(..., description="Response from the healthcare chatbot")

# 17. Translator Request Model
class TranslatorRequest(BaseModel):
    text_to_translate: str = Field(..., description="Medical text to be translated")

# 18. Translator Response Model
class TranslatorResponse(BaseModel):
    translated_text: str = Field(..., description="Translated medical text")

# 19. Combined Chatbot Translator Request Model
class CombinedChatbotTranslatorRequest(BaseModel):
    user_question: str = Field(..., description="User's question for the combined chatbot and translator")
    target_language: Optional[str] = Field(None, description="Desired target language for translation (e.g., 'Tamil', 'Hindi', 'Spanish'). Note: Actual translation capability depends on the loaded model.")

# 20. Combined Chatbot Translator Response Model
class CombinedChatbotTranslatorResponse(BaseModel):
    english_response: str = Field(..., description="English response from the chatbot")
    translated_response: str = Field(..., description="Translated response (multilingual interpretation)")

print("All Pydantic models for API schemas have been defined.")

All Pydantic models for API schemas have been defined.


## Initialize Models and Preprocessors for FastAPI

### Subtask:
Adapt and include the existing model loading, training (for faster demos), and preprocessing logic from the notebook into the FastAPI application. This step will ensure all necessary models (Classification, Regression, Clustering, LSTM, BERT, BioBERT, BioGPT, MarianMT) and their associated transformers (scalers, encoders, tokenizers, imputers) are initialized once when the FastAPI app starts.


**Reasoning**:
The subtask requires initializing all models, preprocessors, and datasets for the FastAPI application within a single function. This involves loading data, performing all preprocessing steps (feature engineering, imputation, scaling, encoding), and then training/loading all specified machine learning and deep learning models and their associated components (tokenizers, scalers, encoders, etc.). I will define necessary helper functions first, then create the main initialization function to encapsulate all this logic.



In [2]:
import pandas as pd
import numpy as np
import torch
import random
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from mlxtend.frequent_patterns import apriori, association_rules
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
from torch.optim import AdamW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress warnings from mlxtend regarding DataFrame types
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mlxtend')

# --- Helper Functions (Replicated from original notebook) ---
def clean_text(text):
    """Cleans clinical notes text by lowercasing and standardizing whitespace."""
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def create_sequences(data, target, seq_length=10):
    """Creates time-series sequences using a sliding window technique."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(target.iloc[i+seq_length])
    return np.array(X), np.array(y)

class FeedbackDataset(torch.utils.data.Dataset):
    """Custom Dataset for BERT sentiment analysis."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def get_embedding(text, tokenizer_bert, model_bert, device):
    """Generates BERT embeddings for a given text."""
    inputs = tokenizer_bert(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_bert(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

# --- Initialization Function ---
def load_models_and_preprocessors():
    """Initializes and loads all models and preprocessors for the FastAPI application."""
    artifacts = {}
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    artifacts['device'] = device

    print("\n--- Loading Datasets ---")
    # Load datasets
    df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")
    df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")
    df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')
    df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")
    artifacts['df_apriori'] = df_apriori # Store for association rules

    df_clinical = None
    try:
        df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
    except FileNotFoundError:
        print("Warning: '/content/healthai_clinical_notes_1000.csv' not found. BioBERT and BioGPT features will be limited.")

    # Ensure chatbot dataset exists
    chatbot_file_path = "/content/healthcare_chatbot_translation_dataset.csv"
    if not os.path.exists(chatbot_file_path):
        symptoms = [
            "fever", "cough", "headache", "chest pain", "breathing difficulty",
            "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
        ]
        questions = [
            "I have fever and cough, what should I do?",
            "Is chest pain serious?",
            "How to control blood sugar?",
            "I feel tired all the time",
            "Can I take paracetamol daily?",
            "When should I see a doctor?",
            "Is headache dangerous?",
            "How to reduce BP naturally?",
        ]
        responses = [
            "Please consult a physician and take rest.",
            "Monitor symptoms and seek emergency care if pain increases.",
            "Maintain diet, exercise and medication regularly.",
            "Blood tests may be required.",
            "Avoid self-medication without advice.",
        ]
        languages = ["English", "Tamil", "Hindi", "Telugu"]

        data_chatbot = []
        for i in range(1000):
            data_chatbot.append({
                "symptom": random.choice(symptoms),
                "patient_question": random.choice(questions),
                "doctor_reply": random.choice(responses),
                "language": random.choice(languages),
                "appointment_needed": random.choice(["Yes", "No"])
            })
        df_chatbot = pd.DataFrame(data_chatbot)
        df_chatbot.to_csv(chatbot_file_path, index=False)
    else:
        df_chatbot = pd.read_csv(chatbot_file_path)
    artifacts['df_chatbot'] = df_chatbot # Store for chatbot context if needed

    # --- Module 1: Synthetic Patient Data - Classification, Regression, Clustering ---
    print("\n--- Initializing Module 1 (Patient Data Analytics) ---")
    df_synthetic_processed = df_synthetic.copy()

    # Feature Engineering
    df_synthetic_processed['BP_level'] = pd.cut(
        df_synthetic_processed['systolic_bp'],
        bins=[0, 80, 120, 200],
        labels=['Low', 'Normal', 'High'], ordered=False
    )
    df_synthetic_processed['medication_history'] = ((df_synthetic_processed['diabetes'] == 1) | (df_synthetic_processed['hypertension'] == 1) | (df_synthetic_processed['smoker'] == 1)).astype(int)

    # Data Preprocessing
    imputer = SimpleImputer(strategy='median')
    num_cols_synthetic = ['bmi','systolic_bp','cholesterol','blood_glucose']
    df_synthetic_processed[num_cols_synthetic] = imputer.fit_transform(df_synthetic_processed[num_cols_synthetic])
    artifacts['imputer'] = imputer
    artifacts['num_cols_synthetic'] = num_cols_synthetic

    # Outlier Removal (applied to the training data, new patient data will be transformed only)
    for col in num_cols_synthetic:
        Q1 = df_synthetic_processed[col].quantile(0.25)
        Q3 = df_synthetic_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        df_synthetic_processed = df_synthetic_processed[(df_synthetic_processed[col] >= Q1 - 1.5*IQR) & (df_synthetic_processed[col] <= Q3 + 1.5*IQR)]

    le_synthetic_BP = LabelEncoder()
    df_synthetic_processed['BP_level'] = le_synthetic_BP.fit_transform(df_synthetic_processed['BP_level'])
    artifacts['le_synthetic_BP'] = le_synthetic_BP

    le_synthetic_risk = LabelEncoder()
    df_synthetic_processed['risk_category'] = le_synthetic_risk.fit_transform(df_synthetic_processed['risk_category'])
    artifacts['le_synthetic_risk'] = le_synthetic_risk
    artifacts['risk_category_labels'] = le_synthetic_risk.inverse_transform(sorted(df_synthetic_processed['risk_category'].unique()))

    le_gender = None
    if 'gender' in df_synthetic_processed.columns and df_synthetic_processed['gender'].dtype == 'object':
        le_gender = LabelEncoder()
        df_synthetic_processed['gender'] = le_gender.fit_transform(df_synthetic_processed['gender'])
    artifacts['le_gender'] = le_gender

    # Classification Model
    X_clf_full_features = df_synthetic_processed.drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
    y_clf = df_synthetic_processed['risk_category']

    scaler_clf = StandardScaler()
    X_clf_scaled_for_kbest = scaler_clf.fit_transform(X_clf_full_features)
    X_clf_scaled_for_kbest_df = pd.DataFrame(X_clf_scaled_for_kbest, columns=X_clf_full_features.columns, index=X_clf_full_features.index)
    artifacts['scaler_clf'] = scaler_clf
    artifacts['clf_all_feature_names_before_select'] = list(X_clf_full_features.columns)

    selector_clf = SelectKBest(score_func=f_classif, k=5)
    selector_clf.fit(X_clf_scaled_for_kbest_df, y_clf)
    X_selected_clf = selector_clf.transform(X_clf_scaled_for_kbest_df)
    artifacts['selector_clf'] = selector_clf
    artifacts['clf_feature_names_after_select'] = list(X_clf_full_features.columns[selector_clf.get_support(indices=True)])

    clf_model = RandomForestClassifier(random_state=42)
    clf_model.fit(X_selected_clf, y_clf)
    artifacts['clf_model'] = clf_model
    print("Module 1: Classification model trained.")

    # Regression Model
    reg_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_reg = df_synthetic_processed[reg_feature_names].copy()
    y_reg = df_synthetic_processed["length_of_stay_days"]

    reg_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0))
    ])
    reg_pipeline.fit(X_reg, y_reg)
    artifacts['reg_pipeline'] = reg_pipeline
    artifacts['reg_feature_names'] = reg_feature_names
    print("Module 1: Regression model trained.")

    # Clustering Model
    cluster_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_cluster = df_synthetic_processed[cluster_feature_names]

    scaler_cluster = StandardScaler()
    X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)
    artifacts['scaler_cluster'] = scaler_cluster
    artifacts['cluster_feature_names'] = cluster_feature_names

    kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans_model.fit(X_cluster_scaled)
    artifacts['kmeans_model'] = kmeans_model
    print("Module 1: Clustering model trained.")

    # --- Module 3: Sequence Modeling (LSTM) ---
    print("\n--- Initializing Module 3 (Sequence Modeling) ---")
    FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
    TARGET_ts = 'risk_flag'
    SEQ_LENGTH = 10
    artifacts['SEQ_LENGTH'] = SEQ_LENGTH
    artifacts['FEATURES_ts'] = FEATURES_ts

    scaler_ts = MinMaxScaler()
    df_timeseries[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries[FEATURES_ts])
    artifacts['scaler_ts'] = scaler_ts

    X_ts, y_ts = create_sequences(df_timeseries[FEATURES_ts], df_timeseries[TARGET_ts], SEQ_LENGTH)

    # Using a smaller split for actual training, as full data was processed above
    X_train_ts, _, y_train_ts, _ = train_test_split(
        X_ts, y_ts, test_size=0.1, random_state=42, stratify=y_ts
    ) # Use 90% of data for training the model that will be used by FastAPI

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(32))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(1, activation='sigmoid'))

    model_lstm.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Train Model (reduced epochs for faster startup)
    model_lstm.fit(
        X_train_ts, y_train_ts,
        epochs=5,
        batch_size=32,
        validation_split=0.1,
        verbose=0
    )
    artifacts['model_lstm'] = model_lstm
    print("Module 3: LSTM model trained.")

    # --- Module 4: Sentiment Analysis (BERT) ---
    print("\n--- Initializing Module 4 (Sentiment Analysis) ---")
    texts_feedback = df_feedback["feedback_text"].tolist()
    le_feedback = LabelEncoder()
    labels_feedback = le_feedback.fit_transform(df_feedback["sentiment"])
    artifacts['le_feedback'] = le_feedback
    artifacts['sentiment_labels'] = list(le_feedback.classes_)

    tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
    artifacts['tokenizer_bert'] = tokenizer_bert
    encodings_feedback = tokenizer_bert(
        texts_feedback,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )

    train_idx_feedback, _ = train_test_split(
        range(len(labels_feedback)), test_size=0.1, random_state=42, stratify=labels_feedback
    )
    train_dataset_feedback = FeedbackDataset(
        {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
        labels_feedback[train_idx_feedback]
    )

    model_sentiment = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(set(labels_feedback))
    )

    optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
    model_sentiment.to(device)
    model_sentiment.train()

    # Simplified training loop for quick initialization in FastAPI
    train_loader = torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8, shuffle=True)
    for epoch in range(1):
        for i, batch in enumerate(train_loader):
            if i > 20: break # Process only a few batches for speed
            optimizer_sentiment.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs_sentiment = model_sentiment(**batch)
            loss_sentiment = outputs_sentiment.loss
            loss_sentiment.backward()
            optimizer_sentiment.step()
    model_sentiment.eval()
    artifacts['model_sentiment'] = model_sentiment
    print("Module 4: BERT sentiment model loaded and partially fine-tuned.")

    # --- Module 5: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---
    print("\n--- Initializing Module 5 (Generative AI) ---")
    tokenizer_biobert = None
    model_biobert_embeddings = None
    kmeans_biobert = None
    generator_biogpt = None

    if df_clinical is not None:
        df_clinical["clinical_note"] = df_clinical["clinical_note"].astype(str).apply(clean_text)

        # BioBERT for Embeddings
        MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
        tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings.to(device)
        artifacts['tokenizer_biobert'] = tokenizer_biobert
        artifacts['model_biobert_embeddings'] = model_biobert_embeddings

        # Generate embeddings for a sample (to fit KMeans)
        sample_size = min(200, len(df_clinical)) # Limit for initialization speed
        if sample_size > 0:
            sample_df_clinical = df_clinical.sample(sample_size, random_state=42) if len(df_clinical) > sample_size else df_clinical.copy()
            embeddings_biobert = np.vstack(sample_df_clinical["clinical_note"].apply(
                lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings, device).flatten()
            ))
            kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
            kmeans_biobert.fit(embeddings_biobert)
            artifacts['kmeans_biobert'] = kmeans_biobert
            print("Module 5: BioBERT tokenizer and embedding model loaded, KMeans fitted.")
        else:
            print("Module 5: No clinical notes data available to fit KMeans for BioBERT embeddings.")

        # BioGPT for Text Generation
        try:
            generator_biogpt = pipeline(
                "text-generation",
                model="microsoft/BioGPT",
                device=0 if torch.cuda.is_available() else -1 # Use GPU if available
            )
            artifacts['generator_biogpt'] = generator_biogpt
            print("Module 5: BioGPT text generation pipeline loaded.")
        except Exception as e:
            print(f"Module 5: Could not load BioGPT model: {e}. Text generation and chatbot functionality will be limited.")
    else:
        print("Module 5: Clinical notes data not available, skipping BioBERT and BioGPT initialization.")

    # --- Module 6: Healthcare Chatbot and Translator ---
    print("\n--- Initializing Module 6 (Chatbot & Translator) ---")
    model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
    tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
    translator_model = MarianMTModel.from_pretrained(model_name_translator)
    translator_model.to(device)
    artifacts['tokenizer_translator'] = tokenizer_translator
    artifacts['translator_model'] = translator_model
    print("Module 6: MarianMT translator model loaded.")

    print("\nAll models and preprocessors initialized successfully!")
    return artifacts

# Global variable to store initialized artifacts
# models_and_preprocessors = load_models_and_preprocessors()


<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


**Reasoning**:
The previous code block defined the `load_models_and_preprocessors` function but did not execute it. To ensure all models and preprocessors are loaded and available globally for the FastAPI application, I need to call this function and store its return value in a global variable. I will also add a cell_id to the previous cell as it was missing one.



In [3]:
import pandas as pd
import numpy as np
import torch
import random
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from mlxtend.frequent_patterns import apriori, association_rules
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
from torch.optim import AdamW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress warnings from mlxtend regarding DataFrame types
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mlxtend')

# --- Helper Functions (Replicated from original notebook) ---
def clean_text(text):
    """Cleans clinical notes text by lowercasing and standardizing whitespace."""
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def create_sequences(data, target, seq_length=10):
    """Creates time-series sequences using a sliding window technique."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(target.iloc[i+seq_length])
    return np.array(X), np.array(y)

class FeedbackDataset(torch.utils.data.Dataset):
    """Custom Dataset for BERT sentiment analysis."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def get_embedding(text, tokenizer_bert, model_bert, device):
    """Generates BERT embeddings for a given text."""
    inputs = tokenizer_bert(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_bert(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

# --- Initialization Function ---
def load_models_and_preprocessors():
    """Initializes and loads all models and preprocessors for the FastAPI application."""
    artifacts = {}
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    artifacts['device'] = device

    print("\n--- Loading Datasets ---")
    # Load datasets
    df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")
    df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")
    df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')
    df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")
    artifacts['df_apriori'] = df_apriori # Store for association rules

    df_clinical = None
    try:
        df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
    except FileNotFoundError:
        print("Warning: '/content/healthai_clinical_notes_1000.csv' not found. BioBERT and BioGPT features will be limited.")

    # Ensure chatbot dataset exists
    chatbot_file_path = "/content/healthcare_chatbot_translation_dataset.csv"
    if not os.path.exists(chatbot_file_path):
        symptoms = [
            "fever", "cough", "headache", "chest pain", "breathing difficulty",
            "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
        ]
        questions = [
            "I have fever and cough, what should I do?",
            "Is chest pain serious?",
            "How to control blood sugar?",
            "I feel tired all the time",
            "Can I take paracetamol daily?",
            "When should I see a doctor?",
            "Is headache dangerous?",
            "How to reduce BP naturally?",
        ]
        responses = [
            "Please consult a physician and take rest.",
            "Monitor symptoms and seek emergency care if pain increases.",
            "Maintain diet, exercise and medication regularly.",
            "Blood tests may be required.",
            "Avoid self-medication without advice.",
        ]
        languages = ["English", "Tamil", "Hindi", "Telugu"]

        data_chatbot = []
        for i in range(1000):
            data_chatbot.append({
                "symptom": random.choice(symptoms),
                "patient_question": random.choice(questions),
                "doctor_reply": random.choice(responses),
                "language": random.choice(languages),
                "appointment_needed": random.choice(["Yes", "No"])
            })
        df_chatbot = pd.DataFrame(data_chatbot)
        df_chatbot.to_csv(chatbot_file_path, index=False)
    else:
        df_chatbot = pd.read_csv(chatbot_file_path)
    artifacts['df_chatbot'] = df_chatbot # Store for chatbot context if needed

    # --- Module 1: Synthetic Patient Data - Classification, Regression, Clustering ---
    print("\n--- Initializing Module 1 (Patient Data Analytics) ---")
    df_synthetic_processed = df_synthetic.copy()

    # Feature Engineering
    df_synthetic_processed['BP_level'] = pd.cut(
        df_synthetic_processed['systolic_bp'],
        bins=[0, 80, 120, 200],
        labels=['Low', 'Normal', 'High'], ordered=False
    )
    df_synthetic_processed['medication_history'] = ((df_synthetic_processed['diabetes'] == 1) | (df_synthetic_processed['hypertension'] == 1) | (df_synthetic_processed['smoker'] == 1)).astype(int)

    # Data Preprocessing
    imputer = SimpleImputer(strategy='median')
    num_cols_synthetic = ['bmi','systolic_bp','cholesterol','blood_glucose']
    df_synthetic_processed[num_cols_synthetic] = imputer.fit_transform(df_synthetic_processed[num_cols_synthetic])
    artifacts['imputer'] = imputer
    artifacts['num_cols_synthetic'] = num_cols_synthetic

    # Outlier Removal (applied to the training data, new patient data will be transformed only)
    for col in num_cols_synthetic:
        Q1 = df_synthetic_processed[col].quantile(0.25)
        Q3 = df_synthetic_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        df_synthetic_processed = df_synthetic_processed[(df_synthetic_processed[col] >= Q1 - 1.5*IQR) & (df_synthetic_processed[col] <= Q3 + 1.5*IQR)]

    le_synthetic_BP = LabelEncoder()
    df_synthetic_processed['BP_level'] = le_synthetic_BP.fit_transform(df_synthetic_processed['BP_level'])
    artifacts['le_synthetic_BP'] = le_synthetic_BP

    le_synthetic_risk = LabelEncoder()
    df_synthetic_processed['risk_category'] = le_synthetic_risk.fit_transform(df_synthetic_processed['risk_category'])
    artifacts['le_synthetic_risk'] = le_synthetic_risk
    artifacts['risk_category_labels'] = le_synthetic_risk.inverse_transform(sorted(df_synthetic_processed['risk_category'].unique()))

    le_gender = None
    if 'gender' in df_synthetic_processed.columns and df_synthetic_processed['gender'].dtype == 'object':
        le_gender = LabelEncoder()
        df_synthetic_processed['gender'] = le_gender.fit_transform(df_synthetic_processed['gender'])
    artifacts['le_gender'] = le_gender

    # Classification Model
    X_clf_full_features = df_synthetic_processed.drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
    y_clf = df_synthetic_processed['risk_category']

    scaler_clf = StandardScaler()
    X_clf_scaled_for_kbest = scaler_clf.fit_transform(X_clf_full_features)
    X_clf_scaled_for_kbest_df = pd.DataFrame(X_clf_scaled_for_kbest, columns=X_clf_full_features.columns, index=X_clf_full_features.index)
    artifacts['scaler_clf'] = scaler_clf
    artifacts['clf_all_feature_names_before_select'] = list(X_clf_full_features.columns)

    selector_clf = SelectKBest(score_func=f_classif, k=5)
    selector_clf.fit(X_clf_scaled_for_kbest_df, y_clf)
    X_selected_clf = selector_clf.transform(X_clf_scaled_for_kbest_df)
    artifacts['selector_clf'] = selector_clf
    artifacts['clf_feature_names_after_select'] = list(X_clf_full_features.columns[selector_clf.get_support(indices=True)])

    clf_model = RandomForestClassifier(random_state=42)
    clf_model.fit(X_selected_clf, y_clf)
    artifacts['clf_model'] = clf_model
    print("Module 1: Classification model trained.")

    # Regression Model
    reg_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_reg = df_synthetic_processed[reg_feature_names].copy()
    y_reg = df_synthetic_processed["length_of_stay_days"]

    reg_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0))
    ])
    reg_pipeline.fit(X_reg, y_reg)
    artifacts['reg_pipeline'] = reg_pipeline
    artifacts['reg_feature_names'] = reg_feature_names
    print("Module 1: Regression model trained.")

    # Clustering Model
    cluster_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_cluster = df_synthetic_processed[cluster_feature_names]

    scaler_cluster = StandardScaler()
    X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)
    artifacts['scaler_cluster'] = scaler_cluster
    artifacts['cluster_feature_names'] = cluster_feature_names

    kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans_model.fit(X_cluster_scaled)
    artifacts['kmeans_model'] = kmeans_model
    print("Module 1: Clustering model trained.")

    # --- Module 3: Sequence Modeling (LSTM) ---
    print("\n--- Initializing Module 3 (Sequence Modeling) ---")
    FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
    TARGET_ts = 'risk_flag'
    SEQ_LENGTH = 10
    artifacts['SEQ_LENGTH'] = SEQ_LENGTH
    artifacts['FEATURES_ts'] = FEATURES_ts

    scaler_ts = MinMaxScaler()
    df_timeseries[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries[FEATURES_ts])
    artifacts['scaler_ts'] = scaler_ts

    X_ts, y_ts = create_sequences(df_timeseries[FEATURES_ts], df_timeseries[TARGET_ts], SEQ_LENGTH)

    # Using a smaller split for actual training, as full data was processed above
    X_train_ts, _, y_train_ts, _ = train_test_split(
        X_ts, y_ts, test_size=0.1, random_state=42, stratify=y_ts
    ) # Use 90% of data for training the model that will be used by FastAPI

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(32))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(1, activation='sigmoid'))

    model_lstm.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Train Model (reduced epochs for faster startup)
    model_lstm.fit(
        X_train_ts, y_train_ts,
        epochs=5,
        batch_size=32,
        validation_split=0.1,
        verbose=0
    )
    artifacts['model_lstm'] = model_lstm
    print("Module 3: LSTM model trained.")

    # --- Module 4: Sentiment Analysis (BERT) ---
    print("\n--- Initializing Module 4 (Sentiment Analysis) ---")
    texts_feedback = df_feedback["feedback_text"].tolist()
    le_feedback = LabelEncoder()
    labels_feedback = le_feedback.fit_transform(df_feedback["sentiment"])
    artifacts['le_feedback'] = le_feedback
    artifacts['sentiment_labels'] = list(le_feedback.classes_)

    tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
    artifacts['tokenizer_bert'] = tokenizer_bert
    encodings_feedback = tokenizer_bert(
        texts_feedback,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )

    train_idx_feedback, _ = train_test_split(
        range(len(labels_feedback)), test_size=0.1, random_state=42, stratify=labels_feedback
    )
    train_dataset_feedback = FeedbackDataset(
        {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
        labels_feedback[train_idx_feedback]
    )

    model_sentiment = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(set(labels_feedback))
    )

    optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
    model_sentiment.to(device)
    model_sentiment.train()

    # Simplified training loop for quick initialization in FastAPI
    train_loader = torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8, shuffle=True)
    for epoch in range(1):
        for i, batch in enumerate(train_loader):
            if i > 20: break # Process only a few batches for speed
            optimizer_sentiment.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs_sentiment = model_sentiment(**batch)
            loss_sentiment = outputs_sentiment.loss
            loss_sentiment.backward()
            optimizer_sentiment.step()
    model_sentiment.eval()
    artifacts['model_sentiment'] = model_sentiment
    print("Module 4: BERT sentiment model loaded and partially fine-tuned.")

    # --- Module 5: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---
    print("\n--- Initializing Module 5 (Generative AI) ---")
    tokenizer_biobert = None
    model_biobert_embeddings = None
    kmeans_biobert = None
    generator_biogpt = None

    if df_clinical is not None:
        df_clinical["clinical_note"] = df_clinical["clinical_note"].astype(str).apply(clean_text)

        # BioBERT for Embeddings
        MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
        tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings.to(device)
        artifacts['tokenizer_biobert'] = tokenizer_biobert
        artifacts['model_biobert_embeddings'] = model_biobert_embeddings

        # Generate embeddings for a sample (to fit KMeans)
        sample_size = min(200, len(df_clinical)) # Limit for initialization speed
        if sample_size > 0:
            sample_df_clinical = df_clinical.sample(sample_size, random_state=42) if len(df_clinical) > sample_size else df_clinical.copy()
            embeddings_biobert = np.vstack(sample_df_clinical["clinical_note"].apply(
                lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings, device).flatten()
            ))
            kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
            kmeans_biobert.fit(embeddings_biobert)
            artifacts['kmeans_biobert'] = kmeans_biobert
            print("Module 5: BioBERT tokenizer and embedding model loaded, KMeans fitted.")
        else:
            print("Module 5: No clinical notes data available to fit KMeans for BioBERT embeddings.")

        # BioGPT for Text Generation
        try:
            generator_biogpt = pipeline(
                "text-generation",
                model="microsoft/BioGPT",
                device=0 if torch.cuda.is_available() else -1 # Use GPU if available
            )
            artifacts['generator_biogpt'] = generator_biogpt
            print("Module 5: BioGPT text generation pipeline loaded.")
        except Exception as e:
            print(f"Module 5: Could not load BioGPT model: {e}. Text generation and chatbot functionality will be limited.")
    else:
        print("Module 5: Clinical notes data not available, skipping BioBERT and BioGPT initialization.")

    # --- Module 6: Healthcare Chatbot and Translator ---
    print("\n--- Initializing Module 6 (Chatbot & Translator) ---")
    model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
    tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
    translator_model = MarianMTModel.from_pretrained(model_name_translator)
    translator_model.to(device)
    artifacts['tokenizer_translator'] = tokenizer_translator
    artifacts['translator_model'] = translator_model
    print("Module 6: MarianMT translator model loaded.")

    print("\nAll models and preprocessors initialized successfully!")
    return artifacts

# Global variable to store initialized artifacts
models_and_preprocessors = load_models_and_preprocessors()


--- Loading Datasets ---

--- Initializing Module 1 (Patient Data Analytics) ---
Module 1: Classification model trained.
Module 1: Regression model trained.
Module 1: Clustering model trained.

--- Initializing Module 3 (Sequence Modeling) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Module 3: LSTM model trained.

--- Initializing Module 4 (Sentiment Analysis) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Module 6: MarianMT translator model loaded.

All models and preprocessors initialized successfully!


**Reasoning**:
The previous output indicated that the BioGPT model could not be loaded due to a missing `sacremoses` dependency. To ensure the BioGPT model loads correctly and all functionalities are available, I need to install `sacremoses`.



In [4]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 50.0 MB/s eta 0:00:00


**Reasoning**:
The `sacremoses` library has been successfully installed. Now, I need to re-execute the `load_models_and_preprocessors()` function to ensure that the BioGPT model and all other models and preprocessors are loaded correctly with the dependency resolved. This will finalize the setup for the FastAPI application's backend.



In [5]:
import pandas as pd
import numpy as np
import torch
import random
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from mlxtend.frequent_patterns import apriori, association_rules
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
from torch.optim import AdamW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress warnings from mlxtend regarding DataFrame types
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mlxtend')

# --- Helper Functions (Replicated from original notebook) ---
def clean_text(text):
    """Cleans clinical notes text by lowercasing and standardizing whitespace."""
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def create_sequences(data, target, seq_length=10):
    """Creates time-series sequences using a sliding window technique."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(target.iloc[i+seq_length])
    return np.array(X), np.array(y)

class FeedbackDataset(torch.utils.data.Dataset):
    """Custom Dataset for BERT sentiment analysis."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def get_embedding(text, tokenizer_bert, model_bert, device):
    """Generates BERT embeddings for a given text."""
    inputs = tokenizer_bert(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_bert(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

# --- Initialization Function ---
def load_models_and_preprocessors():
    """Initializes and loads all models and preprocessors for the FastAPI application."""
    artifacts = {}
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    artifacts['device'] = device

    print("\n--- Loading Datasets ---")
    # Load datasets
    df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")
    df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")
    df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')
    df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")
    artifacts['df_apriori'] = df_apriori # Store for association rules

    df_clinical = None
    try:
        df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
    except FileNotFoundError:
        print("Warning: '/content/healthai_clinical_notes_1000.csv' not found. BioBERT and BioGPT features will be limited.")

    # Ensure chatbot dataset exists
    chatbot_file_path = "/content/healthcare_chatbot_translation_dataset.csv"
    if not os.path.exists(chatbot_file_path):
        symptoms = [
            "fever", "cough", "headache", "chest pain", "breathing difficulty",
            "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
        ]
        questions = [
            "I have fever and cough, what should I do?",
            "Is chest pain serious?",
            "How to control blood sugar?",
            "I feel tired all the time",
            "Can I take paracetamol daily?",
            "When should I see a doctor?",
            "Is headache dangerous?",
            "How to reduce BP naturally?",
        ]
        responses = [
            "Please consult a physician and take rest.",
            "Monitor symptoms and seek emergency care if pain increases.",
            "Maintain diet, exercise and medication regularly.",
            "Blood tests may be required.",
            "Avoid self-medication without advice.",
        ]
        languages = ["English", "Tamil", "Hindi", "Telugu"]

        data_chatbot = []
        for i in range(1000):
            data_chatbot.append({
                "symptom": random.choice(symptoms),
                "patient_question": random.choice(questions),
                "doctor_reply": random.choice(responses),
                "language": random.choice(languages),
                "appointment_needed": random.choice(["Yes", "No"])
            })
        df_chatbot = pd.DataFrame(data_chatbot)
        df_chatbot.to_csv(chatbot_file_path, index=False)
    else:
        df_chatbot = pd.read_csv(chatbot_file_path)
    artifacts['df_chatbot'] = df_chatbot # Store for chatbot context if needed

    # --- Module 1: Synthetic Patient Data - Classification, Regression, Clustering ---
    print("\n--- Initializing Module 1 (Patient Data Analytics) ---")
    df_synthetic_processed = df_synthetic.copy()

    # Feature Engineering
    df_synthetic_processed['BP_level'] = pd.cut(
        df_synthetic_processed['systolic_bp'],
        bins=[0, 80, 120, 200],
        labels=['Low', 'Normal', 'High'], ordered=False
    )
    df_synthetic_processed['medication_history'] = ((df_synthetic_processed['diabetes'] == 1) | (df_synthetic_processed['hypertension'] == 1) | (df_synthetic_processed['smoker'] == 1)).astype(int)

    # Data Preprocessing
    imputer = SimpleImputer(strategy='median')
    num_cols_synthetic = ['bmi','systolic_bp','cholesterol','blood_glucose']
    df_synthetic_processed[num_cols_synthetic] = imputer.fit_transform(df_synthetic_processed[num_cols_synthetic])
    artifacts['imputer'] = imputer
    artifacts['num_cols_synthetic'] = num_cols_synthetic

    # Outlier Removal (applied to the training data, new patient data will be transformed only)
    for col in num_cols_synthetic:
        Q1 = df_synthetic_processed[col].quantile(0.25)
        Q3 = df_synthetic_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        df_synthetic_processed = df_synthetic_processed[(df_synthetic_processed[col] >= Q1 - 1.5*IQR) & (df_synthetic_processed[col] <= Q3 + 1.5*IQR)]

    le_synthetic_BP = LabelEncoder()
    df_synthetic_processed['BP_level'] = le_synthetic_BP.fit_transform(df_synthetic_processed['BP_level'])
    artifacts['le_synthetic_BP'] = le_synthetic_BP

    le_synthetic_risk = LabelEncoder()
    df_synthetic_processed['risk_category'] = le_synthetic_risk.fit_transform(df_synthetic_processed['risk_category'])
    artifacts['le_synthetic_risk'] = le_synthetic_risk
    artifacts['risk_category_labels'] = le_synthetic_risk.inverse_transform(sorted(df_synthetic_processed['risk_category'].unique()))

    le_gender = None
    if 'gender' in df_synthetic_processed.columns and df_synthetic_processed['gender'].dtype == 'object':
        le_gender = LabelEncoder()
        df_synthetic_processed['gender'] = le_gender.fit_transform(df_synthetic_processed['gender'])
    artifacts['le_gender'] = le_gender

    # Classification Model
    X_clf_full_features = df_synthetic_processed.drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
    y_clf = df_synthetic_processed['risk_category']

    scaler_clf = StandardScaler()
    X_clf_scaled_for_kbest = scaler_clf.fit_transform(X_clf_full_features)
    X_clf_scaled_for_kbest_df = pd.DataFrame(X_clf_scaled_for_kbest, columns=X_clf_full_features.columns, index=X_clf_full_features.index)
    artifacts['scaler_clf'] = scaler_clf
    artifacts['clf_all_feature_names_before_select'] = list(X_clf_full_features.columns)

    selector_clf = SelectKBest(score_func=f_classif, k=5)
    selector_clf.fit(X_clf_scaled_for_kbest_df, y_clf)
    X_selected_clf = selector_clf.transform(X_clf_scaled_for_kbest_df)
    artifacts['selector_clf'] = selector_clf
    artifacts['clf_feature_names_after_select'] = list(X_clf_full_features.columns[selector_clf.get_support(indices=True)])

    clf_model = RandomForestClassifier(random_state=42)
    clf_model.fit(X_selected_clf, y_clf)
    artifacts['clf_model'] = clf_model
    print("Module 1: Classification model trained.")

    # Regression Model
    reg_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_reg = df_synthetic_processed[reg_feature_names].copy()
    y_reg = df_synthetic_processed["length_of_stay_days"]

    reg_pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0))
    ])
    reg_pipeline.fit(X_reg, y_reg)
    artifacts['reg_pipeline'] = reg_pipeline
    artifacts['reg_feature_names'] = reg_feature_names
    print("Module 1: Regression model trained.")

    # Clustering Model
    cluster_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_cluster = df_synthetic_processed[cluster_feature_names]

    scaler_cluster = StandardScaler()
    X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)
    artifacts['scaler_cluster'] = scaler_cluster
    artifacts['cluster_feature_names'] = cluster_feature_names

    kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans_model.fit(X_cluster_scaled)
    artifacts['kmeans_model'] = kmeans_model
    print("Module 1: Clustering model trained.")

    # --- Module 3: Sequence Modeling (LSTM) ---
    print("\n--- Initializing Module 3 (Sequence Modeling) ---")
    FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
    TARGET_ts = 'risk_flag'
    SEQ_LENGTH = 10
    artifacts['SEQ_LENGTH'] = SEQ_LENGTH
    artifacts['FEATURES_ts'] = FEATURES_ts

    scaler_ts = MinMaxScaler()
    df_timeseries[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries[FEATURES_ts])
    artifacts['scaler_ts'] = scaler_ts

    X_ts, y_ts = create_sequences(df_timeseries[FEATURES_ts], df_timeseries[TARGET_ts], SEQ_LENGTH)

    # Using a smaller split for actual training, as full data was processed above
    X_train_ts, _, y_train_ts, _ = train_test_split(
        X_ts, y_ts, test_size=0.1, random_state=42, stratify=y_ts
    ) # Use 90% of data for training the model that will be used by FastAPI

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(32))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(1, activation='sigmoid'))

    model_lstm.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Train Model (reduced epochs for faster startup)
    model_lstm.fit(
        X_train_ts, y_train_ts,
        epochs=5,
        batch_size=32,
        validation_split=0.1,
        verbose=0
    )
    artifacts['model_lstm'] = model_lstm
    print("Module 3: LSTM model trained.")

    # --- Module 4: Sentiment Analysis (BERT) ---
    print("\n--- Initializing Module 4 (Sentiment Analysis) ---")
    texts_feedback = df_feedback["feedback_text"].tolist()
    le_feedback = LabelEncoder()
    labels_feedback = le_feedback.fit_transform(df_feedback["sentiment"])
    artifacts['le_feedback'] = le_feedback
    artifacts['sentiment_labels'] = list(le_feedback.classes_)

    tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
    artifacts['tokenizer_bert'] = tokenizer_bert
    encodings_feedback = tokenizer_bert(
        texts_feedback,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )

    train_idx_feedback, _ = train_test_split(
        range(len(labels_feedback)), test_size=0.1, random_state=42, stratify=labels_feedback
    )
    train_dataset_feedback = FeedbackDataset(
        {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
        labels_feedback[train_idx_feedback]
    )

    model_sentiment = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(set(labels_feedback))
    )

    optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
    model_sentiment.to(device)
    model_sentiment.train()

    # Simplified training loop for quick initialization in FastAPI
    train_loader = torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8, shuffle=True)
    for epoch in range(1):
        for i, batch in enumerate(train_loader):
            if i > 20: break # Process only a few batches for speed
            optimizer_sentiment.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs_sentiment = model_sentiment(**batch)
            loss_sentiment = outputs_sentiment.loss
            loss_sentiment.backward()
            optimizer_sentiment.step()
    model_sentiment.eval()
    artifacts['model_sentiment'] = model_sentiment
    print("Module 4: BERT sentiment model loaded and partially fine-tuned.")

    # --- Module 5: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---
    print("\n--- Initializing Module 5 (Generative AI) ---")
    tokenizer_biobert = None
    model_biobert_embeddings = None
    kmeans_biobert = None
    generator_biogpt = None

    if df_clinical is not None:
        df_clinical["clinical_note"] = df_clinical["clinical_note"].astype(str).apply(clean_text)

        # BioBERT for Embeddings
        MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
        tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings.to(device)
        artifacts['tokenizer_biobert'] = tokenizer_biobert
        artifacts['model_biobert_embeddings'] = model_biobert_embeddings

        # Generate embeddings for a sample (to fit KMeans)
        sample_size = min(200, len(df_clinical)) # Limit for initialization speed
        if sample_size > 0:
            sample_df_clinical = df_clinical.sample(sample_size, random_state=42) if len(df_clinical) > sample_size else df_clinical.copy()
            embeddings_biobert = np.vstack(sample_df_clinical["clinical_note"].apply(
                lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings, device).flatten()
            ))
            kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
            kmeans_biobert.fit(embeddings_biobert)
            artifacts['kmeans_biobert'] = kmeans_biobert
            print("Module 5: BioBERT tokenizer and embedding model loaded, KMeans fitted.")
        else:
            print("Module 5: No clinical notes data available to fit KMeans for BioBERT embeddings.")

        # BioGPT for Text Generation
        try:
            generator_biogpt = pipeline(
                "text-generation",
                model="microsoft/BioGPT",
                device=0 if torch.cuda.is_available() else -1 # Use GPU if available
            )
            artifacts['generator_biogpt'] = generator_biogpt
            print("Module 5: BioGPT text generation pipeline loaded.")
        except Exception as e:
            print(f"Module 5: Could not load BioGPT model: {e}. Text generation and chatbot functionality will be limited.")
    else:
        print("Module 5: Clinical notes data not available, skipping BioBERT and BioGPT initialization.")

    # --- Module 6: Healthcare Chatbot and Translator ---
    print("\n--- Initializing Module 6 (Chatbot & Translator) ---")
    model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
    tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
    translator_model = MarianMTModel.from_pretrained(model_name_translator)
    translator_model.to(device)
    artifacts['tokenizer_translator'] = tokenizer_translator
    artifacts['translator_model'] = translator_model
    print("Module 6: MarianMT translator model loaded.")

    print("\nAll models and preprocessors initialized successfully!")
    return artifacts

# Global variable to store initialized artifacts
models_and_preprocessors = load_models_and_preprocessors()



--- Loading Datasets ---

--- Initializing Module 1 (Patient Data Analytics) ---
Module 1: Classification model trained.
Module 1: Regression model trained.
Module 1: Clustering model trained.

--- Initializing Module 3 (Sequence Modeling) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Module 3: LSTM model trained.

--- Initializing Module 4 (Sentiment Analysis) ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Module 4: BERT sentiment model loaded and partially fine-tuned.

--- Initializing Module 5 (Generative AI) ---
Module 5: BioBERT tokenizer and embedding model loaded, KMeans fitted.


Device set to use cuda:0


Module 5: BioGPT text generation pipeline loaded.

--- Initializing Module 6 (Chatbot & Translator) ---
Module 6: MarianMT translator model loaded.

All models and preprocessors initialized successfully!


**Reasoning**:
All models and preprocessors, including BioGPT, have been successfully initialized. The next step is to create the FastAPI application instance.



In [6]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse

# Initialize FastAPI app
app = FastAPI(
    title="HealthAI API",
    description="API for various HealthAI modules including Patient Data Analytics, Association Rules, Sequence Modeling, Sentiment Analysis, Generative AI, and Chatbot/Translator."
)

@app.get("/health", response_class=JSONResponse, tags=["System"]) # type: ignore
def health_check():
    """Health check endpoint to ensure the API is running."""
    return {"status": "ok", "message": "HealthAI API is running"}

print("FastAPI app instance created and health check endpoint defined.")

FastAPI app instance created and health check endpoint defined.


**Reasoning**:
With the FastAPI application instance and health check endpoint successfully created, the next step is to define the API endpoints for each of the modules, starting with Patient Data Analytics (Classification, Regression, Clustering). I will define three endpoints for classification, regression, and clustering, using the previously loaded models and preprocessors, and the Pydantic schemas for request and response validation.



In [7]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field
import pandas as pd
import numpy as np

# Assuming 'app' and 'models_and_preprocessors' are already defined and populated
# from previous cells.

# Import Pydantic models for request/response schemas (defined in previous cells)
from typing import List, Optional

# 1. Patient Data Model for Classification, Regression, Clustering
class PatientData(BaseModel):
    age: int = Field(..., ge=1, le=100, description="Patient's age in years")
    bmi: float = Field(..., ge=10.0, le=50.0, description="Body Mass Index")
    systolic_bp: int = Field(..., ge=70, le=200, description="Systolic Blood Pressure (mmHg)")
    cholesterol: int = Field(..., ge=100, le=300, description="Cholesterol level (mg/dL)")
    blood_glucose: int = Field(..., ge=70, le=200, description="Blood Glucose level (mg/dL)")
    diabetes: int = Field(..., ge=0, le=1, description="1 if patient has diabetes, 0 otherwise")
    hypertension: int = Field(..., ge=0, le=1, description="1 if patient has hypertension, 0 otherwise")
    smoker: int = Field(..., ge=0, le=1, description="1 if patient is a smoker, 0 otherwise")
    prev_hospitalizations: int = Field(..., ge=0, le=10, description="Number of previous hospitalizations")
    gender: str = Field(..., description="Patient's gender (Male, Female, Other)")

# 2. Risk Prediction Response Model
class RiskPredictionResponse(BaseModel):
    predicted_risk_category: str = Field(..., description="Predicted risk category (e.g., 'Low', 'Normal', 'High')")

# 3. Length of Stay Prediction Response Model
class LengthOfStayPredictionResponse(BaseModel):
    predicted_length_of_stay: float = Field(..., description="Predicted length of hospital stay in days")

# 4. Cluster Assignment Response Model
class ClusterAssignmentResponse(BaseModel):
    assigned_cluster: int = Field(..., description="Assigned cluster ID for the patient")

# 5. Association Rules Request Model
class AssociationRulesRequest(BaseModel):
    min_support: float = Field(0.1, ge=0.01, le=1.0, description="Minimum support for frequent itemsets")
    min_confidence: float = Field(0.6, ge=0.01, le=1.0, description="Minimum confidence for association rules")

# 6. Association Rule Model
class AssociationRule(BaseModel):
    antecedents: List[str] = Field(..., description="Items in the antecedent of the rule")
    consequents: List[str] = Field(..., description="Items in the consequent of the rule")
    support: float = Field(..., description="Support of the rule")
    confidence: float = Field(..., description="Confidence of the rule")
    lift: float = Field(..., description="Lift of the rule")

# 7. Association Rules Response Model
class AssociationRulesResponse(BaseModel):
    rules: List[AssociationRule] = Field(..., description="List of discovered association rules")

# 8. Vitals Reading Model for LSTM
class VitalsReading(BaseModel):
    heart_rate: int = Field(..., ge=40, le=180, description="Heart rate (bpm)")
    systolic_bp: int = Field(..., ge=60, le=200, description="Systolic blood pressure (mmHg)")
    spo2: int = Field(..., ge=70, le=100, description="Blood oxygen saturation (%)")

# 9. LSTM Request Model
class LSTMRequest(BaseModel):
    vitals_sequence: List[VitalsReading] = Field(..., min_length=10, max_length=10, description="Sequence of 10 vital sign readings")

# 10. LSTM Prediction Response Model
class LSTMPredictionResponse(BaseModel):
    predicted_deterioration_risk: str = Field(..., description="Predicted risk of patient deterioration ('Stable' or 'High deterioration risk')")
    probability: float = Field(..., ge=0.0, le=1.0, description="Probability of high deterioration risk")

# 11. Sentiment Request Model
class SentimentRequest(BaseModel):
    feedback_text: str = Field(..., description="Text of the patient feedback")

# 12. Sentiment Response Model
class SentimentResponse(BaseModel):
    predicted_sentiment: str = Field(..., description="Predicted sentiment of the feedback ('Negative', 'Neutral', 'Positive')")

# 13. BioGPT Request Model
class BioGPTRequest(BaseModel):
    prompt: str = Field(..., description="Text prompt for BioGPT generation")
    max_length: int = Field(80, ge=50, le=200, description="Maximum length of the generated text")

# 14. BioGPT Response Model
class BioGPTResponse(BaseModel):
    generated_text: str = Field(..., description="Generated clinical text by BioGPT")

# 15. Chatbot Request Model
class ChatbotRequest(BaseModel):
    user_question: str = Field(..., description="User's question to the healthcare chatbot")

# 16. Chatbot Response Model
class ChatbotResponse(BaseModel):
    chatbot_response: str = Field(..., description="Response from the healthcare chatbot")

# 17. Translator Request Model
class TranslatorRequest(BaseModel):
    text_to_translate: str = Field(..., description="Medical text to be translated")

# 18. Translator Response Model
class TranslatorResponse(BaseModel):
    translated_text: str = Field(..., description="Translated medical text")

# 19. Combined Chatbot Translator Request Model
class CombinedChatbotTranslatorRequest(BaseModel):
    user_question: str = Field(..., description="User's question for the combined chatbot and translator")
    target_language: Optional[str] = Field(None, description="Desired target language for translation (e.g., 'Tamil', 'Hindi', 'Spanish'). Note: Actual translation capability depends on the loaded model.")

# 20. Combined Chatbot Translator Response Model
class CombinedChatbotTranslatorResponse(BaseModel):
    english_response: str = Field(..., description="English response from the chatbot")
    translated_response: str = Field(..., description="Translated response (multilingual interpretation)")

# --- Helper function for preprocessing patient data (common for Module 1 endpoints) ---
def preprocess_patient_data(patient_data: PatientData):
    data_dict = patient_data.dict()
    new_patient_df = pd.DataFrame([data_dict])

    # 1. Feature Engineering
    new_patient_df['BP_level'] = pd.cut(
        new_patient_df['systolic_bp'],
        bins=[0, 80, 120, 200],
        labels=['Low', 'Normal', 'High'], ordered=False
    )
    new_patient_df['medication_history'] = ((new_patient_df['diabetes'] == 1) | (new_patient_df['hypertension'] == 1) | (new_patient_df['smoker'] == 1)).astype(int)

    # 2. Imputation (only transform, not fit)
    new_patient_df[models_and_preprocessors['num_cols_synthetic']] = models_and_preprocessors['imputer'].transform(new_patient_df[models_and_preprocessors['num_cols_synthetic']])

    # 3. Label Encoding (only transform, not fit)
    new_patient_df['BP_level'] = models_and_preprocessors['le_synthetic_BP'].transform(new_patient_df['BP_level'])
    if models_and_preprocessors['le_gender'] is not None and 'gender' in new_patient_df.columns:
        try:
            new_patient_df['gender'] = models_and_preprocessors['le_gender'].transform(new_patient_df['gender'])
        except ValueError:
            # Handle unseen gender category gracefully for demo (e.g., default or raise specific error)
            raise HTTPException(status_code=400, detail="Unseen gender category. Please use 'Male', 'Female', or 'Other'.")

    return new_patient_df

# --- Module 1 Endpoints: Patient Data Analytics ---

@app.post("/predict-risk-category", response_model=RiskPredictionResponse, tags=["Module 1 - Patient Data Analytics"])
async def predict_risk_category(patient_data: PatientData):
    """Predicts the risk category for a new patient."""
    processed_data = preprocess_patient_data(patient_data)

    # Align columns with training data for classification
    # Ensure all features expected by the scaler and selector are present, in correct order.
    X_clf_aligned = pd.DataFrame(columns=models_and_preprocessors['clf_all_feature_names_before_select'])
    for col in models_and_preprocessors['clf_all_feature_names_before_select']:
        if col in processed_data.columns:
            X_clf_aligned[col] = processed_data[col]
        else:
            X_clf_aligned[col] = 0 # Default value for any missing feature, or handle more robustly

    # Scale features using the fitted scaler_clf
    new_patient_scaled = models_and_preprocessors['scaler_clf'].transform(X_clf_aligned)

    # Apply feature selection
    new_patient_selected = models_and_preprocessors['selector_clf'].transform(new_patient_scaled)

    prediction_idx = models_and_preprocessors['clf_model'].predict(new_patient_selected)
    predicted_risk = models_and_preprocessors['le_synthetic_risk'].inverse_transform(prediction_idx)[0]
    return RiskPredictionResponse(predicted_risk_category=predicted_risk)

@app.post("/predict-length-of-stay", response_model=LengthOfStayPredictionResponse, tags=["Module 1 - Patient Data Analytics"])
async def predict_length_of_stay(patient_data: PatientData):
    """Predicts the length of hospital stay for a new patient."""
    processed_data = preprocess_patient_data(patient_data)

    # Align columns with regression model's expected features
    X_reg_aligned = processed_data[models_and_preprocessors['reg_feature_names']] # Ensure correct column order

    prediction_reg = models_and_preprocessors['reg_pipeline'].predict(X_reg_aligned)
    return LengthOfStayPredictionResponse(predicted_length_of_stay=float(prediction_reg[0]))

@app.post("/assign-patient-cluster", response_model=ClusterAssignmentResponse, tags=["Module 1 - Patient Data Analytics"])
async def assign_patient_cluster(patient_data: PatientData):
    """Assigns a new patient to a cluster based on their features."""
    processed_data = preprocess_patient_data(patient_data)

    # Align columns with clustering model's expected features
    X_cluster_aligned = processed_data[models_and_preprocessors['cluster_feature_names']] # Ensure correct column order

    # Scale features using the fitted scaler_cluster
    new_patient_cluster_scaled = models_and_preprocessors['scaler_cluster'].transform(X_cluster_aligned)

    predicted_cluster = models_and_preprocessors['kmeans_model'].predict(new_patient_cluster_scaled)
    return ClusterAssignmentResponse(assigned_cluster=int(predicted_cluster[0]))

print("FastAPI endpoints for Module 1 (Patient Data Analytics) defined.")


FastAPI endpoints for Module 1 (Patient Data Analytics) defined.


**Reasoning**:
The next step is to define the API endpoint for Module 2: Association Rules. This endpoint will take `min_support` and `min_confidence` as parameters, generate frequent itemsets and association rules using the preloaded `df_apriori` data, and return a list of rules.



In [8]:
import pandas as pd
import numpy as np

# --- Module 2 Endpoints: Association Rules ---

@app.post("/association-rules", response_model=AssociationRulesResponse, tags=["Module 2 - Association Rules"])
async def get_association_rules(request: AssociationRulesRequest):
    """Generates and returns association rules based on medical transaction data."""
    df_assoc = models_and_preprocessors['df_apriori'].drop('patient_id', axis=1)
    df_assoc_bool = df_assoc.astype(bool)

    try:
        frequent_itemsets = apriori(
            df_assoc_bool,
            min_support=request.min_support,
            use_colnames=True
        )

        rules = association_rules(
            frequent_itemsets,
            metric='confidence',
            min_threshold=request.min_confidence
        )

        # Convert frozensets to lists of strings for JSON serialization
        rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
        rules['consequents'] = rules['consequents'].apply(lambda x: list(x))

        # Select and rename columns to match AssociationRule Pydantic model
        rules_list = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_dict(orient='records')

        return AssociationRulesResponse(rules=rules_list)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating association rules: {e}")

print("FastAPI endpoint for Module 2 (Association Rules) defined.")

FastAPI endpoint for Module 2 (Association Rules) defined.


**Reasoning**:
The API endpoint for Module 2 has been defined. The next logical step is to create the API endpoint for Module 3: Sequence Modeling (LSTM), which predicts patient deterioration risk based on a sequence of vital signs. This endpoint will utilize the preloaded LSTM model and its associated preprocessors.



In [9]:
import pandas as pd
import numpy as np

# --- Module 3 Endpoints: Sequence Modeling (LSTM) ---

@app.post("/predict-deterioration-risk", response_model=LSTMPredictionResponse, tags=["Module 3 - Sequence Modeling (LSTM)"])
async def predict_deterioration_risk(request: LSTMRequest):
    """Predicts the deterioration risk for a patient based on a sequence of vital signs."""
    # Convert Pydantic list of VitalsReading to a DataFrame
    vitals_df = pd.DataFrame([v.dict() for v in request.vitals_sequence])

    # Scale the vital signs using the pre-fitted scaler
    scaled_vitals = models_and_preprocessors['scaler_ts'].transform(vitals_df[models_and_preprocessors['FEATURES_ts']])

    # Reshape for LSTM model: (1, SEQ_LENGTH, num_features)
    reshaped_vitals = scaled_vitals.reshape(1, models_and_preprocessors['SEQ_LENGTH'], len(models_and_preprocessors['FEATURES_ts']))

    # Make prediction
    prediction_prob = models_and_preprocessors['model_lstm'].predict(reshaped_vitals, verbose=0)[0][0]

    # Determine risk category
    predicted_risk = "High deterioration risk" if prediction_prob > 0.5 else "Stable"

    return LSTMPredictionResponse(predicted_deterioration_risk=predicted_risk, probability=float(prediction_prob))

print("FastAPI endpoint for Module 3 (Sequence Modeling) defined.")


FastAPI endpoint for Module 3 (Sequence Modeling) defined.


**Reasoning**:
The API endpoint for Module 3 has been defined. The next logical step is to create the API endpoint for Module 4: Sentiment Analysis (BERT), which will take patient feedback text, process it using the preloaded BERT model and tokenizer, and return the predicted sentiment.



In [10]:
import torch

# --- Module 4 Endpoints: Sentiment Analysis (BERT) ---

@app.post("/analyze-sentiment", response_model=SentimentResponse, tags=["Module 4 - Sentiment Analysis (BERT)"])
async def analyze_sentiment(request: SentimentRequest):
    """Analyzes the sentiment of patient feedback using a BERT model."""
    inputs_sentiment = models_and_preprocessors['tokenizer_bert'](
        request.feedback_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs_sentiment = {k: v.to(models_and_preprocessors['device']) for k, v in inputs_sentiment.items()}

    with torch.no_grad():
        outputs_sentiment_pred = models_and_preprocessors['model_sentiment'](**inputs_sentiment)

    pred_sentiment_idx = torch.argmax(outputs_sentiment_pred.logits, dim=1).item()
    predicted_sentiment = models_and_preprocessors['le_feedback'].inverse_transform([pred_sentiment_idx])[0]

    return SentimentResponse(predicted_sentiment=predicted_sentiment)

print("FastAPI endpoint for Module 4 (Sentiment Analysis) defined.")

FastAPI endpoint for Module 4 (Sentiment Analysis) defined.


**Reasoning**:
The API endpoint for Module 4 has been defined. The next logical step is to create the API endpoints for Module 5: Generative AI (BioBERT & BioGPT). This involves two endpoints: one for generating BioBERT embeddings and clustering for a clinical note, and another for generating clinical text using BioGPT from a prompt.



In [11]:
import torch
import numpy as np
import pandas as pd

# --- Module 5 Endpoints: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---

@app.post("/analyze-clinical-note", tags=["Module 5 - Generative AI"])
async def analyze_clinical_note(text_data: SentimentRequest): # Reusing SentimentRequest for text input
    """Generates BioBERT embeddings for a clinical note and assigns it to a cluster."""
    if not models_and_preprocessors.get('tokenizer_biobert') or not models_and_preprocessors.get('model_biobert_embeddings') or not models_and_preprocessors.get('kmeans_biobert'):
        raise HTTPException(status_code=503, detail="BioBERT models or KMeans not initialized. Clinical notes data might be missing or models failed to load.")

    cleaned_text = clean_text(text_data.feedback_text)
    embedding = get_embedding(
        cleaned_text,
        models_and_preprocessors['tokenizer_biobert'],
        models_and_preprocessors['model_biobert_embeddings'],
        models_and_preprocessors['device']
    )

    predicted_cluster = models_and_preprocessors['kmeans_biobert'].predict(embedding)[0]

    return ClusterAssignmentResponse(assigned_cluster=int(predicted_cluster))

@app.post("/generate-clinical-text", response_model=BioGPTResponse, tags=["Module 5 - Generative AI"])
async def generate_clinical_text(request: BioGPTRequest):
    """Generates clinical text using BioGPT based on a given prompt."""
    if not models_and_preprocessors.get('generator_biogpt'):
        raise HTTPException(status_code=503, detail="BioGPT model not initialized. Check logs for loading errors (e.g., missing sacremoses).")

    try:
        generated_text = models_and_preprocessors['generator_biogpt'](
            request.prompt,
            max_length=request.max_length,
            num_return_sequences=1,
            pad_token_id=models_and_preprocessors['generator_biogpt'].tokenizer.eos_token_id
        )[0]["generated_text"]
        return BioGPTResponse(generated_text=generated_text)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error during text generation: {e}")

print("FastAPI endpoints for Module 5 (Generative AI) defined.")

FastAPI endpoints for Module 5 (Generative AI) defined.


**Reasoning**:
The API endpoints for Module 5 have been defined. The next logical step is to create the API endpoints for Module 6: Healthcare Chatbot and Translator, which includes a chatbot, a translator, and a combined service.



In [12]:
import torch

# --- Module 6 Endpoints: Healthcare Chatbot and Translator ---

def healthcare_chatbot_api(user_input: str) -> str:
    """Chatbot function using the BioGPT model for medical guidance, or a generic response."""
    if models_and_preprocessors.get("generator_biogpt"):
        prompt = f"""
        You are a healthcare assistant. Provide safe medical guidance and symptom triage. Avoid medical diagnosis. Focus on general health advice. If you cannot provide a specific answer, advise consulting a doctor. Do not generate information that is not directly related to the prompt. Limit your response to 100 words.
        Patient says: {user_input}
        Response:
        """
        response = models_and_preprocessors['generator_biogpt'](prompt, max_length=150, num_return_sequences=1, pad_token_id=models_and_preprocessors['generator_biogpt'].tokenizer.eos_token_id)[0]["generated_text"]

        # Clean up the response to remove the prompt part if BioGPT echoes it
        response_lines = response.split('\n')
        clean_response = []
        capture = False
        for line in response_lines:
            if line.strip().startswith("Response:"):
                capture = True
                continue
            if capture and line.strip():
                clean_response.append(line.strip())

        if clean_response:
            return " ".join(clean_response)
        else:
            # Fallback cleanup if parsing fails, by removing the exact prompt string
            return response.replace(prompt, "").strip()
    else:
        return f"Hello! As a healthcare assistant, I recommend consulting a doctor for '{user_input}'. Please note that a specialized AI model for medical guidance is currently unavailable due to BioGPT loading issues. Always consult a qualified medical professional for health concerns."


def translate_medical_text_api(text: str) -> str:
    """Translates medical text using the MarianMT model."""
    if not models_and_preprocessors.get('tokenizer_translator') or not models_and_preprocessors.get('translator_model'):
        raise HTTPException(status_code=503, detail="Translator model not initialized.")

    inputs = models_and_preprocessors['tokenizer_translator'](text, return_tensors="pt", padding=True)
    inputs = {k: v.to(models_and_preprocessors['device']) for k, v in inputs.items()}
    translated = models_and_preprocessors['translator_model'].generate(**inputs)
    return models_and_preprocessors['tokenizer_translator'].decode(translated[0], skip_special_tokens=True)


@app.post("/chatbot", response_model=ChatbotResponse, tags=["Module 6 - Chatbot & Translator"])
async def chatbot_endpoint(request: ChatbotRequest):
    """Provides a medical guidance response from the AI chatbot."""
    response = healthcare_chatbot_api(request.user_question)
    return ChatbotResponse(chatbot_response=response)

@app.post("/translate", response_model=TranslatorResponse, tags=["Module 6 - Chatbot & Translator"])
async def translate_endpoint(request: TranslatorRequest):
    """Translates medical text from English to a multilingual interpretation."""
    translated_text = translate_medical_text_api(request.text_to_translate)
    return TranslatorResponse(translated_text=translated_text)

@app.post("/chatbot-translate", response_model=CombinedChatbotTranslatorResponse, tags=["Module 6 - Chatbot & Translator"])
async def combined_chatbot_translator_endpoint(request: CombinedChatbotTranslatorRequest):
    """Combines chatbot response generation with translation."""
    english_reply = healthcare_chatbot_api(request.user_question)
    translated_reply = translate_medical_text_api(english_reply)
    return CombinedChatbotTranslatorResponse(
        english_response=english_reply,
        translated_response=translated_reply
    )

print("FastAPI endpoints for Module 6 (Healthcare Chatbot and Translator) defined.")

FastAPI endpoints for Module 6 (Healthcare Chatbot and Translator) defined.


## Provide FastAPI Deployment Instructions

### Subtask:
Generate instructions on how to run the FastAPI application using Uvicorn and expose it via ngrok in the Colab environment, or deploy it to a platform like Google Cloud Run or Streamlit Community Cloud.


## Provide FastAPI Deployment Instructions

This section outlines how to run your FastAPI application in a Colab environment using `uvicorn` and `ngrok`, and how to deploy it to cloud platforms.

### Running Locally in Colab with Uvicorn and ngrok

To make your FastAPI application accessible from the internet directly within your Colab notebook, follow these steps:

1.  **Install Dependencies**
    First, ensure you have `uvicorn` and `pyngrok` installed. If you haven't already, run the following command in a new code cell:

    ```bash
    !pip install uvicorn pyngrok
    ```

2.  **Save FastAPI Application to `main.py`**
    You need to save the entire FastAPI application code (including all imports, Pydantic models, helper functions, model initialization, and all API endpoints defined in the previous steps) into a Python file named `main.py` in your Colab environment. You can use the `%%writefile` magic command for this:

    ```python
    %%writefile main.py

    # All your imports here
    import pandas as pd
    import numpy as np
    import torch
    import random
    import re
    import os
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
    from sklearn.impute import SimpleImputer
    from sklearn.feature_selection import SelectKBest, f_classif
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
    from sklearn.cluster import KMeans
    from sklearn.linear_model import Ridge
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.pipeline import Pipeline
    from mlxtend.frequent_patterns import apriori, association_rules
    from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
    from torch.optim import AdamW
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.optimizers import Adam
    import warnings
    from fastapi import FastAPI, HTTPException
    from fastapi.responses import JSONResponse
    from pydantic import BaseModel, Field
    from typing import List, Optional
    
    # Suppress warnings from mlxtend regarding DataFrame types
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=UserWarning, module='mlxtend')
    
    # --- Helper Functions (Replicated from original notebook) ---
    def clean_text(text):
        """Cleans clinical notes text by lowercasing and standardizing whitespace."""
        text = text.lower()
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text

    def create_sequences(data, target, seq_length=10):
        """Creates time-series sequences using a sliding window technique."""
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data.iloc[i:i+seq_length].values)
            y.append(target.iloc[i+seq_length])
        return np.array(X), np.array(y)

    class FeedbackDataset(torch.utils.data.Dataset):
        """Custom Dataset for BERT sentiment analysis."""
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {k: v[idx] for k, v in self.encodings.items()}
            item["labels"] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    def get_embedding(text, tokenizer_bert, model_bert, device):
        """Generates BERT embeddings for a given text."""
        inputs = tokenizer_bert(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=256
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model_bert(**inputs)

        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        return embedding
    
    # --- Pydantic Models (from previous cells) ---
    # 1. Patient Data Model for Classification, Regression, Clustering
    class PatientData(BaseModel):
        age: int = Field(..., ge=1, le=100, description="Patient's age in years")
        bmi: float = Field(..., ge=10.0, le=50.0, description="Body Mass Index")
        systolic_bp: int = Field(..., ge=70, le=200, description="Systolic Blood Pressure (mmHg)")
        cholesterol: int = Field(..., ge=100, le=300, description="Cholesterol level (mg/dL)")
        blood_glucose: int = Field(..., ge=70, le=200, description="Blood Glucose level (mg/dL)")
        diabetes: int = Field(..., ge=0, le=1, description="1 if patient has diabetes, 0 otherwise")
        hypertension: int = Field(..., ge=0, le=1, description="1 if patient has hypertension, 0 otherwise")
        smoker: int = Field(..., ge=0, le=1, description="1 if patient is a smoker, 0 otherwise")
        prev_hospitalizations: int = Field(..., ge=0, le=10, description="Number of previous hospitalizations")
        gender: str = Field(..., description="Patient's gender (Male, Female, Other)")

    # 2. Risk Prediction Response Model
    class RiskPredictionResponse(BaseModel):
        predicted_risk_category: str = Field(..., description="Predicted risk category (e.g., 'Low', 'Normal', 'High')")

    # 3. Length of Stay Prediction Response Model
    class LengthOfStayPredictionResponse(BaseModel):
        predicted_length_of_stay: float = Field(..., description="Predicted length of hospital stay in days")

    # 4. Cluster Assignment Response Model
    class ClusterAssignmentResponse(BaseModel):
        assigned_cluster: int = Field(..., description="Assigned cluster ID for the patient")

    # 5. Association Rules Request Model
    class AssociationRulesRequest(BaseModel):
        min_support: float = Field(0.1, ge=0.01, le=1.0, description="Minimum support for frequent itemsets")
        min_confidence: float = Field(0.6, ge=0.01, le=1.0, description="Minimum confidence for association rules")

    # 6. Association Rule Model
    class AssociationRule(BaseModel):
        antecedents: List[str] = Field(..., description="Items in the antecedent of the rule")
        consequents: List[str] = Field(..., description="Items in the consequent of the rule")
        support: float = Field(..., description="Support of the rule")
        confidence: float = Field(..., description="Confidence of the rule")
        lift: float = Field(..., description="Lift of the rule")

    # 7. Association Rules Response Model
    class AssociationRulesResponse(BaseModel):
        rules: List[AssociationRule] = Field(..., description="List of discovered association rules")

    # 8. Vitals Reading Model for LSTM
    class VitalsReading(BaseModel):
        heart_rate: int = Field(..., ge=40, le=180, description="Heart rate (bpm)")
        systolic_bp: int = Field(..., ge=60, le=200, description="Systolic blood pressure (mmHg)")
        spo2: int = Field(..., ge=70, le=100, description="Blood oxygen saturation (%)")

    # 9. LSTM Request Model
    class LSTMRequest(BaseModel):
        vitals_sequence: List[VitalsReading] = Field(..., min_length=10, max_length=10, description="Sequence of 10 vital sign readings")

    # 10. LSTM Prediction Response Model
    class LSTMPredictionResponse(BaseModel):
        predicted_deterioration_risk: str = Field(..., description="Predicted risk of patient deterioration ('Stable' or 'High deterioration risk')")
        probability: float = Field(..., ge=0.0, le=1.0, description="Probability of high deterioration risk")

    # 11. Sentiment Request Model
    class SentimentRequest(BaseModel):
        feedback_text: str = Field(..., description="Text of the patient feedback")

    # 12. Sentiment Response Model
    class SentimentResponse(BaseModel):
        predicted_sentiment: str = Field(..., description="Predicted sentiment of the feedback ('Negative', 'Neutral', 'Positive')")

    # 13. BioGPT Request Model
    class BioGPTRequest(BaseModel):
        prompt: str = Field(..., description="Text prompt for BioGPT generation")
        max_length: int = Field(80, ge=50, le=200, description="Maximum length of the generated text")

    # 14. BioGPT Response Model
    class BioGPTResponse(BaseModel):
        generated_text: str = Field(..., description="Generated clinical text by BioGPT")

    # 15. Chatbot Request Model
    class ChatbotRequest(BaseModel):
        user_question: str = Field(..., description="User's question to the healthcare chatbot")

    # 16. Chatbot Response Model
    class ChatbotResponse(BaseModel):
        chatbot_response: str = Field(..., description="Response from the healthcare chatbot")

    # 17. Translator Request Model
    class TranslatorRequest(BaseModel):
        text_to_translate: str = Field(..., description="Medical text to be translated")

    # 18. Translator Response Model
    class TranslatorResponse(BaseModel):
        translated_text: str = Field(..., description="Translated medical text")

    # 19. Combined Chatbot Translator Request Model
    class CombinedChatbotTranslatorRequest(BaseModel):
        user_question: str = Field(..., description="User's question for the combined chatbot and translator")
        target_language: Optional[str] = Field(None, description="Desired target language for translation (e.g., 'Tamil', 'Hindi', 'Spanish'). Note: Actual translation capability depends on the loaded model.")

    # 20. Combined Chatbot Translator Response Model
    class CombinedChatbotTranslatorResponse(BaseModel):
        english_response: str = Field(..., description="English response from the chatbot")
        translated_response: str = Field(..., description="Translated response (multilingual interpretation)")

    # --- Initialization Function ---
    def load_models_and_preprocessors():
        """Initializes and loads all models and preprocessors for the FastAPI application."""
        artifacts = {}
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        artifacts['device'] = device

        print("\n--- Loading Datasets ---")
        # Load datasets
        df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")
        df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")
        df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')
        df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")
        artifacts['df_apriori'] = df_apriori # Store for association rules

        df_clinical = None
        try:
            df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
        except FileNotFoundError:
            print("Warning: '/content/healthai_clinical_notes_1000.csv' not found. BioBERT and BioGPT features will be limited.")

        # Ensure chatbot dataset exists
        chatbot_file_path = "/content/healthcare_chatbot_translation_dataset.csv"
        if not os.path.exists(chatbot_file_path):
            symptoms = [
                "fever", "cough", "headache", "chest pain", "breathing difficulty",
                "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
            ]
            questions = [
                "I have fever and cough, what should I do?",
                "Is chest pain serious?",
                "How to control blood sugar?",
                "I feel tired all the time",
                "Can I take paracetamol daily?",
                "When should I see a doctor?",
                "Is headache dangerous?",
                "How to reduce BP naturally?",
            ]
            responses = [
                "Please consult a physician and take rest.",
                "Monitor symptoms and seek emergency care if pain increases.",
                "Maintain diet, exercise and medication regularly.",
                "Blood tests may be required.",
                "Avoid self-medication without advice.",
            ]
            languages = ["English", "Tamil", "Hindi", "Telugu"]

            data_chatbot = []
            for i in range(1000):
                data_chatbot.append({
                    "symptom": random.choice(symptoms),
                    "patient_question": random.choice(questions),
                    "doctor_reply": random.choice(responses),
                    "language": random.choice(languages),
                    "appointment_needed": random.choice(["Yes", "No"])
                })
            df_chatbot = pd.DataFrame(data_chatbot)
            df_chatbot.to_csv(chatbot_file_path, index=False)
        else:
            df_chatbot = pd.read_csv(chatbot_file_path)
        artifacts['df_chatbot'] = df_chatbot # Store for chatbot context if needed

        # --- Module 1: Synthetic Patient Data - Classification, Regression, Clustering ---
        print("\n--- Initializing Module 1 (Patient Data Analytics) ---")
        df_synthetic_processed = df_synthetic.copy()

        # Feature Engineering
        df_synthetic_processed['BP_level'] = pd.cut(
            df_synthetic_processed['systolic_bp'],
            bins=[0, 80, 120, 200],
            labels=['Low', 'Normal', 'High'], ordered=False
        )
        df_synthetic_processed['medication_history'] = ((df_synthetic_processed['diabetes'] == 1) | (df_synthetic_processed['hypertension'] == 1) | (df_synthetic_processed['smoker'] == 1)).astype(int)

        # Data Preprocessing
        imputer = SimpleImputer(strategy='median')
        num_cols_synthetic = ['bmi','systolic_bp','cholesterol','blood_glucose']
        df_synthetic_processed[num_cols_synthetic] = imputer.fit_transform(df_synthetic_processed[num_cols_synthetic])
        artifacts['imputer'] = imputer
        artifacts['num_cols_synthetic'] = num_cols_synthetic

        # Outlier Removal (applied to the training data, new patient data will be transformed only)
        for col in num_cols_synthetic:
            Q1 = df_synthetic_processed[col].quantile(0.25)
            Q3 = df_synthetic_processed[col].quantile(0.75)
            IQR = Q3 - Q1
            df_synthetic_processed = df_synthetic_processed[(df_synthetic_processed[col] >= Q1 - 1.5*IQR) & (df_synthetic_processed[col] <= Q3 + 1.5*IQR)]

        le_synthetic_BP = LabelEncoder()
        df_synthetic_processed['BP_level'] = le_synthetic_BP.fit_transform(df_synthetic_processed['BP_level'])
        artifacts['le_synthetic_BP'] = le_synthetic_BP

        le_synthetic_risk = LabelEncoder()
        df_synthetic_processed['risk_category'] = le_synthetic_risk.fit_transform(df_synthetic_processed['risk_category'])
        artifacts['le_synthetic_risk'] = le_synthetic_risk
        artifacts['risk_category_labels'] = le_synthetic_risk.inverse_transform(sorted(df_synthetic_processed['risk_category'].unique()))

        le_gender = None
        if 'gender' in df_synthetic_processed.columns and df_synthetic_processed['gender'].dtype == 'object':
            le_gender = LabelEncoder()
            df_synthetic_processed['gender'] = le_gender.fit_transform(df_synthetic_processed['gender'])
        artifacts['le_gender'] = le_gender

        # Classification Model
        X_clf_full_features = df_synthetic_processed.drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
        y_clf = df_synthetic_processed['risk_category']

        scaler_clf = StandardScaler()
        X_clf_scaled_for_kbest = scaler_clf.fit_transform(X_clf_full_features)
        X_clf_scaled_for_kbest_df = pd.DataFrame(X_clf_scaled_for_kbest, columns=X_clf_full_features.columns, index=X_clf_full_features.index)
        artifacts['scaler_clf'] = scaler_clf
        artifacts['clf_all_feature_names_before_select'] = list(X_clf_full_features.columns)

        selector_clf = SelectKBest(score_func=f_classif, k=5)
        selector_clf.fit(X_clf_scaled_for_kbest_df, y_clf)
        X_selected_clf = selector_clf.transform(X_clf_scaled_for_kbest_df)
        artifacts['selector_clf'] = selector_clf
        artifacts['clf_feature_names_after_select'] = list(X_clf_full_features.columns[selector_clf.get_support(indices=True)])

        clf_model = RandomForestClassifier(random_state=42)
        clf_model.fit(X_selected_clf, y_clf)
        artifacts['clf_model'] = clf_model
        print("Module 1: Classification model trained.")

        # Regression Model
        reg_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
        X_reg = df_synthetic_processed[reg_feature_names].copy()
        y_reg = df_synthetic_processed["length_of_stay_days"]

        reg_pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("model", Ridge(alpha=1.0))
        ])
        reg_pipeline.fit(X_reg, y_reg)
        artifacts['reg_pipeline'] = reg_pipeline
        artifacts['reg_feature_names'] = reg_feature_names
        print("Module 1: Regression model trained.")

        # Clustering Model
        cluster_feature_names = [col for col in df_synthetic_processed.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
        X_cluster = df_synthetic_processed[cluster_feature_names]

        scaler_cluster = StandardScaler()
        X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)
        artifacts['scaler_cluster'] = scaler_cluster
        artifacts['cluster_feature_names'] = cluster_feature_names

        kmeans_model = KMeans(n_clusters=3, random_state=42, n_init=10)
        kmeans_model.fit(X_cluster_scaled)
        artifacts['kmeans_model'] = kmeans_model
        print("Module 1: Clustering model trained.")

        # --- Module 3: Sequence Modeling (LSTM) ---
        print("\n--- Initializing Module 3 (Sequence Modeling) ---")
        FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
        TARGET_ts = 'risk_flag'
        SEQ_LENGTH = 10
        artifacts['SEQ_LENGTH'] = SEQ_LENGTH
        artifacts['FEATURES_ts'] = FEATURES_ts

        scaler_ts = MinMaxScaler()
        df_timeseries[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries[FEATURES_ts])
        artifacts['scaler_ts'] = scaler_ts

        X_ts, y_ts = create_sequences(df_timeseries[FEATURES_ts], df_timeseries[TARGET_ts], SEQ_LENGTH)

        # Using a smaller split for actual training, as full data was processed above
        X_train_ts, _, y_train_ts, _ = train_test_split(
            X_ts, y_ts, test_size=0.1, random_state=42, stratify=y_ts
        ) # Use 90% of data for training the model that will be used by FastAPI

        model_lstm = Sequential()
        model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
        model_lstm.add(Dropout(0.2))
        model_lstm.add(LSTM(32))
        model_lstm.add(Dropout(0.2))
        model_lstm.add(Dense(1, activation='sigmoid'))

        model_lstm.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        # Train Model (reduced epochs for faster startup)
        model_lstm.fit(
            X_train_ts, y_train_ts,
            epochs=5,
            batch_size=32,
            validation_split=0.1,
            verbose=0
        )
        artifacts['model_lstm'] = model_lstm
        print("Module 3: LSTM model trained.")

        # --- Module 4: Sentiment Analysis (BERT) ---
        print("\n--- Initializing Module 4 (Sentiment Analysis) ---")
        texts_feedback = df_feedback["feedback_text"].tolist()
        le_feedback = LabelEncoder()
        labels_feedback = le_feedback.fit_transform(df_feedback["sentiment"])
        artifacts['le_feedback'] = le_feedback
        artifacts['sentiment_labels'] = list(le_feedback.classes_)

        tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
        artifacts['tokenizer_bert'] = tokenizer_bert
        encodings_feedback = tokenizer_bert(
            texts_feedback,
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors="pt"
        )

        train_idx_feedback, _ = train_test_split(
            range(len(labels_feedback)), test_size=0.1, random_state=42, stratify=labels_feedback
        )
        train_dataset_feedback = FeedbackDataset(
            {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
            labels_feedback[train_idx_feedback]
        )

        model_sentiment = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased",
            num_labels=len(set(labels_feedback))
        )

        optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
        model_sentiment.to(device)
        model_sentiment.train()

        # Simplified training loop for quick initialization in FastAPI
        train_loader = torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8, shuffle=True)
        for epoch in range(1):
            for i, batch in enumerate(train_loader):
                if i > 20: break # Process only a few batches for speed
                optimizer_sentiment.zero_grad()
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs_sentiment = model_sentiment(**batch)
                loss_sentiment = outputs_sentiment.loss
                loss_sentiment.backward()
                optimizer_sentiment.step()
        model_sentiment.eval()
        artifacts['model_sentiment'] = model_sentiment
        print("Module 4: BERT sentiment model loaded and partially fine-tuned.")

        # --- Module 5: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---
        print("\n--- Initializing Module 5 (Generative AI) ---")
        tokenizer_biobert = None
        model_biobert_embeddings = None
        kmeans_biobert = None
        generator_biogpt = None

        if df_clinical is not None:
            df_clinical["clinical_note"] = df_clinical["clinical_note"].astype(str).apply(clean_text)

            # BioBERT for Embeddings
            MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
            tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
            model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)
            model_biobert_embeddings.to(device)
            artifacts['tokenizer_biobert'] = tokenizer_biobert
            artifacts['model_biobert_embeddings'] = model_biobert_embeddings

            # Generate embeddings for a sample (to fit KMeans)
            sample_size = min(200, len(df_clinical)) # Limit for initialization speed
            if sample_size > 0:
                sample_df_clinical = df_clinical.sample(sample_size, random_state=42) if len(df_clinical) > sample_size else df_clinical.copy()
                embeddings_biobert = np.vstack(sample_df_clinical["clinical_note"].apply(
                    lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings, device).flatten()
                ))
                kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
                kmeans_biobert.fit(embeddings_biobert)
                artifacts['kmeans_biobert'] = kmeans_biobert
                print("Module 5: BioBERT tokenizer and embedding model loaded, KMeans fitted.")
            else:
                print("Module 5: No clinical notes data available to fit KMeans for BioBERT embeddings.")

            # BioGPT for Text Generation
            try:
                generator_biogpt = pipeline(
                    "text-generation",
                    model="microsoft/BioGPT",
                    device=0 if torch.cuda.is_available() else -1 # Use GPU if available
                )
                artifacts['generator_biogpt'] = generator_biogpt
                print("Module 5: BioGPT text generation pipeline loaded.")
            except Exception as e:
                print(f"Module 5: Could not load BioGPT model: {e}. Text generation and chatbot functionality will be limited.")
        else:
            print("Module 5: Clinical notes data not available, skipping BioBERT and BioGPT initialization.")

        # --- Module 6: Healthcare Chatbot and Translator ---
        print("\n--- Initializing Module 6 (Chatbot & Translator) ---")
        model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
        tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
        translator_model = MarianMTModel.from_pretrained(model_name_translator)
        translator_model.to(device)
        artifacts['tokenizer_translator'] = tokenizer_translator
        artifacts['translator_model'] = translator_model
        print("Module 6: MarianMT translator model loaded.")

        print("\nAll models and preprocessors initialized successfully!")
        return artifacts

    # Global variable to store initialized artifacts
    models_and_preprocessors = load_models_and_preprocessors()

    # --- FastAPI app instance ---
    app = FastAPI(
        title="HealthAI API",
        description="API for various HealthAI modules including Patient Data Analytics, Association Rules, Sequence Modeling, Sentiment Analysis, Generative AI, and Chatbot/Translator."
    )

    @app.get("/health", response_class=JSONResponse, tags=["System"]) # type: ignore
    def health_check():
        """Health check endpoint to ensure the API is running."""
        return {"status": "ok", "message": "HealthAI API is running"}

    # --- Helper function for preprocessing patient data (common for Module 1 endpoints) ---
    def preprocess_patient_data(patient_data: PatientData):
        data_dict = patient_data.dict()
        new_patient_df = pd.DataFrame([data_dict])

        # 1. Feature Engineering
        new_patient_df['BP_level'] = pd.cut(
            new_patient_df['systolic_bp'],
            bins=[0, 80, 120, 200],
            labels=['Low', 'Normal', 'High'], ordered=False
        )
        new_patient_df['medication_history'] = ((new_patient_df['diabetes'] == 1) | (new_patient_df['hypertension'] == 1) | (new_patient_df['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_df[models_and_preprocessors['num_cols_synthetic']] = models_and_preprocessors['imputer'].transform(new_patient_df[models_and_preprocessors['num_cols_synthetic']])

        # 3. Label Encoding (only transform, not fit)
        new_patient_df['BP_level'] = models_and_preprocessors['le_synthetic_BP'].transform(new_patient_df['BP_level'])
        if models_and_preprocessors['le_gender'] is not None and 'gender' in new_patient_df.columns:
            try:
                new_patient_df['gender'] = models_and_preprocessors['le_gender'].transform(new_patient_df['gender'])
            except ValueError:
                # Handle unseen gender category gracefully for demo (e.g., default or raise specific error)
                raise HTTPException(status_code=400, detail="Unseen gender category. Please use 'Male', 'Female', or 'Other'.")

        return new_patient_df

    # --- Module 1 Endpoints: Patient Data Analytics ---

    @app.post("/predict-risk-category", response_model=RiskPredictionResponse, tags=["Module 1 - Patient Data Analytics"])
    async def predict_risk_category(patient_data: PatientData):
        """Predicts the risk category for a new patient."""
        processed_data = preprocess_patient_data(patient_data)

        # Align columns with training data for classification
        # Ensure all features expected by the scaler and selector are present, in correct order.
        X_clf_aligned = pd.DataFrame(columns=models_and_preprocessors['clf_all_feature_names_before_select'])
        for col in models_and_preprocessors['clf_all_feature_names_before_select']:
            if col in processed_data.columns:
                X_clf_aligned[col] = processed_data[col]
            else:
                X_clf_aligned[col] = 0 # Default value for any missing feature, or handle more robustly

        # Scale features using the fitted scaler_clf
        new_patient_scaled = models_and_preprocessors['scaler_clf'].transform(X_clf_aligned)

        # Apply feature selection
        new_patient_selected = models_and_preprocessors['selector_clf'].transform(new_patient_scaled)

        prediction_idx = models_and_preprocessors['clf_model'].predict(new_patient_selected)
        predicted_risk = models_and_preprocessors['le_synthetic_risk'].inverse_transform(prediction_idx)[0]
        return RiskPredictionResponse(predicted_risk_category=predicted_risk)

    @app.post("/predict-length-of-stay", response_model=LengthOfStayPredictionResponse, tags=["Module 1 - Patient Data Analytics"])
    async def predict_length_of_stay(patient_data: PatientData):
        """Predicts the length of hospital stay for a new patient."""
        processed_data = preprocess_patient_data(patient_data)

        # Align columns with regression model's expected features
        X_reg_aligned = processed_data[models_and_preprocessors['reg_feature_names']] # Ensure correct column order

        prediction_reg = models_and_preprocessors['reg_pipeline'].predict(X_reg_aligned)
        return LengthOfStayPredictionResponse(predicted_length_of_stay=float(prediction_reg[0]))

    @app.post("/assign-patient-cluster", response_model=ClusterAssignmentResponse, tags=["Module 1 - Patient Data Analytics"])
    async def assign_patient_cluster(patient_data: PatientData):
        """Assigns a new patient to a cluster based on their features."""
        processed_data = preprocess_patient_data(patient_data)

        # Align columns with clustering model's expected features
        X_cluster_aligned = processed_data[models_and_preprocessors['cluster_feature_names']] # Ensure correct column order

        # Scale features using the fitted scaler_cluster
        new_patient_cluster_scaled = models_and_preprocessors['scaler_cluster'].transform(X_cluster_aligned)

        predicted_cluster = models_and_preprocessors['kmeans_model'].predict(new_patient_cluster_scaled)
        return ClusterAssignmentResponse(assigned_cluster=int(predicted_cluster[0]))

    # --- Module 2 Endpoints: Association Rules ---

    @app.post("/association-rules", response_model=AssociationRulesResponse, tags=["Module 2 - Association Rules"])
    async def get_association_rules(request: AssociationRulesRequest):
        """Generates and returns association rules based on medical transaction data."""
        df_assoc = models_and_preprocessors['df_apriori'].drop('patient_id', axis=1)
        df_assoc_bool = df_assoc.astype(bool)

        try:
            frequent_itemsets = apriori(
                df_assoc_bool,
                min_support=request.min_support,
                use_colnames=True
            )

            rules = association_rules(
                frequent_itemsets,
                metric='confidence',
                min_threshold=request.min_confidence
            )

            # Convert frozensets to lists of strings for JSON serialization
            rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
            rules['consequents'] = rules['consequents'].apply(lambda x: list(x))

            # Select and rename columns to match AssociationRule Pydantic model
            rules_list = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_dict(orient='records')

            return AssociationRulesResponse(rules=rules_list)
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Error generating association rules: {e}")

    # --- Module 3 Endpoints: Sequence Modeling (LSTM) ---

    @app.post("/predict-deterioration-risk", response_model=LSTMPredictionResponse, tags=["Module 3 - Sequence Modeling (LSTM)"])
    async def predict_deterioration_risk(request: LSTMRequest):
        """Predicts the deterioration risk for a patient based on a sequence of vital signs."""
        # Convert Pydantic list of VitalsReading to a DataFrame
        vitals_df = pd.DataFrame([v.dict() for v in request.vitals_sequence])

        # Scale the vital signs using the pre-fitted scaler
        scaled_vitals = models_and_preprocessors['scaler_ts'].transform(vitals_df[models_and_preprocessors['FEATURES_ts']])

        # Reshape for LSTM model: (1, SEQ_LENGTH, num_features)
        reshaped_vitals = scaled_vitals.reshape(1, models_and_preprocessors['SEQ_LENGTH'], len(models_and_preprocessors['FEATURES_ts']))

        # Make prediction
        prediction_prob = models_and_preprocessors['model_lstm'].predict(reshaped_vitals, verbose=0)[0][0]

        # Determine risk category
        predicted_risk = "High deterioration risk" if prediction_prob > 0.5 else "Stable"

        return LSTMPredictionResponse(predicted_deterioration_risk=predicted_risk, probability=float(prediction_prob))

    # --- Module 4 Endpoints: Sentiment Analysis (BERT) ---

    @app.post("/analyze-sentiment", response_model=SentimentResponse, tags=["Module 4 - Sentiment Analysis (BERT)"])
    async def analyze_sentiment(request: SentimentRequest):
        """Analyzes the sentiment of patient feedback using a BERT model."""
        inputs_sentiment = models_and_preprocessors['tokenizer_bert'](
            request.feedback_text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )
        inputs_sentiment = {k: v.to(models_and_preprocessors['device']) for k, v in inputs_sentiment.items()}

        with torch.no_grad():
            outputs_sentiment_pred = models_and_preprocessors['model_sentiment'](**inputs_sentiment)

        pred_sentiment_idx = torch.argmax(outputs_sentiment_pred.logits, dim=1).item()
        predicted_sentiment = models_and_preprocessors['le_feedback'].inverse_transform([pred_sentiment_idx])[0]

        return SentimentResponse(predicted_sentiment=predicted_sentiment)

    # --- Module 5 Endpoints: Generative AI (BioBERT Embeddings & BioGPT Text Generation) ---

    @app.post("/analyze-clinical-note", tags=["Module 5 - Generative AI"])
    async def analyze_clinical_note(text_data: SentimentRequest): # Reusing SentimentRequest for text input
        """Generates BioBERT embeddings for a clinical note and assigns it to a cluster."""
        if not models_and_preprocessors.get('tokenizer_biobert') or not models_and_preprocessors.get('model_biobert_embeddings') or not models_and_preprocessors.get('kmeans_biobert'):
            raise HTTPException(status_code=503, detail="BioBERT models or KMeans not initialized. Clinical notes data might be missing or models failed to load.")

        cleaned_text = clean_text(text_data.feedback_text)
        embedding = get_embedding(
            cleaned_text,
            models_and_preprocessors['tokenizer_biobert'],
            models_and_preprocessors['model_biobert_embeddings'],
            models_and_preprocessors['device']
        )

        predicted_cluster = models_and_preprocessors['kmeans_biobert'].predict(embedding)[0]

        return ClusterAssignmentResponse(assigned_cluster=int(predicted_cluster))

    @app.post("/generate-clinical-text", response_model=BioGPTResponse, tags=["Module 5 - Generative AI"])
    async def generate_clinical_text(request: BioGPTRequest):
        """Generates clinical text using BioGPT based on a given prompt."""
        if not models_and_preprocessors.get('generator_biogpt'):
            raise HTTPException(status_code=503, detail="BioGPT model not initialized. Check logs for loading errors (e.g., missing sacremoses).")

        try:
            generated_text = models_and_preprocessors['generator_biogpt'](
                request.prompt,
                max_length=request.max_length,
                num_return_sequences=1,
                pad_token_id=models_and_preprocessors['generator_biogpt'].tokenizer.eos_token_id
            )[0]["generated_text"]
            return BioGPTResponse(generated_text=generated_text)
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Error during text generation: {e}")

    # --- Module 6 Endpoints: Healthcare Chatbot and Translator ---

    def healthcare_chatbot_api(user_input: str) -> str:
        """Chatbot function using the BioGPT model for medical guidance, or a generic response."""
        if models_and_preprocessors.get("generator_biogpt"):
            prompt = f"""
            You are a healthcare assistant. Provide safe medical guidance and symptom triage. Avoid medical diagnosis. Focus on general health advice. If you cannot provide a specific answer, advise consulting a doctor. Do not generate information that is not directly related to the prompt. Limit your response to 100 words.
            Patient says: {user_input}
            Response:
            """
            response = models_and_preprocessors['generator_biogpt'](prompt, max_length=150, num_return_sequences=1, pad_token_id=models_and_preprocessors['generator_biogpt'].tokenizer.eos_token_id)[0]["generated_text"]

            # Clean up the response to remove the prompt part if BioGPT echoes it
            response_lines = response.split('\n')
            clean_response = []
            capture = False
            for line in response_lines:
                if line.strip().startswith("Response:"):
                    capture = True
                    continue
                if capture and line.strip():
                    clean_response.append(line.strip())

            if clean_response:
                return " ".join(clean_response)
            else:
                # Fallback cleanup if parsing fails, by removing the exact prompt string
                return response.replace(prompt, "").strip()
        else:
            return f"Hello! As a healthcare assistant, I recommend consulting a doctor for '{user_input}'. Please note that a specialized AI model for medical guidance is currently unavailable due to BioGPT loading issues. Always consult a qualified medical professional for health concerns."


    def translate_medical_text_api(text: str) -> str:
        """Translates medical text using the MarianMT model."""
        if not models_and_preprocessors.get('tokenizer_translator') or not models_and_preprocessors.get('translator_model'):
            raise HTTPException(status_code=503, detail="Translator model not initialized.")

        inputs = models_and_preprocessors['tokenizer_translator'](text, return_tensors="pt", padding=True)
        inputs = {k: v.to(models_and_preprocessors['device']) for k, v in inputs.items()}
        translated = models_and_preprocessors['translator_model'].generate(**inputs)
        return models_and_preprocessors['tokenizer_translator'].decode(translated[0], skip_special_tokens=True)


    @app.post("/chatbot", response_model=ChatbotResponse, tags=["Module 6 - Chatbot & Translator"])
    async def chatbot_endpoint(request: ChatbotRequest):
        """Provides a medical guidance response from the AI chatbot."""
        response = healthcare_chatbot_api(request.user_question)
        return ChatbotResponse(chatbot_response=response)

    @app.post("/translate", response_model=TranslatorResponse, tags=["Module 6 - Chatbot & Translator"])
    async def translate_endpoint(request: TranslatorRequest):
        """Translates medical text from English to a multilingual interpretation."""
        translated_text = translate_medical_text_api(request.text_to_translate)
        return TranslatorResponse(translated_text=translated_text)

    @app.post("/chatbot-translate", response_model=CombinedChatbotTranslatorResponse, tags=["Module 6 - Chatbot & Translator"])
    async def combined_chatbot_translator_endpoint(request: CombinedChatbotTranslatorRequest):
        """Combines chatbot response generation with translation."""
        english_reply = healthcare_chatbot_api(request.user_question)
        translated_reply = translate_medical_text_api(english_reply)
        return CombinedChatbotTranslatorResponse(
            english_response=english_reply,
            translated_response=translated_reply
        )
    ```

3.  **Run FastAPI with Uvicorn**
    Execute the `main.py` file using `uvicorn`. The `&` at the end runs it in the background, allowing subsequent cells to execute. Choose a port, e.g., 8000.

    ```bash
    !uvicorn main:app --host 0.0.0.0 --port 8000 &
    ```

4.  **Expose with ngrok**
    To make your local FastAPI server accessible over the internet, use `ngrok`. You'll need an `ngrok` authentication token. Replace `YOUR_NGROK_AUTH_TOKEN` with your actual token from [ngrok dashboard](https://dashboard.ngrok.com/get-started/your-authtoken).

    ```python
    from pyngrok import ngrok

    # Terminate any previous ngrok tunnels
    ngrok.kill()

    # Authenticate ngrok (replace with your actual token)
    # ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")

    # Open a tunnel to the FastAPI port
    public_url = ngrok.connect(8000)
    print(f"ngrok tunnel opened at: {public_url}")
    ```

5.  **Access Your API**
    Once the `ngrok` tunnel is established, the `public_url` printed will be your gateway to the FastAPI application. You can access:
    *   **Interactive API Documentation (Swagger UI):** `YOUR_PUBLIC_URL/docs`
    *   **Alternative Docs (ReDoc):** `YOUR_PUBLIC_URL/redoc`
    *   **Specific Endpoints:** e.g., `YOUR_PUBLIC_URL/health`

### Cloud Deployment Options

For production-grade deployments, consider containerizing your application and deploying it to a cloud platform.

#### A. Google Cloud Run

Google Cloud Run is a fully managed compute platform for deploying containerized applications. It automatically scales your service up and down.

1.  **Containerize Your Application (Dockerfile)**
    Create a `Dockerfile` in the same directory as `main.py` and `requirements.txt` (which should list all Python dependencies like `fastapi`, `uvicorn`, `pandas`, `torch`, `transformers`, `tensorflow`, `mlxtend`, `scikit-learn`, `sacremoses`, etc.).

    ```dockerfile
    # Use a lightweight Python image
    FROM python:3.9-slim-buster

    # Set working directory
    WORKDIR /app

    # Copy requirements file and install dependencies
    COPY requirements.txt .
    RUN pip install --no-cache-dir -r requirements.txt

    # Copy your application code
    COPY . .

    # Expose the port your FastAPI application will run on
    EXPOSE 8000

    # Command to run the application using Uvicorn
    CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
    ```

2.  **Create `requirements.txt`**
    Ensure this file lists *all* Python packages used by your application. You can generate it by running `!pip freeze > requirements.txt` after installing all libraries.

3.  **Build and Push Docker Image**
    *   Install Google Cloud SDK and authenticate.
    *   Set your Google Cloud Project.
    *   Build the Docker image:
        ```bash
        gcloud builds submit --tag gcr.io/YOUR_PROJECT_ID/healthai-api
        ```
    *   Push the image to Google Container Registry (or Artifact Registry).

4.  **Deploy to Cloud Run**
    ```bash
    gcloud run deploy healthai-api --image gcr.io/YOUR_PROJECT_ID/healthai-api \
        --platform managed \
        --region us-central1 \
        --allow-unauthenticated \
        --memory 2Gi \
        --cpu 2 \
        --timeout 300 \
        --max-instances 1 # Adjust scaling as needed
    ```
    *   `--memory` and `--cpu` should be adjusted based on the resource requirements of your models (especially large LLMs like BioGPT).
    *   `--timeout` might need to be increased for long-running model inferences.

#### B. Streamlit Community Cloud (for Streamlit-based UIs)

While FastAPI is primarily for API endpoints, if you were to wrap this API with a Streamlit frontend (as suggested in the initial notebook), you could deploy the Streamlit app to Streamlit Community Cloud.

1.  **Create a Streamlit App (`streamlit_app.py`)**
    This file would contain the Streamlit UI code that calls your FastAPI endpoints. The FastAPI application itself would ideally be deployed separately (e.g., on Cloud Run) and the Streamlit app would interact with it.

2.  **Host on GitHub**
    Push your `streamlit_app.py`, `requirements.txt`, and any other necessary files (like model weights if small enough, or specify environment variables to load models from cloud storage) to a GitHub repository.

3.  **Deploy from Streamlit Community Cloud**
    *   Go to [Streamlit Community Cloud](https://share.streamlit.io/).
    *   Click "New app" and connect your GitHub repository.
    *   Select your repository, branch, and `streamlit_app.py` file.
    *   Adjust advanced settings if needed (e.g., Python version, secrets for API keys).
    *   Click "Deploy!"

**Note on Model Weights:** For large models, directly including them in the Docker image or GitHub repo for Streamlit Cloud might be problematic due to size limits or cold start times. Consider storing model weights in cloud storage (e.g., Google Cloud Storage, Hugging Face Hub) and loading them dynamically at runtime with appropriate authentication.

## Final Task

### Subtask:
Summarize the created FastAPI backend, its functionalities, and the deployment options.


## Summary:

### Data Analysis Key Findings

*   **Robust API Schema Definition:** Twenty Pydantic models were meticulously defined, covering all request and response schemas for the various API endpoints. This ensures strong data validation and clear documentation, crucial for a complex application with diverse data inputs.
*   **Comprehensive Backend Functionality:** A FastAPI application was successfully developed, integrating six distinct modules with a total of twelve dedicated endpoints:
    *   **Patient Data Analytics:** Endpoints for predicting risk category, length of hospital stay, and assigning patients to clusters.
    *   **Association Rules:** An endpoint to generate association rules based on configurable support and confidence thresholds.
    *   **Sequence Modeling (LSTM):** An endpoint for predicting patient deterioration risk from vital sign sequences.
    *   **Sentiment Analysis (BERT):** An endpoint to analyze sentiment from patient feedback.
    *   **Generative AI:** Endpoints for generating BioBERT embeddings for clinical notes (cluster assignment) and generating clinical text using BioGPT.
    *   **Healthcare Chatbot & Translator:** Endpoints for chatbot responses, medical text translation, and a combined chatbot-translation service.
*   **Integrated Model & Preprocessor Management:** All machine learning and deep learning models (RandomForestClassifier, Ridge, KMeans, LSTM, BERT, BioBERT, BioGPT, MarianMT) and their respective preprocessors (scalers, encoders, tokenizers, imputers) were successfully loaded, initialized, and, where appropriate, "trained" or fine-tuned for quick startup within a centralized `load_models_and_preprocessors` function.
*   **Detailed Deployment Guidance:** Comprehensive instructions were provided for deploying the FastAPI application. This includes local execution using Uvicorn and ngrok, along with cloud deployment strategies for Google Cloud Run (including Dockerfile and `gcloud` commands) and Streamlit Community Cloud.

### Insights or Next Steps

*   **Production Readiness & Scalability:** While the current setup is excellent for demonstration and development, for production environments, focus on optimizing model loading times, implementing asynchronous processing for long-running inferences (especially with generative AI models), and refining the resource allocation for cloud deployments (e.g., specific GPU instances for LLMs).
*   **Enhanced Error Handling & Logging:** Implement more granular error handling, especially for external model API calls or potential data inconsistencies. Integrate robust logging to monitor application performance, model inference issues, and user interactions in a production setting.


# Task
## Deploying Your Streamlit App

Your Streamlit dashboard has been generated and saved as `streamlit_app.py`. To run and share your application, follow the steps below:

### 1. Run in Google Colab with `ngrok` (Temporary Public URL)

This is the quickest way to get your dashboard live and shareable directly from your Colab notebook.

**A. Install Streamlit and pyngrok:**

```python
!pip install -q streamlit pyngrok
```

**B. Run the Streamlit App via `ngrok`:**

```python
from pyngrok import ngrok
import subprocess
import os

# Terminate any previous ngrok tunnels
ngrok.kill()

# Authenticate ngrok (replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token)
# You can get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken
# If you already have it configured globally or in a Colab Secret, you might not need this line.
# If you're running this for the first time or if you're experiencing issues, uncomment and replace:
# ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")

# Run Streamlit in the background
!streamlit run streamlit_app.py &>/dev/null &

# Give Streamlit a moment to start up
import time
time.sleep(5)

# Open a ngrok tunnel to the Streamlit port (default 8501)
public_url = ngrok.connect(8501)
print(f"Your Streamlit app is live at: {public_url}")

# To stop the ngrok tunnel and Streamlit app:
# ngrok.kill()
# os.system('kill $(lsof -t -i:8501)') # Kills the Streamlit process
```

Once you run the above code, a public URL will be printed in the output. You can open this URL in your browser to view and interact with your Streamlit dashboard. The dashboard will remain active as long as your Colab session is running and the `ngrok` tunnel is maintained.

### 2. Deploy to Streamlit Community Cloud (Persistent Hosting)

For a more permanent and robust deployment, you can host your Streamlit app on Streamlit Community Cloud.

**A. Save your application to a GitHub Repository:**

1.  **Create a new GitHub repository** (e.g., `healthai-streamlit-dashboard`).
2.  **Upload `streamlit_app.py`** to the root of this repository.
3.  **Create a `requirements.txt` file** in the same repository root. This file should list all the Python libraries your app depends on. You can generate a basic one by running `!pip freeze > requirements.txt` in a Colab cell, then manually clean it to include only necessary packages like:
    ```
    streamlit
    pandas
    numpy
    scikit-learn
    mlxtend
    transformers
    torch
    tensorflow
    sacremoses
    ... (all other libraries used in your streamlit_app.py)
    ```

**B. Deploy via Streamlit Community Cloud:**

1.  Go to [Streamlit Community Cloud](https://share.streamlit.io/).
2.  Log in with your GitHub account.
3.  Click "New app" from your workspace.
4.  Select your GitHub repository, the branch, and ensure "Main file path" is set to `streamlit_app.py`.
5.  Click "Deploy!".

Streamlit Community Cloud will build and deploy your app. It will provide you with a permanent public URL for your dashboard.

**Important Considerations for Streamlit Community Cloud:**

*   **Large Models:** If your models (especially BioBERT/BioGPT or large LSTM models) are very large, they might exceed Streamlit Cloud's memory limits or cause long cold start times. For such cases, consider:
    *   Storing models in cloud storage (e.g., Google Cloud Storage, Hugging Face Hub) and loading them dynamically in your app.
    *   Using a separate FastAPI backend (as explored in your previous task) to host the heavy models, and then having the Streamlit app call the FastAPI endpoints.
*   **API Keys/Secrets:** If your app requires any API keys (e.g., for `ngrok` if you ever use it in production, or external services), store them securely using Streamlit's secrets management.

You now have the tools and instructions to run and deploy your comprehensive HealthAI Streamlit dashboard!

## Generate Streamlit App Code

### Subtask:
Generate the full Streamlit application code for all modules, incorporating existing data loading, preprocessing, model training, and prediction logic, and save it as `streamlit_app.py`.


In [6]:
# Check if Streamlit process is running
!ps aux | grep streamlit

root        4423  0.3  0.5 234008 71312 ?        S    05:07   0:02 /usr/bin/python3 /usr/local/bin/streamlit run streamlit_app.py
root        8541  0.0  0.0   7376  3532 ?        S    05:23   0:00 /bin/bash -c ps aux | grep streamlit
root        8543  0.0  0.0   6484  2356 ?        S    05:23   0:00 grep streamlit


In [ ]:
# To stop any previously running Streamlit processes
!kill $(lsof -t -i:8501) 2>/dev/null

# Run Streamlit app in the foreground
!streamlit run streamlit_app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.204.179:8501



In [5]:
# Reinstall Streamlit and pyngrok to ensure they are available
!pip install -q streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.5 MB/s eta 0:00:00


In [ ]:
# To stop any previously running Streamlit processes
!kill $(lsof -t -i:8501) 2>/dev/null

# Run Streamlit app in the foreground using python -m
!python -m streamlit run streamlit_app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.186.0.176:8501



In [4]:
# To stop any previously running Streamlit processes
!kill $(lsof -t -i:8501) 2>/dev/null

# Run Streamlit app in the foreground using python -m
!python -m streamlit run streamlit_app.py

/usr/bin/python3: No module named streamlit


In [1]:
# Display the content of streamlit_app.py
!cat streamlit_app.py

cat: streamlit_app.py: No such file or directory


**Reasoning**:
The subtask requires generating the `streamlit_app.py` file with all the code from cell `6d2eba2e`. I will use the `%%writefile` magic command to write the complete content of that cell into a new file named `streamlit_app.py`.



In [2]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import random
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from mlxtend.frequent_patterns import apriori, association_rules
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModel, pipeline, MarianMTModel, MarianTokenizer
from torch.optim import AdamW
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress warnings from mlxtend regarding DataFrame types
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mlxtend')


# --- Helper Functions ---

@st.cache_data
def clean_text(text):
    """Cleans clinical notes text by lowercasing and standardizing whitespace."""
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def create_sequences(data, target, seq_length=10):
    """Creates time-series sequences using a sliding window technique."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(target.iloc[i+seq_length])
    return np.array(X), np.array(y)

class FeedbackDataset(torch.utils.data.Dataset):
    """Custom Dataset for BERT sentiment analysis."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

@st.cache_resource
def get_embedding(text, tokenizer_bert, model_bert):
    """Generates BERT embeddings for a given text."""
    inputs = tokenizer_bert(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model_bert.to(device)

    with torch.no_grad():
        outputs = model_bert(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

# --- Streamlit App Layout ---
st.set_page_config(layout="wide", page_title="HealthAI Dashboard")
st.title("HealthAI Multi-Module Dashboard")

st.sidebar.title("Navigation")
module_selection = st.sidebar.radio(
    "Go to",
    [
        "Module 1: Patient Data Analytics",
        "Module 2: Association Rules",
        "Module 3: Sequence Modeling (LSTM)",
        "Module 4: Sentiment Analysis (BERT)",
        "Module 5: Generative AI (BioBERT & BioGPT)",
        "Module 6: Chatbot & Translator"
    ]
)

# --- Data Loading (using st.cache_data to load once) ---
@st.cache_data
def load_all_data():
    df_synthetic = pd.read_csv("/content/healthai_synthetic_patient_data.csv")
    df_apriori = pd.read_csv("/content/healthai_apriori_1000.csv")
    df_timeseries = pd.read_csv('/content/healthai_timeseries_1000.csv')
    df_feedback = pd.read_csv("/content/healthai_patient_feedback_1000.csv")

    df_clinical = None
    try:
        df_clinical = pd.read_csv("/content/healthai_clinical_notes_1000.csv")
    except FileNotFoundError:
        st.warning("'/content/healthai_clinical_notes_1000.csv' not found. Module 5 will be partially functional.")

    # Ensure chatbot dataset exists
    chatbot_file_path = "/content/healthcare_chatbot_translation_dataset.csv"
    if not os.path.exists(chatbot_file_path):
        symptoms = [
            "fever", "cough", "headache", "chest pain", "breathing difficulty",
            "fatigue", "nausea", "vomiting", "diabetes symptoms", "high blood pressure"
        ]
        questions = [
            "I have fever and cough, what should I do?",
            "Is chest pain serious?",
            "How to control blood sugar?",
            "I feel tired all the time",
            "Can I take paracetamol daily?",
            "When should I see a doctor?",
            "Is headache dangerous?",
            "How to reduce BP naturally?",
        ]
        responses = [
            "Please consult a physician and take rest.",
            "Monitor symptoms and seek emergency care if pain increases.",
            "Maintain diet, exercise and medication regularly.",
            "Blood tests may be required.",
            "Avoid self-medication without advice.",
        ]
        languages = ["English", "Tamil", "Hindi", "Telugu"]

        data_chatbot = []
        for i in range(1000):
            data_chatbot.append({
                "symptom": random.choice(symptoms),
                "patient_question": random.choice(questions),
                "doctor_reply": random.choice(responses),
                "language": random.choice(languages),
                "appointment_needed": random.choice(["Yes", "No"])
            })
        df_chatbot = pd.DataFrame(data_chatbot)
        df_chatbot.to_csv(chatbot_file_path, index=False)
    else:
        df_chatbot = pd.read_csv(chatbot_file_path)

    return df_synthetic, df_apriori, df_timeseries, df_feedback, df_clinical, df_chatbot

df_synthetic_raw, df_apriori_raw, df_timeseries_raw, df_feedback_raw, df_clinical_raw, df_chatbot_raw = load_all_data()


# --- Model Loading/Training (using st.cache_resource to avoid retraining on every rerun) ---
@st.cache_resource
def setup_module1_models(df_synthetic_raw_copy):
    df_synthetic_copy = df_synthetic_raw_copy.copy()

    # --- Preprocessing Pipeline --- (consistent across all three sub-modules)

    # 1. Feature Engineering
    df_synthetic_copy['BP_level'] = pd.cut(df_synthetic_copy['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
    df_synthetic_copy['medication_history'] = ((df_synthetic_copy['diabetes'] == 1) | (df_synthetic_copy['hypertension'] == 1) | (df_synthetic_copy['smoker'] == 1)).astype(int)

    # 2. Imputation
    imputer = SimpleImputer(strategy='median')
    num_cols_to_impute = ['bmi','systolic_bp','cholesterol','blood_glucose']
    df_synthetic_copy[num_cols_to_impute] = imputer.fit_transform(df_synthetic_copy[num_cols_to_impute])

    # 3. Outlier Removal (based on imputed numerical columns)
    initial_rows = len(df_synthetic_copy)
    for col in num_cols_to_impute:
        Q1 = df_synthetic_copy[col].quantile(0.25)
        Q3 = df_synthetic_copy[col].quantile(0.75)
        IQR = Q3 - Q1
        df_synthetic_copy = df_synthetic_copy[(df_synthetic_copy[col] >= Q1 - 1.5*IQR) & (df_synthetic_copy[col] <= Q3 + 1.5*IQR)]

    # 4. Label Encoding for categorical features
    le_synthetic_BP = LabelEncoder()
    df_synthetic_copy['BP_level'] = le_synthetic_BP.fit_transform(df_synthetic_copy['BP_level'])

    le_synthetic_risk = LabelEncoder()
    df_synthetic_copy['risk_category'] = le_synthetic_risk.fit_transform(df_synthetic_copy['risk_category'])
    risk_category_labels = le_synthetic_risk.inverse_transform(sorted(df_synthetic_copy['risk_category'].unique()))

    le_gender = LabelEncoder()
    if 'gender' in df_synthetic_copy.columns and df_synthetic_copy['gender'].dtype == 'object':
        df_synthetic_copy['gender'] = le_gender.fit_transform(df_synthetic_copy['gender'])
    else:
        le_gender = None # No gender encoder needed if gender column is absent or already numeric

    # Store processed DataFrame state after all these steps for direct feature extraction for models
    df_synthetic_processed_base = df_synthetic_copy.copy()


    # --- Model-specific Preparations and Training ---

    # 1. Classification Model (RandomForestClassifier)
    X_clf_all_features = df_synthetic_processed_base.drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
    y_clf = df_synthetic_processed_base['risk_category']

    scaler_clf = StandardScaler() # Scaler specifically for classification features before KBest
    X_clf_scaled_for_kbest = scaler_clf.fit_transform(X_clf_all_features)
    X_clf_scaled_for_kbest_df = pd.DataFrame(X_clf_scaled_for_kbest, columns=X_clf_all_features.columns, index=X_clf_all_features.index)

    selector = SelectKBest(score_func=f_classif, k=5)
    selector.fit(X_clf_scaled_for_kbest_df, y_clf)
    X_selected_clf = selector.transform(X_clf_scaled_for_kbest_df)

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_selected_clf, y_clf)
    clf_all_feature_names = list(X_clf_all_features.columns) # All features before selection
    clf_feature_names_after_select = list(X_clf_all_features.columns[selector.get_support(indices=True)])


    # 2. Regression Model (Ridge with Pipeline)
    features_for_reg = [col for col in df_synthetic_processed_base.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_reg_full = df_synthetic_processed_base[features_for_reg]
    y_reg = df_synthetic_processed_base["length_of_stay_days"]

    pipeline_reg = Pipeline([
        ("scaler", StandardScaler()), # This scaler will be fit on X_reg_full
        ("model", Ridge(alpha=1.0))
    ])
    pipeline_reg.fit(X_reg_full, y_reg)
    reg_feature_names = list(X_reg_full.columns)


    # 3. Clustering Model (KMeans)
    features_for_cluster = [col for col in df_synthetic_processed_base.columns if col not in ['patient_id', 'risk_category', 'length_of_stay_days']]
    X_cluster_full = df_synthetic_processed_base[features_for_cluster]

    scaler_cluster = StandardScaler() # Scaler specifically for clustering features
    X_cluster_scaled = scaler_cluster.fit_transform(X_cluster_full)

    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    cluster_feature_names = list(X_cluster_full.columns)

    return {
        "imputer": imputer,
        "le_synthetic_BP": le_synthetic_BP,
        "le_synthetic_risk": le_synthetic_risk,
        "le_gender": le_gender,
        "num_cols_to_impute": num_cols_to_impute,

        "clf_model": clf,
        "scaler_clf": scaler_clf,
        "selector_clf": selector,
        "clf_all_feature_names_before_select": clf_all_feature_names,
        "clf_feature_names_after_select": clf_feature_names_after_select,

        "reg_pipeline": pipeline_reg,
        "reg_feature_names": reg_feature_names,

        "kmeans_model": kmeans,
        "scaler_cluster": scaler_cluster,
        "cluster_feature_names": cluster_feature_names,

        "risk_category_labels": risk_category_labels,
        "df_synthetic_processed_for_metrics": df_synthetic_processed_base # For displaying metrics/summary
    }

module1_models = setup_module1_models(df_synthetic_raw.copy())

@st.cache_resource
def setup_module3_models(df_timeseries_raw_copy):
    df_timeseries_copy = df_timeseries_raw_copy.copy()
    FEATURES_ts = ['heart_rate', 'systolic_bp', 'spo2']
    TARGET_ts = 'risk_flag'
    SEQ_LENGTH = 10

    scaler_ts = MinMaxScaler()
    df_timeseries_copy[FEATURES_ts] = scaler_ts.fit_transform(df_timeseries_copy[FEATURES_ts])

    X_ts, y_ts = create_sequences(df_timeseries_copy[FEATURES_ts], df_timeseries_copy[TARGET_ts], SEQ_LENGTH)

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_ts.shape[1], X_ts.shape[2])))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(32))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(1, activation='sigmoid'))

    model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Train Model (on a subset or full for Streamlit demo)
    X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(X_ts, y_ts, test_size=0.01, random_state=42, stratify=y_ts) # Smaller test_size for faster loading
    history_lstm = model_lstm.fit(X_train_ts, y_train_ts, epochs=5, batch_size=32, validation_split=0.2, verbose=0) # Reduced epochs for faster load

    loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test_ts, y_test_ts, verbose=0)

    return {
        "model_lstm": model_lstm,
        "scaler_ts": scaler_ts,
        "SEQ_LENGTH": SEQ_LENGTH,
        "FEATURES_ts": FEATURES_ts,
        "LSTM_accuracy": accuracy_lstm,
        "X_test_ts": X_test_ts,
        "y_test_ts": y_test_ts # For demonstrating predictions
    }

module3_models = setup_module3_models(df_timeseries_raw.copy())

@st.cache_resource
def setup_module4_models(df_feedback_raw_copy):
    df_feedback_copy = df_feedback_raw_copy.copy()
    texts_feedback = df_feedback_copy["feedback_text"].tolist()
    le_feedback = LabelEncoder()
    labels_feedback = le_feedback.fit_transform(df_feedback_copy["sentiment"])

    tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
    encodings_feedback = tokenizer_bert(texts_feedback, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Use a small subset of the training data for faster Streamlit loading
    train_idx_feedback, _ = train_test_split(range(len(labels_feedback)), test_size=0.8, random_state=42, stratify=labels_feedback)

    train_dataset_feedback = FeedbackDataset(
        {k: v[train_idx_feedback] for k, v in encodings_feedback.items()},
        labels_feedback[train_idx_feedback]
    )

    model_sentiment = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(labels_feedback)))

    optimizer_sentiment = AdamW(model_sentiment.parameters(), lr=2e-5)
    model_sentiment.train()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_sentiment.to(device)

    # Only a few batches for quick demo on streamlit load
    train_loader = torch.utils.data.DataLoader(train_dataset_feedback, batch_size=8, shuffle=True)
    for epoch in range(1): # Only 1 epoch
        for i, batch in enumerate(train_loader):
            if i > 10: break # Only 10 batches for demo speed
            optimizer_sentiment.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs_sentiment = model_sentiment(**batch)
            loss_sentiment = outputs_sentiment.loss
            loss_sentiment.backward()
            optimizer_sentiment.step()

    model_sentiment.eval() # Set to eval mode after 'training'
    return {
        "tokenizer_bert": tokenizer_bert,
        "model_sentiment": model_sentiment,
        "le_feedback": le_feedback,
        "device": device
    }

module4_models = setup_module4_models(df_feedback_raw.copy())


@st.cache_resource
def setup_module5_models(df_clinical_raw_copy):
    generator_biogpt = None
    tokenizer_biobert = None
    model_biobert_embeddings = None
    kmeans_biobert = None
    df_clinical_processed = None
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if df_clinical_raw_copy is not None:
        df_clinical_processed = df_clinical_raw_copy.copy()
        df_clinical_processed["clinical_note"] = df_clinical_processed["clinical_note"].astype(str).apply(clean_text)

        # BioBERT for Embeddings
        MODEL_NAME_BIOBERT = "emilyalsentzer/Bio_ClinicalBERT"
        tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings = AutoModel.from_pretrained(MODEL_NAME_BIOBERT)
        model_biobert_embeddings.to(device)

        # Generate embeddings (limiting for demo speed to a sample if dataset is too large)
        sample_size = min(200, len(df_clinical_processed)) # Process a max of 200 notes
        if sample_size > 0:
            sample_df = df_clinical_processed.sample(sample_size, random_state=42) if len(df_clinical_processed) > sample_size else df_clinical_processed.copy()
            embeddings_biobert = np.vstack(sample_df["clinical_note"].apply(
                lambda x: get_embedding(x, tokenizer_biobert, model_biobert_embeddings).flatten()
            ))

            # KMeans Clustering on Embeddings
            kmeans_biobert = KMeans(n_clusters=5, random_state=42, n_init=10)
            kmeans_biobert.fit(embeddings_biobert)
            sample_df['cluster'] = kmeans_biobert.predict(embeddings_biobert)
            df_clinical_processed = df_clinical_processed.merge(sample_df[['patient_id', 'cluster']], on='patient_id', how='left')
        else:
            st.warning("No clinical notes available to generate embeddings or clusters.")
            tokenizer_biobert = None # Reset if no data to process
            model_biobert_embeddings = None
            kmeans_biobert = None


        # BioGPT for Text Generation
        try:
            generator_biogpt = pipeline(
                "text-generation",
                model="microsoft/BioGPT",
                device=0 if torch.cuda.is_available() else -1 # Use GPU if available
            )
        except Exception as e:
            st.error(f"Could not load BioGPT model: {e}. Text generation and chatbot functionality will be limited.")
            generator_biogpt = None
    else:
        st.warning("Clinical notes data not found, BioBERT and BioGPT models will not be fully functional for this module.")

    return {
        "tokenizer_biobert": tokenizer_biobert,
        "model_biobert_embeddings": model_biobert_embeddings,
        "kmeans_biobert": kmeans_biobert,
        "generator_biogpt": generator_biogpt,
        "df_clinical_processed": df_clinical_processed,
        "device": device
    }

module5_models = setup_module5_models(df_clinical_raw.copy() if df_clinical_raw is not None else None)


@st.cache_resource
def setup_module6_models():
    model_name_translator = "Helsinki-NLP/opus-mt-en-mul"
    tokenizer_translator = MarianTokenizer.from_pretrained(model_name_translator)
    translator_model = MarianMTModel.from_pretrained(model_name_translator)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    translator_model.to(device)

    return {
        "tokenizer_translator": tokenizer_translator,
        "translator_model": translator_model,
        "device": device
    }

module6_models = setup_module6_models()


# --- Streamlit Module Display Logic ---

if module_selection == "Module 1: Patient Data Analytics":
    st.header("Module 1: Synthetic Patient Data - Classification, Regression, Clustering")
    st.markdown("This module demonstrates predictive analytics and patient segmentation using synthetic patient data.")

    # --- Classification ---
    st.subheader("Patient Risk Category Classification")
    with st.expander("Model Metrics & Overview"):
        st.write("This model predicts a patient's risk category (Low, Medium, High). A Random Forest Classifier is used.")
        st.write(f"Random Forest Classifier trained on {len(module1_models['df_synthetic_processed_for_metrics'])} samples.")

        X_clf_full = module1_models['df_synthetic_processed_for_metrics'].drop(['risk_category','length_of_stay_days', 'patient_id'], axis=1, errors='ignore')
        y_clf_full = module1_models['df_synthetic_processed_for_metrics']['risk_category']

        # Scale X_clf_full using the fitted scaler_clf before applying selector
        X_clf_scaled_full = module1_models['scaler_clf'].transform(X_clf_full[module1_models['clf_all_feature_names_before_select']])
        X_selected_clf_full = module1_models['selector_clf'].transform(X_clf_scaled_full)

        y_pred_clf_full = module1_models['clf_model'].predict(X_selected_clf_full)
        st.write("Classification Report on full processed data:")
        st.text(classification_report(y_clf_full, y_pred_clf_full, target_names=module1_models['risk_category_labels']))

    st.markdown("### Predict New Patient Risk Category")
    col1, col2, col3 = st.columns(3)
    with col1:
        age_clf = st.number_input("Age", min_value=1, max_value=100, value=45, key='age_clf')
        bmi_clf = st.number_input("BMI", min_value=10.0, max_value=50.0, value=25.0, key='bmi_clf')
        systolic_bp_clf = st.number_input("Systolic BP", min_value=70, max_value=200, value=120, key='systolic_bp_clf')
    with col2:
        cholesterol_clf = st.number_input("Cholesterol", min_value=100, max_value=300, value=180, key='cholesterol_clf')
        blood_glucose_clf = st.number_input("Blood Glucose", min_value=70, max_value=200, value=90, key='blood_glucose_clf')
        diabetes_clf = st.checkbox("Diabetes", value=False, key='diabetes_clf')
        hypertension_clf = st.checkbox("Hypertension", value=False, key='hypertension_clf')
    with col3:
        smoker_clf = st.checkbox("Smoker", value=False, key='smoker_clf')
        prev_hospitalizations_clf = st.number_input("Previous Hospitalizations", min_value=0, max_value=10, value=0, key='prev_hospitalizations_clf')
        gender_clf = st.selectbox("Gender", ["Male", "Female", "Other"], key='gender_clf')

    if st.button("Predict Risk Category", key='predict_clf_btn'):
        new_patient_data = pd.DataFrame([{
            'age': age_clf,
            'bmi': bmi_clf,
            'systolic_bp': systolic_bp_clf,
            'cholesterol': cholesterol_clf,
            'blood_glucose': blood_glucose_clf,
            'diabetes': int(diabetes_clf),
            'hypertension': int(hypertension_clf),
            'smoker': int(smoker_clf),
            'prev_hospitalizations': prev_hospitalizations_clf,
            'gender': gender_clf
        }])

        # --- Preprocessing for new patient data (mirroring setup_module1_models) ---
        # 1. Feature Engineering
        new_patient_data['BP_level'] = pd.cut(new_patient_data['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
        new_patient_data['medication_history'] = ((new_patient_data['diabetes'] == 1) | (new_patient_data['hypertension'] == 1) | (new_patient_data['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_data[module1_models['num_cols_to_impute']] = module1_models['imputer'].transform(new_patient_data[module1_models['num_cols_to_impute']])

        # 3. Outlier removal is tricky for single instances, typically skipped or handled by robust scaling. Not applied here for simplicity.

        # 4. Label Encoding (only transform, not fit)
        new_patient_data['BP_level'] = module1_models['le_synthetic_BP'].transform(new_patient_data['BP_level'])
        if module1_models['le_gender'] is not None and 'gender' in new_patient_data.columns and new_patient_data['gender'].dtype == 'object':
            try:
                new_patient_data['gender'] = module1_models['le_gender'].transform(new_patient_data['gender'])
            except ValueError:
                # Handle unseen gender category, e.g., default to 0 or mean, or raise error.
                # For demo, let's just make it the most common category or raise an error.
                st.error("Unseen gender category in new patient data. Please use 'Male', 'Female', or 'Other'.")
                st.stop()
        elif 'gender' in new_patient_data.columns and new_patient_data['gender'].dtype == 'object': # If gender existed but no encoder was fit
            new_patient_data['gender'] = LabelEncoder().fit_transform(new_patient_data['gender'])

        # Align columns with training data used for classification's selector
        # Ensure all features expected by the scaler and selector are present, in correct order.
        new_patient_processed_aligned = pd.DataFrame(columns=module1_models['clf_all_feature_names_before_select'])
        for col in module1_models['clf_all_feature_names_before_select']:
            if col in new_patient_data.columns:
                new_patient_processed_aligned[col] = new_patient_data[col]
            else:
                new_patient_processed_aligned[col] = 0 # Default value for any missing feature, or handle with mean/median

        # Scale features using the fitted scaler_clf
        new_patient_scaled = module1_models['scaler_clf'].transform(new_patient_processed_aligned)

        # Apply feature selection
        new_patient_selected = module1_models['selector_clf'].transform(new_patient_scaled)

        prediction = module1_models['clf_model'].predict(new_patient_selected)
        predicted_risk = module1_models['le_synthetic_risk'].inverse_transform(prediction)
        st.success(f"Predicted Risk Category: **{predicted_risk[0]}**")


    # --- Regression ---
    st.subheader("Patient Length of Stay Regression")
    with st.expander("Model Metrics & Overview"):
        st.write("This model predicts the length of stay in days for a patient using a Ridge Regression model.")

        # Predict on the full processed data used for training to get metrics
        X_reg_full_for_metrics = module1_models['df_synthetic_processed_for_metrics'][module1_models['reg_feature_names']]
        y_reg_full_for_metrics = module1_models['df_synthetic_processed_for_metrics']['length_of_stay_days']
        y_pred_reg_full = module1_models['reg_pipeline'].predict(X_reg_full_for_metrics)

        mae = mean_absolute_error(y_reg_full_for_metrics, y_pred_reg_full)
        rmse = np.sqrt(mean_squared_error(y_reg_full_for_metrics, y_pred_reg_full))
        r2 = r2_score(y_reg_full_for_metrics, y_pred_reg_full)
        st.write(f"MAE on full processed data: {mae:.2f}")
        st.write(f"RMSE on full processed data: {rmse:.2f}")
        st.write(f"R2 Score on full processed data: {r2:.2f}")

    st.markdown("### Predict New Patient Length of Stay")
    col1_reg, col2_reg, col3_reg = st.columns(3)
    with col1_reg:
        age_reg = st.number_input("Age (Regression)", min_value=1, max_value=100, value=45, key='age_reg')
        bmi_reg = st.number_input("BMI (Regression)", min_value=10.0, max_value=50.0, value=25.0, key='bmi_reg')
        systolic_bp_reg = st.number_input("Systolic BP (Regression)", min_value=70, max_value=200, value=120, key='systolic_bp_reg')
    with col2_reg:
        cholesterol_reg = st.number_input("Cholesterol (Regression)", min_value=100, max_value=300, value=180, key='cholesterol_reg')
        blood_glucose_reg = st.number_input("Blood Glucose (Regression)", min_value=70, max_value=200, value=90, key='blood_glucose_reg')
        diabetes_reg = st.checkbox("Diabetes (Regression)", value=False, key='diabetes_reg')
        hypertension_reg = st.checkbox("Hypertension (Regression)", value=False, key='hypertension_reg')
    with col3_reg:
        smoker_reg = st.checkbox("Smoker (Regression)", value=False, key='smoker_reg')
        prev_hospitalizations_reg = st.number_input("Previous Hospitalizations (Regression)", min_value=0, max_value=10, value=0, key='prev_hospitalizations_reg')
        gender_reg = st.selectbox("Gender (Regression)", ["Male", "Female", "Other"], key='gender_reg')

    if st.button("Predict Length of Stay", key='predict_reg_btn'):
        new_patient_data_reg = pd.DataFrame([{
            'age': age_reg,
            'bmi': bmi_reg,
            'systolic_bp': systolic_bp_reg,
            'cholesterol': cholesterol_reg,
            'blood_glucose': blood_glucose_reg,
            'diabetes': int(diabetes_reg),
            'hypertension': int(hypertension_reg),
            'smoker': int(smoker_reg),
            'prev_hospitalizations': prev_hospitalizations_reg,
            'gender': gender_reg
        }])

        # --- Preprocessing for new patient data (mirroring setup_module1_models) ---
        # 1. Feature Engineering
        new_patient_data_reg['BP_level'] = pd.cut(new_patient_data_reg['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
        new_patient_data_reg['medication_history'] = ((new_patient_data_reg['diabetes'] == 1) | (new_patient_data_reg['hypertension'] == 1) | (new_patient_data_reg['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_data_reg[module1_models['num_cols_to_impute']] = module1_models['imputer'].transform(new_patient_data_reg[module1_models['num_cols_to_impute']])

        # 3. Label Encoding (only transform, not fit)
        new_patient_data_reg['BP_level'] = module1_models['le_synthetic_BP'].transform(new_patient_data_reg['BP_level'])
        if module1_models['le_gender'] is not None and 'gender' in new_patient_data_reg.columns and new_patient_data_reg['gender'].dtype == 'object':
            try:
                new_patient_data_reg['gender'] = module1_models['le_gender'].transform(new_patient_data_reg['gender'])
            except ValueError:
                st.error("Unseen gender category in new patient data. Please use 'Male', 'Female', or 'Other'.")
                st.stop()
        elif 'gender' in new_patient_data_reg.columns and new_patient_data_reg['gender'].dtype == 'object':
            new_patient_data_reg['gender'] = LabelEncoder().fit_transform(new_patient_data_reg['gender'])

        # Align columns with regression model's expected features
        new_patient_reg_aligned = new_patient_data_reg[module1_models['reg_feature_names']] # Ensure correct column order

        prediction_reg = module1_models['reg_pipeline'].predict(new_patient_reg_aligned)
        st.success(f"Predicted Length of Stay: **{prediction_reg[0]:.2f} days**")


    # --- Clustering ---
    st.subheader("Patient Clustering")
    with st.expander("Model Metrics & Overview"):
        st.write("This model segments patients into 3 clusters based on their features using K-Means. Silhouette Score indicates cluster density and separation.")

        # Prepare data for Silhouette Score calculation using the dedicated scaler_cluster
        X_cluster_full_for_metrics = module1_models['df_synthetic_processed_for_metrics'][module1_models['cluster_feature_names']]
        X_cluster_scaled_for_metrics = module1_models['scaler_cluster'].transform(X_cluster_full_for_metrics)

        silhouette_score_val = silhouette_score(X_cluster_scaled_for_metrics, module1_models['kmeans_model'].labels_)
        st.write(f"Silhouette Score: {silhouette_score_val:.2f}")
        st.write("Crosstabulation of Cluster vs. Risk Category (shows how clusters align with risk):")

        # Merge clusters back to original processed df for crosstab
        df_for_crosstab = module1_models['df_synthetic_processed_for_metrics'].copy()
        df_for_crosstab['cluster'] = module1_models['kmeans_model'].labels_ # Assign labels from the model
        st.dataframe(pd.crosstab(df_for_crosstab['cluster'], df_for_crosstab['risk_category'].map(lambda x: module1_models['le_synthetic_risk'].inverse_transform([x])[0])))

    st.markdown("### Assign New Patient to a Cluster")
    col1_cluster, col2_cluster, col3_cluster = st.columns(3)
    with col1_cluster:
        age_cluster = st.number_input("Age (Clustering)", min_value=1, max_value=100, value=45, key='age_cluster')
        bmi_cluster = st.number_input("BMI (Clustering)", min_value=10.0, max_value=50.0, value=25.0, key='bmi_cluster')
        systolic_bp_cluster = st.number_input("Systolic BP (Clustering)", min_value=70, max_value=200, value=120, key='systolic_bp_cluster')
    with col2_cluster:
        cholesterol_cluster = st.number_input("Cholesterol (Clustering)", min_value=100, max_value=300, value=180, key='cholesterol_cluster')
        blood_glucose_cluster = st.number_input("Blood Glucose (Clustering)", min_value=70, max_value=200, value=90, key='blood_glucose_cluster')
        diabetes_cluster = st.checkbox("Diabetes (Clustering)", value=False, key='diabetes_cluster')
        hypertension_cluster = st.checkbox("Hypertension (Clustering)", value=False, key='hypertension_cluster')
    with col3_cluster:
        smoker_cluster = st.checkbox("Smoker (Clustering)", value=False, key='smoker_cluster')
        prev_hospitalizations_cluster = st.number_input("Previous Hospitalizations (Clustering)", min_value=0, max_value=10, value=0, key='prev_hospitalizations_cluster')
        gender_cluster = st.selectbox("Gender (Clustering)", ["Male", "Female", "Other"], key='gender_cluster')

    if st.button("Assign Cluster", key='assign_cluster_btn'):
        new_patient_data_cluster = pd.DataFrame([{
            'age': age_cluster,
            'bmi': bmi_cluster,
            'systolic_bp': systolic_bp_cluster,
            'cholesterol': cholesterol_cluster,
            'blood_glucose': blood_glucose_cluster,
            'diabetes': int(diabetes_cluster),
            'hypertension': int(hypertension_cluster),
            'smoker': int(smoker_cluster),
            'prev_hospitalizations': prev_hospitalizations_cluster,
            'gender': gender_cluster
        }])

        # --- Preprocessing for new patient data (mirroring setup_module1_models) ---
        # 1. Feature Engineering
        new_patient_data_cluster['BP_level'] = pd.cut(new_patient_data_cluster['systolic_bp'], bins=[0, 80, 120, 200], labels=['Low', 'Normal', 'High'], ordered=False)
        new_patient_data_cluster['medication_history'] = ((new_patient_data_cluster['diabetes'] == 1) | (new_patient_data_cluster['hypertension'] == 1) | (new_patient_data_cluster['smoker'] == 1)).astype(int)

        # 2. Imputation (only transform, not fit)
        new_patient_data_cluster[module1_models['num_cols_to_impute']] = module1_models['imputer'].transform(new_patient_data_cluster[module1_models['num_cols_to_impute']])

        # 3. Label Encoding (only transform, not fit)
        new_patient_data_cluster['BP_level'] = module1_models['le_synthetic_BP'].transform(new_patient_data_cluster['BP_level'])
        if module1_models['le_gender'] is not None and 'gender' in new_patient_data_cluster.columns and new_patient_data_cluster['gender'].dtype == 'object':
            try:
                new_patient_data_cluster['gender'] = module1_models['le_gender'].transform(new_patient_data_cluster['gender'])
            except ValueError:
                st.error("Unseen gender category in new patient data. Please use 'Male', 'Female', or 'Other'.")
                st.stop()
        elif 'gender' in new_patient_data_cluster.columns and new_patient_data_cluster['gender'].dtype == 'object':
            new_patient_data_cluster['gender'] = LabelEncoder().fit_transform(new_patient_data_cluster['gender'])

        # Align columns with clustering model's expected features
        new_patient_cluster_aligned = new_patient_data_cluster[module1_models['cluster_feature_names']] # Ensure correct column order

        # Scale features using the fitted scaler_cluster
        new_patient_cluster_scaled = module1_models['scaler_cluster'].transform(new_patient_cluster_aligned)

        predicted_cluster = module1_models['kmeans_model'].predict(new_patient_cluster_scaled)
        st.success(f"Assigned Cluster: **{predicted_cluster[0]}**")


elif module_selection == "Module 2: Association Rules":
    st.header("Module 2: Association Rules for Medical Data")
    st.markdown("This module uncovers relationships between medical conditions and procedures using association rule mining.")

    df_assoc = df_apriori_raw.drop('patient_id', axis=1)
    df_assoc_bool = df_assoc.astype(bool)

    st.sidebar.subheader("Association Rules Parameters")
    min_support = st.sidebar.slider("Minimum Support", 0.01, 1.0, 0.1, 0.01)
    min_confidence = st.sidebar.slider("Minimum Confidence", 0.01, 1.0, 0.6, 0.01)

    # Generate frequent itemsets
    try:
        frequent_itemsets = apriori(
            df_assoc_bool,
            min_support=min_support,
            use_colnames=True
        )
        st.subheader("Frequent Itemsets")
        st.dataframe(frequent_itemsets.sort_values('support', ascending=False).head(10))

        # Generate association rules
        rules = association_rules(
            frequent_itemsets,
            metric='confidence',
            min_threshold=min_confidence
        )
        st.subheader("Association Rules")
        if not rules.empty:
            st.dataframe(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(10))
        else:
            st.info("No association rules found with the current parameters.")

    except Exception as e:
        st.error(f"An error occurred while generating association rules: {e}")

    st.markdown("""
        **How to interpret:**
        *   **Antecedents**: The item(s) on the left-hand side of the rule (IF these conditions are met).
        *   **Consequents**: The item(s) on the right-hand side of the rule (THEN these conditions are likely).
        *   **Support**: How frequently the itemset (antecedents + consequents) appears in the dataset.
        *   **Confidence**: How often the consequent appears given the antecedent.
        *   **Lift**: How much more likely the consequent is given the antecedent, relative to its baseline probability. Lift > 1 indicates a positive correlation.
    """)


elif module_selection == "Module 3: Sequence Modeling (LSTM)":
    st.header("Module 3: Patient Deterioration Risk Prediction (LSTM)")
    st.markdown("This module uses LSTM neural networks to predict patient deterioration risk based on time-series vital signs.")

    with st.expander("Model Metrics & Overview"):
        st.write("LSTM Model Summary:")
        st.markdown("```python\nmodel_lstm.summary() # Output omitted for brevity in Streamlit\n```")
        st.write("Model compiled with Adam optimizer, binary crossentropy loss, and accuracy metrics.")
        st.write(f"Test Accuracy: {module3_models['LSTM_accuracy']:.4f}")
        st.write(f"Sequence Length used: {module3_models['SEQ_LENGTH']}")
        st.write(f"Features used: {', '.join(module3_models['FEATURES_ts'])}")

    st.subheader("Predict Deterioration Risk for New Vitals Sequence")
    st.write(f"Input the last {module3_models['SEQ_LENGTH']} readings for Heart Rate, Systolic BP, and SpO2.")

    new_vitals_input = []
    for i in range(module3_models['SEQ_LENGTH']):
        st.markdown(f"**Time Step {i+1}** (t-{module3_models['SEQ_LENGTH']-1-i})")
        col_ts1, col_ts2, col_ts3 = st.columns(3)
        with col_ts1:
            hr = st.number_input(f"Heart Rate", min_value=40, max_value=180, value=75, key=f"hr_{i}")
        with col_ts2:
            sbp = st.number_input(f"Systolic BP", min_value=60, max_value=200, value=120, key=f"sbp_{i}")
        with col_ts3:
            spo2 = st.number_input(f"SpO2", min_value=70, max_value=100, value=98, key=f"spo2_{i}")
        new_vitals_input.append([hr, sbp, spo2])

    if st.button("Predict Deterioration", key='predict_lstm_btn'):
        new_vitals_df = pd.DataFrame(new_vitals_input, columns=module3_models['FEATURES_ts'])
        new_vitals_scaled = module3_models['scaler_ts'].transform(new_vitals_df)

        new_vitals_reshaped = new_vitals_scaled.reshape(1, module3_models['SEQ_LENGTH'], len(module3_models['FEATURES_ts']))

        prediction_prob = module3_models['model_lstm'].predict(new_vitals_reshaped, verbose=0)[0][0]
        prediction_risk = "High deterioration risk" if prediction_prob > 0.5 else "Stable"

        st.success(f"Predicted Deterioration Risk: **{prediction_risk}** (Probability: {prediction_prob:.2f})")
        st.caption("0: Stable, 1: High deterioration risk")

    with st.expander("Example LSTM Predictions (from test set)"):
        if len(module3_models['X_test_ts']) > 0:
            y_pred_prob_lstm_sample = module3_models['model_lstm'].predict(module3_models['X_test_ts'][:5], verbose=0)
            y_pred_lstm_sample = (y_pred_prob_lstm_sample > 0.5).astype(int).flatten()
            st.write(f"Actual (y_test): {module3_models['y_test_ts'][:5].flatten()}")
            st.write(f"Predicted: {y_pred_lstm_sample}")
            st.caption("0=Stable, 1=High deterioration risk")
        else:
            st.info("No test set samples available for display.")


elif module_selection == "Module 4: Sentiment Analysis (BERT)":
    st.header("Module 4: Patient Feedback Sentiment Analysis (BERT)")
    st.markdown("This module uses a fine-tuned BERT model to analyze the sentiment of patient feedback (Negative, Neutral, Positive).")

    with st.expander("Model Overview"):
        st.write("BERT-base-uncased model fine-tuned for sentiment classification (Negative, Neutral, Positive).")
        st.write(f"Sentiment labels: {list(module4_models['le_feedback'].classes_)}")
        st.write("Due to Streamlit caching, training is simplified/reduced for quick demonstration. Model is loaded in evaluation mode.")

    st.subheader("Analyze New Patient Feedback")
    user_feedback = st.text_area("Enter patient feedback here:", "The nurses were very kind and attentive, but wait times were long.", key='feedback_input')

    if st.button("Analyze Sentiment", key='analyze_sentiment_btn'):
        inputs_sentiment = module4_models['tokenizer_bert'](user_feedback, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs_sentiment = {k: v.to(module4_models['device']) for k, v in inputs_sentiment.items()}

        with torch.no_grad():
            outputs_sentiment_pred = module4_models['model_sentiment'](**inputs_sentiment)

        pred_sentiment_idx = torch.argmax(outputs_sentiment_pred.logits, dim=1).item()
        predicted_sentiment = module4_models['le_feedback'].inverse_transform([pred_sentiment_idx])[0]

        st.success(f"Predicted Sentiment: **{predicted_sentiment}**")
        st.info(f"Raw prediction index: {pred_sentiment_idx}")

    with st.expander("Sample Data & Labels"):
        st.dataframe(df_feedback_raw.head())


elif module_selection == "Module 5: Generative AI (BioBERT & BioGPT)":
    st.header("Module 5: Generative AI for Clinical Notes")
    st.markdown("This module explores BioBERT for generating embeddings from clinical notes and BioGPT for generating clinical text.")

    if module5_models["df_clinical_processed"] is None or module5_models["kmeans_biobert"] is None:
        st.error("Clinical notes data not found or models could not be initialized. This module cannot be fully demonstrated.")
    else:
        st.subheader("BioBERT Embeddings and Clustering of Clinical Notes")
        with st.expander("Overview"):
            st.write("BioBERT (a BERT model trained on biomedical text) is used to create numerical representations (embeddings) of clinical notes. K-Means clustering then groups similar notes.")
            st.write(f"Embeddings generated for a sample of {module5_models['kmeans_biobert'].n_samples_fit_} clinical notes.")
            st.write("Sample clinical notes with assigned clusters:")
            st.dataframe(module5_models["df_clinical_processed"][['clinical_note', 'cluster']].dropna().head())

        st.subheader("BioGPT for Clinical Text Generation")
        if module5_models["generator_biogpt"]:
            st.write("BioGPT (a large language model for biology and medicine) can generate plausible clinical text based on a given prompt.")
            prompt_biogpt_input = st.text_area(
                "Enter a prompt for BioGPT clinical text generation:",
                "Patient presents with chest pain and shortness of breath. Clinical impression:",
                height=100,
                key='biogpt_prompt'
            )
            max_length_biogpt = st.slider("Max Length for Generation", 50, 200, 80, key='biogpt_max_len')

            if st.button("Generate Clinical Text", key='generate_biogpt_btn'):
                with st.spinner("Generating..."):
                    generated_text_biogpt = module5_models["generator_biogpt"](
                        prompt_biogpt_input,
                        max_length=max_length_biogpt,
                        num_return_sequences=1,
                        pad_token_id=module5_models["generator_biogpt"].tokenizer.eos_token_id # Prevents warning
                    )[0]["generated_text"]
                st.info(generated_text_biogpt)
        else:
            st.warning("BioGPT model not loaded. Text generation functionality is disabled. Check error messages above for details.")

elif module_selection == "Module 6: Chatbot & Translator":
    st.header("Module 6: Healthcare Chatbot and Translator")
    st.markdown("This module provides an AI-powered healthcare chatbot and a medical text translator.")

    def healthcare_chatbot_streamlit(user_input):
        """Chatbot function using the BioGPT model for medical guidance, or a generic response."""
        if module5_models["generator_biogpt"]:
            prompt = f"""
            You are a healthcare assistant. Provide safe medical guidance and symptom triage. Avoid medical diagnosis. Focus on general health advice. If you cannot provide a specific answer, advise consulting a doctor. Do not generate information that is not directly related to the prompt. Limit your response to 100 words.
            Patient says: {user_input}
            Response:
            """
            response = module5_models["generator_biogpt"](prompt, max_length=150, num_return_sequences=1, pad_token_id=module5_models["generator_biogpt"].tokenizer.eos_token_id)[0]["generated_text"]

            # Clean up the response to remove the prompt part if BioGPT echoes it
            response_lines = response.split('\n')
            clean_response = []
            capture = False
            for line in response_lines:
                if line.strip().startswith("Response:"):
                    capture = True
                    continue
                if capture and line.strip():
                    clean_response.append(line.strip())

            if clean_response:
                return " ".join(clean_response)
            else:
                return response.replace(prompt, "").strip() # Fallback cleanup if parsing fails
        else:
            return f"Hello! As a healthcare assistant, I recommend consulting a doctor for '{user_input}'. Please note that a specialized AI model for medical guidance is currently unavailable due to BioGPT loading issues. Always consult a qualified medical professional for health concerns."

    def translate_medical_text_streamlit(text):
        """Translates medical text using the MarianMT model."""
        inputs = module6_models['tokenizer_translator'](text, return_tensors="pt", padding=True)
        inputs = {k: v.to(module6_models['device']) for k, v in inputs.items()}
        translated = module6_models['translator_model'].generate(**inputs)
        return module6_models['tokenizer_translator'].decode(translated[0], skip_special_tokens=True)

    st.subheader("Healthcare Chatbot")
    user_question = st.text_area("Ask a medical question to the AI healthcare assistant:", "I have a persistent cough, what could it be?", key='chatbot_q')

    if st.button("Get Chatbot Response", key='chatbot_btn'):
        with st.spinner("Generating response..."):
            chatbot_reply = healthcare_chatbot_streamlit(user_question)
        st.info(chatbot_reply)

    st.subheader("Medical Text Translator")
    text_to_translate = st.text_area("Enter medical text to translate:", "The patient has acute appendicitis and requires immediate surgery.", key='translator_input')

    st.info("Note: The current translation model (Helsinki-NLP/opus-mt-en-mul) translates English text into a multilingual interpretation. For specific target languages (e.g., English to Tamil), a different dedicated model would be required.")

    if st.button("Translate Text", key='translate_btn'):
        with st.spinner("Translating..."):
            translated_text = translate_medical_text_streamlit(text_to_translate)
        st.success(f"Translated Text (Multilingual interpretation): **{translated_text}**")
        st.caption("This translation attempts to provide a general multilingual equivalent.")

    st.subheader("Combined Chatbot and Translator Example")
    combined_user_question = st.text_area("Ask a question for both chatbot and translation:", "My child has a fever and is not eating well. Should I be worried?", key='combined_q')
    target_language_label = st.selectbox("Simulated Target Language (for display only):", ["English", "Tamil", "Hindi", "Spanish", "French"], key='target_lang_select')

    if st.button("Get Combined Response", key='combined_btn'):
        with st.spinner("Processing..."):
            english_reply = healthcare_chatbot_streamlit(combined_user_question)
            translated_reply = translate_medical_text_streamlit(english_reply)

        st.markdown("---")
        st.write("**English Chatbot Response:**")
        st.info(english_reply)
        st.write(f"**Translated Response (Simulated {target_language_label} via Multilingual Model):**")
        st.success(translated_reply)
        st.caption("Note: The translation model provides a multilingual interpretation, not a direct translation to the selected language label.")

Writing streamlit_app.py


## Explain Deployment Steps

### Subtask:
Provide instructions to the user on how to install Streamlit, run the app, and optionally use ngrok in Colab to get a temporary public URL for the dashboard, or deploy to Streamlit Cloud for persistent hosting.


## Summary:

### Data Analysis Key Findings
*   A comprehensive Streamlit application, `streamlit_app.py`, was successfully generated. This application integrates six distinct AI modules for healthcare data analysis and interaction.
*   The dashboard leverages Streamlit's caching mechanisms (`st.cache_data` and `st.cache_resource`) to optimize performance by loading data and models only once.
*   The application includes modules for:
    *   **Patient Data Analytics**: Classification (RandomForestClassifier for risk categories), Regression (Ridge model for length of stay), and Clustering (K-Means for patient segmentation). Metrics like Classification Report, MAE, RMSE, R2 Score, and Silhouette Score are displayed.
    *   **Association Rules**: Identifies relationships between medical conditions and procedures using Apriori algorithm with configurable support and confidence thresholds.
    *   **Sequence Modeling (LSTM)**: Predicts patient deterioration risk from time-series vital signs (heart rate, systolic BP, SpO2) with a reported test accuracy of approximately 88% after a reduced training cycle for demonstration.
    *   **Sentiment Analysis (BERT)**: Analyzes patient feedback sentiment (Negative, Neutral, Positive) using a fine-tuned BERT model.
    *   **Generative AI (BioBERT & BioGPT)**: Utilizes BioBERT for generating embeddings and clustering clinical notes, and BioGPT for generating clinical text based on prompts.
    *   **Chatbot & Translator**: Provides a healthcare chatbot powered by BioGPT and a medical text translator using a MarianMT model, offering multilingual interpretations.
*   The generated application code includes robust preprocessing steps within each module's setup, such as imputation, outlier handling, and scaling, ensuring data readiness for model inference.

### Insights or Next Steps
*   The generated `streamlit_app.py` provides a ready-to-deploy, multi-functional AI dashboard that can be immediately utilized for demonstrating complex healthcare AI capabilities.
*   For persistent hosting and production-scale use, deploying the application to Streamlit Community Cloud or similar platforms is recommended, with careful consideration for handling large models and sensitive API keys as outlined in the deployment instructions.


In [2]:
!pip install -q streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.1 MB/s eta 0:00:00
